In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# 导入数据
print("正在导入数据...")
train_price = pd.read_csv('ruc_Class25Q2_train_price.csv')
test_price = pd.read_csv('ruc_Class25Q2_test_price.csv')
train_rent = pd.read_csv('ruc_Class25Q2_train_rent.csv')
test_rent = pd.read_csv('ruc_Class25Q2_test_rent.csv')

print(f"房价训练集形状: {train_price.shape}")
print(f"房价测试集形状: {test_price.shape}")
print(f"租金训练集形状: {train_rent.shape}")
print(f"租金测试集形状: {test_rent.shape}")

print("\n数据导入完成！")

正在导入数据...
房价训练集形状: (103871, 55)
房价测试集形状: (34017, 55)
租金训练集形状: (98899, 46)
租金测试集形状: (9773, 46)

数据导入完成！


In [6]:
print(train_price.columns)
print(train_rent.columns)
print(test_price.columns)
print(test_rent.columns)

Index(['城市', '区域', '板块', '环线', 'Price', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向',
       '建筑结构', '装修情况', '梯户比例', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途',
       '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon',
       'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数',
       '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm', '物业办公电话',
       '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x',
       'coord_y', '客户反馈'],
      dtype='object')
Index(['城市', '户型', '装修', 'Price', '楼层', '面积', '朝向', '交易时间', '付款方式', '租赁方式',
       '电梯', '车位', '用水', '用电', '燃气', '采暖', '租期', '配套设施', 'lon', 'lat', '年份',
       '区县', '板块', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司',
       '绿 化 率', '容 积 率', '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电',
       '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈'],
      dtype='object')
Index(['ID', '城市', '区域', '板块', '环线', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向',
       '建筑结构', '装修情况'

In [2]:
# 检查城市列的分布
print("房价训练集城市分布:")
print(train_price['城市'].value_counts().sort_index())

print("\n房价测试集城市分布:")
print(test_price['城市'].value_counts().sort_index())

print("\n租金训练集城市分布:")
print(train_rent['城市'].value_counts().sort_index())

print("\n租金测试集城市分布:")
print(test_rent['城市'].value_counts().sort_index())

房价训练集城市分布:
城市
0     16491
1      6437
2     24996
3     21472
4      4363
5      3582
6      2281
7      1184
8      5931
9      1323
10    15057
11      754
Name: count, dtype: int64

房价测试集城市分布:
城市
0     7057
1     1985
2     5431
3     3961
4     5602
5     1057
6      933
7      879
8     2051
9     1182
10    3694
11     185
Name: count, dtype: int64

租金训练集城市分布:
城市
0     14542
1      7725
2     12059
3     15550
4     11796
5      4077
6      1020
7     10211
8      7274
9      1255
10    12979
11      411
Name: count, dtype: int64

租金测试集城市分布:
城市
0     1512
1      567
2      936
3     1802
4     1168
5      592
6      188
7      837
8      668
9      340
10    1070
11      93
Name: count, dtype: int64


In [3]:
# 处理城市列 - 创建哑变量
all_cities = set()
all_cities.update(train_price['城市'].unique())
all_cities.update(test_price['城市'].unique())
all_cities.update(train_rent['城市'].unique())
all_cities.update(test_rent['城市'].unique())
all_cities = sorted(list(all_cities))
print(f"发现的城市类别: {all_cities}")


发现的城市类别: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [4]:
# 为每个数据集创建城市哑变量
def create_city_dummies(df, city_list, prefix='城市'):
    df_copy = df.copy()
    for city in city_list:
        df_copy[f'{prefix}_{city}'] = (df_copy['城市'] == city).astype(int)
    df_copy = df_copy.drop('城市', axis=1)
    return df_copy

# 应用到所有数据集
train_price_processed = create_city_dummies(train_price, all_cities)
test_price_processed = create_city_dummies(test_price, all_cities)
train_rent_processed = create_city_dummies(train_rent, all_cities)
test_rent_processed = create_city_dummies(test_rent, all_cities)

city_columns = [col for col in train_price_processed.columns if col.startswith('城市_')]
print(f"\n新增的城市哑变量列: {city_columns}")


新增的城市哑变量列: ['城市_0', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6', '城市_7', '城市_8', '城市_9', '城市_10', '城市_11']


In [5]:
# 验证哑变量创建是否正确
print("验证城市哑变量创建结果...")

# 检查房价训练集的城市哑变量
city_cols = [col for col in train_price_processed.columns if col.startswith('城市_')]
print(f"\n房价训练集城市哑变量统计:")
for col in city_cols:
    count = train_price_processed[col].sum()
    print(f"{col}: {count} 条记录")

# 验证每行只有一个城市被标记为1
city_sum_per_row = train_price_processed[city_cols].sum(axis=1)
print(f"\n每行城市哑变量之和的统计:")
print(city_sum_per_row.value_counts())

if (city_sum_per_row == 1).all():
    print("✓ 验证通过：每行都有且仅有一个城市被标记")
else:
    print("✗ 验证失败：存在行没有城市标记或有多个城市标记")

# 显示前几行的城市哑变量
print("\n前5行的城市哑变量:")
print(train_price_processed[city_cols].head())

验证城市哑变量创建结果...

房价训练集城市哑变量统计:
城市_0: 16491 条记录
城市_1: 6437 条记录
城市_2: 24996 条记录
城市_3: 21472 条记录
城市_4: 4363 条记录
城市_5: 3582 条记录
城市_6: 2281 条记录
城市_7: 1184 条记录
城市_8: 5931 条记录
城市_9: 1323 条记录
城市_10: 15057 条记录
城市_11: 754 条记录

每行城市哑变量之和的统计:
1    103871
Name: count, dtype: int64
✓ 验证通过：每行都有且仅有一个城市被标记

前5行的城市哑变量:
   城市_0  城市_1  城市_2  城市_3  城市_4  城市_5  城市_6  城市_7  城市_8  城市_9  城市_10  城市_11
0     1     0     0     0     0     0     0     0     0     0      0      0
1     1     0     0     0     0     0     0     0     0     0      0      0
2     1     0     0     0     0     0     0     0     0     0      0      0
3     1     0     0     0     0     0     0     0     0     0      0      0
4     1     0     0     0     0     0     0     0     0     0      0      0


In [7]:
train_price['房屋户型'].unique()


array(['2室1厅1厨1卫', '3室1厅1厨1卫', '3室2厅1厨2卫', '6室3厅1厨3卫', '1房间1卫',
       '5室2厅1厨4卫', nan, '3室1厅1厨2卫', '5室3厅1厨4卫', '3室2厅1厨3卫', '1室1厅1厨1卫',
       '2房间1卫', '4室3厅1厨2卫', '3房间1卫', '4室2厅1厨2卫', '6房间3卫', '2室2厅1厨2卫',
       '1室1厅1厨2卫', '4室1厅1厨1卫', '2室1厅1厨2卫', '5室2厅1厨3卫', '7房间3卫', '3房间2卫',
       '4室3厅1厨3卫', '2室2厅1厨1卫', '5房间2卫', '7房间4卫', '7室2厅1厨3卫', '4室1厅1厨2卫',
       '2室1厅0厨1卫', '3室1厅0厨1卫', '4室2厅1厨3卫', '2房间2卫', '4室3厅1厨6卫',
       '1室0厅1厨1卫', '5房间1卫', '8室3厅1厨3卫', '3室2厅1厨1卫', '6室3厅1厨4卫', '1房间0卫',
       '2室1厅1厨3卫', '2室1厅0厨0卫', '3室1厅0厨0卫', '4房间2卫', '4室2厅1厨5卫', '6房间2卫',
       '6室4厅1厨4卫', '2室2厅1厨3卫', '8房间3卫', '4室2厅1厨4卫', '2室0厅1厨1卫',
       '1室1厅0厨1卫', '5房间3卫', '4室2厅2厨3卫', '5室3厅1厨3卫', '6室2厅1厨3卫',
       '7室3厅2厨4卫', '4室3厅1厨4卫', '6室4厅1厨3卫', '5室3厅1厨2卫', '6室4厅1厨5卫',
       '1室0厅0厨0卫', '4室2厅2厨6卫', '8室4厅3厨4卫', '5室4厅1厨6卫', '4室2厅0厨2卫',
       '4房间1卫', '3室1厅1厨3卫', '3室2厅1厨4卫', '4室1厅1厨4卫', '5室2厅1厨5卫',
       '5室1厅1厨4卫', '1室1厅1厨0卫', '5室3厅1厨6卫', '1室2厅1厨1卫', '4房间3卫', '6房间1卫',
       '3室3厅1厨2卫', '4室4厅1厨3卫', '1室0厅0厨1

In [8]:
train_rent['户型'].unique()

array(['1室1厅1卫', '3室1厅2卫', '3室2厅', '4室2厅', '2室1厅', '4室1厅', '1室0厅1卫',
       '2室1厅1卫', '1室1厅', '1室2厅2卫', '2房间', '2房间1卫', '1房间', '3室1厅',
       '1室1厅2卫', '7室1厅', '2室2厅', '1室0厅', '4房间', '5室4厅', '2室0厅', '2室1厅2卫',
       '3房间', '3室1厅1卫', '7房间', '5室2厅', '2室2厅1卫', '5房间', '3室1厅3卫',
       '4室2厅3卫', '1房间1卫', '8室4厅', '3室2厅2卫', '4房间2卫', '6室3厅', '6室2厅',
       '5室3厅', '2室2厅2卫', '3室2厅3卫', '6房间', '4室3厅', '5室1厅', '0室0厅', '9房间',
       '3室3厅', '6室1厅', '3房间2卫', '3室0厅', '1室2厅', '车库', '4室2厅2卫', '12室4厅',
       '0室1厅', '4室1厅2卫', '5室2厅2卫', '3室2厅1卫', '3室3厅2卫', '5室2厅4卫', '4室2厅4卫',
       '1室2厅1卫', '5室4厅3卫', '5室2厅3卫', '2室0厅1卫', '2房间2卫', '3房间1卫', '9房间5卫',
       '4房间3卫', '5房间3卫', '5房间2卫', '1室1厅0卫', '6房间2卫', '4室1厅3卫', '1室0厅0卫',
       '6房间3卫', '4室1厅1卫', '5室1厅4卫', '6室2厅4卫', '4室3厅3卫', '6室3厅3卫',
       '3室0厅0卫', '4房间1卫', '5室3厅3卫', '4室2厅1卫', '3室1厅0卫', '7室2厅3卫', nan,
       '3室3厅3卫', '3室0厅1卫', '5室3厅5卫', '5室1厅3卫', '2室2厅3卫', '6室2厅3卫',
       '2室1厅3卫', '5房间1卫', '7房间3卫', '5室0厅', '2室3厅', '7室3厅', '8室2厅', '4室4厅',
       '9

In [9]:
print("开始提取户型特征...")
def extract_layout_features(df, column_name):
    """
    使用正则表达式从户型描述字符串中提取特征。
    提取 '室'、'厅'、'厨'、'卫' 的数量。
    - '室' 包含 '室'、'房间'、'居室' 三种情况。
    - '房间总数' 被定义为 '室' + '厅' 的数量。
    参数:
    - df: DataFrame
    - column_name: 户型信息所在的列名 (例如 '房屋户型' 或 '户型')
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    # 将户型列统一转换为字符串类型，以便处理 nan 和其他非字符串类型
    # .astype(str) 会将 np.nan 转换为 'nan' 字符串，正则表达式不会匹配，后续 .fillna('0') 会处理
    layout_str = df_copy[column_name].astype(str)

    # 定义正则表达式模式
    # 我们使用 (\d+) 来捕获数字
    # (?:...) 是一个非捕获组，我们用它来匹配单位（室、房间、居室），但我们不提取它
    patterns = {
        '室': r'(\d+)\s*(?:室|房间|居室)', # 匹配 '室', '房间', 或 '居室'
        '厅': r'(\d+)\s*厅',
        '厨': r'(\d+)\s*厨',
        '卫': r'(\d+)\s*卫'
    }

    new_cols = []
    # 循环提取每个特征
    for feature_name, pattern in patterns.items():
        new_col_name = f'户型_{feature_name}'
        new_cols.append(new_col_name)
        
        # .str.extract(pattern, expand=False)
        # 提取第一个匹配组（即数字 \d+）
        # 如果没有匹配到（例如 '车库' 或 'nan'），则结果为 NaN
        df_copy[new_col_name] = layout_str.str.extract(pattern, expand=False)

    # 清理提取后的数据
    for col in new_cols:
        # .fillna('0')：将所有 NaN (未匹配到的) 替换为 '0'
        # .astype(int)：将列从字符串（或 object）类型转换为整数类型
        df_copy[col] = df_copy[col].fillna('0').astype(int)

    # 根据您的要求，创建“房间总数”
    # 我们这里将“房间总数”定义为 室 + 厅 的数量
    df_copy['户型_房间总数'] = df_copy['户型_室'] + df_copy['户型_厅']
    
    print(f"已从 '{column_name}' 列提取并创建: {new_cols} 和 '户型_房间总数'")
    
    return df_copy

# %%
# 将函数应用到所有四个数据帧
# 注意 price 和 rent 的户型列名不同

print("\n处理房价数据...")
train_price_processed = extract_layout_features(train_price_processed, '房屋户型')
test_price_processed = extract_layout_features(test_price_processed, '房屋户型')

print("\n处理租金数据...")
train_rent_processed = extract_layout_features(train_rent_processed, '户型')
test_rent_processed = extract_layout_features(test_rent_processed, '户型')

print("\n所有户型数据处理完毕！")

# %%
# 验证提取结果
print("\n--- 验证 房价训练集 (train_price) 的提取结果 ---")
# 选取包含原始列和新列的子集
price_layout_cols = ['房屋户型', '户型_室', '户型_厅', '户型_厨', '户型_卫', '户型_房间总数']
# 打印几个样本
print("常规样本 (前5行):")
print(train_price_processed[price_layout_cols].head())

# 检查 '房间' 格式的样本
print("\n'房间' 格式样本:")
print(train_price_processed[train_price_processed['房屋户型'].str.contains('房间', na=False)][price_layout_cols].head())

# 检查 'nan' (空值) 样本
print("\n'nan' (空值) 样本:")
print(train_price_processed[train_price_processed['房屋户型'].isna()][price_layout_cols].head())


print("\n--- 验证 租金训练集 (train_rent) 的提取结果 ---")
rent_layout_cols = ['户型', '户型_室', '户型_厅', '户型_厨', '户型_卫', '户型_房间总数']

# 打印几个样本
print("常规样本 (前5行):")
print(train_rent_processed[rent_layout_cols].head())

# 检查 '居室' 格式的样本
print("\n'居室' 格式样本:")
print(train_rent_processed[train_rent_processed['户型'].str.contains('居室', na=False)][rent_layout_cols].head())

# 检查 '车库'、'·' 等特殊值
print("\n'车库' 或 '·' 特殊值样本:")
print(train_rent_processed[train_rent_processed['户型'].isin(['车库', '·'])][rent_layout_cols].head())

开始提取户型特征...

处理房价数据...
已从 '房屋户型' 列提取并创建: ['户型_室', '户型_厅', '户型_厨', '户型_卫'] 和 '户型_房间总数'
已从 '房屋户型' 列提取并创建: ['户型_室', '户型_厅', '户型_厨', '户型_卫'] 和 '户型_房间总数'

处理租金数据...
已从 '户型' 列提取并创建: ['户型_室', '户型_厅', '户型_厨', '户型_卫'] 和 '户型_房间总数'
已从 '户型' 列提取并创建: ['户型_室', '户型_厅', '户型_厨', '户型_卫'] 和 '户型_房间总数'

所有户型数据处理完毕！

--- 验证 房价训练集 (train_price) 的提取结果 ---
常规样本 (前5行):
       房屋户型  户型_室  户型_厅  户型_厨  户型_卫  户型_房间总数
0  2室1厅1厨1卫     2     1     1     1        3
1  3室1厅1厨1卫     3     1     1     1        4
2  3室2厅1厨2卫     3     2     1     2        5
3  6室3厅1厨3卫     6     3     1     3        9
4     1房间1卫     1     0     0     1        1

'房间' 格式样本:
     房屋户型  户型_室  户型_厅  户型_厨  户型_卫  户型_房间总数
4   1房间1卫     1     0     0     1        1
16  2房间1卫     2     0     0     1        2
23  1房间1卫     1     0     0     1        1
25  3房间1卫     3     0     0     1        3
28  1房间1卫     1     0     0     1        1

'nan' (空值) 样本:
    房屋户型  户型_室  户型_厅  户型_厨  户型_卫  户型_房间总数
7    NaN     0     0     0     0        0
71   NaN     0 

In [10]:
#楼层
train_price_processed['所在楼层'].unique()


array(['中楼层 (共5层)', '顶层 (共6层)', '低楼层 (共6层)', '底层 (共2层)', '中楼层 (共10层)',
       '地下室 (共2层)', '高楼层 (共20层)', '地下室 (共0层)', '高楼层 (共22层)', '底层 (共3层)',
       '顶层 (共5层)', '高楼层 (共9层)', '高楼层 (共24层)', '底层 (共7层)', '底层 (共20层)',
       '底层 (共6层)', '底层 (共12层)', '顶层 (共4层)', '中楼层 (共21层)', '顶层 (共29层)',
       '高楼层 (共11层)', '中楼层 (共15层)', '高楼层 (共12层)', '中楼层 (共11层)',
       '中楼层 (共16层)', '中楼层 (共22层)', '顶层 (共16层)', '高楼层 (共28层)', '中楼层 (共4层)',
       '高楼层 (共13层)', '高楼层 (共15层)', '低楼层 (共19层)', '中楼层 (共13层)',
       '高楼层 (共6层)', '中楼层 (共7层)', '低楼层 (共16层)', '低楼层 (共11层)', '低楼层 (共13层)',
       '高楼层 (共10层)', '底层 (共8层)', '高楼层 (共16层)', '高楼层 (共7层)', '中楼层 (共18层)',
       '中楼层 (共6层)', '低楼层 (共29层)', '高楼层 (共14层)', '中楼层 (共27层)',
       '低楼层 (共22层)', '低楼层 (共56层)', '中楼层 (共14层)', '地下室 (共9层)',
       '中楼层 (共29层)', '中楼层 (共28层)', '地下室 (共4层)', '地下室 (共11层)',
       '高楼层 (共31层)', '高楼层 (共36层)', '中楼层 (共56层)', '顶层 (共12层)', '底层 (共1层)',
       '底层 (共15层)', '低楼层 (共24层)', '底层 (共18层)', '低楼层 (共15层)', '中楼层 (共9层)',
       '低楼层 (共9层)', '底层 (共4层)'

In [17]:
print(repr(train_rent_processed['楼层'].unique()))

array(['4/6层', '1/18层', '1/10层', ..., '8/38层', '21/43层', '37/37层'],
      dtype=object)


In [18]:
#%%
# --------------------------------------------------
# 楼层特征提取
# --------------------------------------------------
print("开始提取楼层特征...")

def process_price_floors(df, column_name='所在楼层'):
    """
    处理 房价数据集 (price) 的楼层特征。
    格式: '中楼层 (共5层)', '顶层 (共6层)', '地下室 (共0层)'
    
    1. 提取楼层类别 (中楼层, 顶层, 底层, 高楼层, 低楼层, 地下室)，并创建哑变量。
    2. 提取总楼层数 (数值型)。
    """
    df_copy = df.copy()
    
    # 统一转为字符串，处理 np.nan
    s = df_copy[column_name].astype(str)
    
    # 1. 提取楼层类别 (例如 '中楼层')
    #    正则表达式 r'^(.*?) \(' 匹配从开头(^)到第一个 ' (' 之间的所有字符(.*?)
    #    .fillna('未知') 处理 'nan' 或不匹配的格式
    floor_category = s.str.extract(r'^(.*?) \(').fillna('未知')
    floor_category.name = '楼层_类别'

    # 2. 创建哑变量
    #    prefix='楼层' 将创建 '楼层_中楼层', '楼层_顶层', '楼层_未知' 等列
    dummies = pd.get_dummies(floor_category, prefix='楼层')
    
    # 3. 提取总楼层数 (例如 '5')
    #    正则表达式 r'共(\d+)层' 匹配 '共' 和 '层' 之间的数字 (\d+)
    total_floors = s.str.extract(r'共(\d+)层')
    
    # 4. 清理总楼层数
    #    .fillna('0') 处理 'nan' 或未匹配到的情况 (例如 '地下室 (共0层)' 会提
    #    取出 '0'，但 'nan' 字符串会变成 NaN，所以用 '0' 填充)
    df_copy['楼层_总楼层'] = total_floors.fillna('0').astype(int)
    
    # 5. 合并新特征
    df_copy = pd.concat([df_copy, dummies], axis=1)
    
    print(f"已处理 '{column_name}': 新增 '楼层_总楼层' 和 {len(dummies.columns)} 个哑变量 (如 '楼层_中楼层' 等)")
    return df_copy

def process_rent_floors(df, column_name='楼层'):
    """
    处理 租金数据集 (rent) 的楼层特征。
    格式: '4/6层', '1/18层'
    
    1. 提取所在楼层 (数值型)。
    2. 提取总楼层数 (数值型)。
    """
    df_copy = df.copy()
    
    # 统一转为字符串，处理 np.nan
    s = df_copy[column_name].astype(str)
    
    # 1. 提取所在楼层 (例如 '4' from '4/6层')
    #    正则表达式 r'^(\d+)/' 匹配开头(^)的数字(\d+)和它后面的 '/'
    current_floor = s.str.extract(r'^(\d+)/')
    
    # 2. 提取总楼层数 (例如 '6' from '4/6层')
    #    正则表达式 r'/(\d+)层?$' 匹配 '/' 后面的数字(\d+)，'层' 是可选的(层?)，
    #    直到字符串末尾($)
    total_floors = s.str.extract(r'/(\d+)层?$')
    
    # 3. 清理并添加新列
    #    .fillna('0') 处理 'nan' 或其他不匹配格式 (如 '车库' 等)
    df_copy['楼层_所在楼层'] = current_floor.fillna('0').astype(int)
    df_copy['楼层_总楼层'] = total_floors.fillna('0').astype(int)
    
    print(f"已处理 '{column_name}': 新增 '楼层_所在楼层' 和 '楼层_总楼层'")
    return df_copy

# %%
# 应用到所有四个数据帧
print("\n处理房价数据 (price)...")
train_price_processed = process_price_floors(train_price_processed, '所在楼层')
test_price_processed = process_price_floors(test_price_processed, '所在楼层')

print("\n处理租金数据 (rent)...")
train_rent_processed = process_rent_floors(train_rent_processed, '楼层')
test_rent_processed = process_rent_floors(test_rent_processed, '楼层')

print("\n所有楼层数据处理完毕！")

# %%
# 验证提取结果
print("\n--- 验证 房价训练集 (train_price) 的提取结果 ---")
# 选取包含原始列和新列的子集
price_floor_cols = [
    '所在楼层', 
    '楼层_总楼层', 
    '楼层_中楼层', 
    '楼层_顶层', 
    '楼层_低楼层', 
    '楼层_底层', 
    '楼层_高楼层', 
    '楼层_地下室',
    '楼层_未知' # 确保检查 '未知' 列
]
# 过滤出所有相关的列
price_cols_to_show = [col for col in price_floor_cols if col in train_price_processed.columns]
print(train_price_processed[price_cols_to_show].head())

# 检查 '未知' (即 'nan') 的处理情况
print("\n检查 'nan' (空值) 样本:")
print(train_price_processed[train_price_processed['所在楼层'].isna()][price_cols_to_show].head())


print("\n--- 验证 租金训练集 (train_rent) 的提取结果 ---")
rent_floor_cols = ['楼层', '楼层_所在楼层', '楼层_总楼层']
print(train_rent_processed[rent_floor_cols].head())

# 检查 'nan' (空值) 的处理情况
print("\n检查 'nan' (空值) 样本:")
print(train_rent_processed[train_rent_processed['楼层'].isna()][rent_floor_cols].head())

# 检查可能的非标准格式（如果存在）
# 假设 '楼层' 列中存在 '车库' 这样的值
if '车库' in train_rent['楼层'].unique():
    print("\n检查 '车库' 样本:")
    print(train_rent_processed[train_rent_processed['楼层'] == '车库'][rent_floor_cols].head())

开始提取楼层特征...

处理房价数据 (price)...
已处理 '所在楼层': 新增 '楼层_总楼层' 和 6 个哑变量 (如 '楼层_中楼层' 等)
已处理 '所在楼层': 新增 '楼层_总楼层' 和 6 个哑变量 (如 '楼层_中楼层' 等)

处理租金数据 (rent)...
已处理 '楼层': 新增 '楼层_所在楼层' 和 '楼层_总楼层'
已处理 '楼层': 新增 '楼层_所在楼层' 和 '楼层_总楼层'

所有楼层数据处理完毕！

--- 验证 房价训练集 (train_price) 的提取结果 ---
         所在楼层  楼层_总楼层  楼层_中楼层  楼层_顶层  楼层_低楼层  楼层_底层  楼层_高楼层  楼层_地下室
0   中楼层 (共5层)       5    True  False   False  False   False   False
1    顶层 (共6层)       6   False   True   False  False   False   False
2   低楼层 (共6层)       6   False  False    True  False   False   False
3    底层 (共2层)       2   False  False   False   True   False   False
4  中楼层 (共10层)      10    True  False   False  False   False   False

检查 'nan' (空值) 样本:
Empty DataFrame
Columns: [所在楼层, 楼层_总楼层, 楼层_中楼层, 楼层_顶层, 楼层_低楼层, 楼层_底层, 楼层_高楼层, 楼层_地下室]
Index: []

--- 验证 租金训练集 (train_rent) 的提取结果 ---
       楼层  楼层_所在楼层  楼层_总楼层
0    4/6层        4       6
1    4/6层        4       6
2   1/18层        1      18
3   1/10层        1      10
4  18/18层       18      18

检查 'nan' (空值)

In [19]:
train_price_processed['建筑面积'].unique()

array(['52.3㎡', '127.44㎡', '118.02㎡', ..., '154.61㎡', '162.93㎡',
       '149.49㎡'], dtype=object)

In [21]:
train_rent_processed['面积'].unique()

array(['36.42㎡', '41.00㎡', '37.36㎡', ..., '162.93㎡', '117.16㎡', '100.09㎡'],
      dtype=object)

In [22]:
print("开始提取面积特征...")
def process_area(df, input_col, output_col='面积_数值'):
    """
    从面积字符串 (如 '52.3㎡') 中提取数值。
    
    1. 提取数字 (包括整数和小数)。
    2. 将新列转换为 float 类型。
    
    参数:
    - df: DataFrame
    - input_col: 面积信息所在的列名 (例如 '建筑面积' 或 '面积')
    - output_col: 新建的数值列的名称
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 统一转为字符串，处理 np.nan
    #    np.nan 会变成 'nan' 字符串
    s = df_copy[input_col].astype(str)
    
    # 2. 提取数字
    #    正则表达式 r'(\d+\.?\d*)'
    #    (\d+\.?\d*) 是一个捕获组:
    #    \d+   匹配一个或多个数字 (整数部分)
    #    \.?   匹配一个可选的(0或1个)小数点
    #    \d* 匹配零个或多个数字 (小数部分)
    #    这可以匹配 '52' 和 '52.3' 以及 '127.44'
    extracted = s.str.extract(r'(\d+\.?\d*)', expand=False)
    
    # 3. 清理并转换为 float
    #    .fillna('0')：将所有 NaN (来自 'nan' 字符串或未匹配项) 替换为 '0'
    #    .astype(float)：将列从字符串转换为浮点数
    df_copy[output_col] = extracted.fillna('0').astype(float)
    
    print(f"已处理 '{input_col}', 新增列 '{output_col}' (dtype: {df_copy[output_col].dtype})")
    return df_copy

print("\n处理房价数据 (price) 的面积...")
train_price_processed = process_area(train_price_processed, '建筑面积', '面积_数值')
test_price_processed = process_area(test_price_processed, '建筑面积', '面积_数值')

print("\n处理租金数据 (rent) 的面积...")
train_rent_processed = process_area(train_rent_processed, '面积', '面积_数值')
test_rent_processed = process_area(test_rent_processed, '面积', '面积_数值')

print("\n所有面积数据处理完毕！")

# %%
# 验证提取结果
print("\n--- 验证 房价训练集 (train_price) 的面积提取 ---")
cols_to_show_price = ['建筑面积', '面积_数值']
print(train_price_processed[cols_to_show_price].head())
print(f"新列 '面积_数值' 的 Dtype: {train_price_processed['面积_数值'].dtype}")

# 检查 'nan' (空值) 样本
print("\n'nan' (空值) 样本:")
print(train_price_processed[train_price_processed['建筑面积'].isna()][cols_to_show_price].head())


print("\n--- 验证 租金训练集 (train_rent) 的面积提取 ---")
cols_to_show_rent = ['面积', '面积_数值']
print(train_rent_processed[cols_to_show_rent].head())
print(f"新列 '面积_数值' 的 Dtype: {train_rent_processed['面积_数值'].dtype}")

# 检查 'nan' (空值) 样本
print("\n'nan' (空值) 样本:")
print(train_rent_processed[train_rent_processed['面积'].isna()][cols_to_show_rent].head())

开始提取面积特征...

处理房价数据 (price) 的面积...
已处理 '建筑面积', 新增列 '面积_数值' (dtype: float64)
已处理 '建筑面积', 新增列 '面积_数值' (dtype: float64)

处理租金数据 (rent) 的面积...
已处理 '面积', 新增列 '面积_数值' (dtype: float64)
已处理 '面积', 新增列 '面积_数值' (dtype: float64)

所有面积数据处理完毕！

--- 验证 房价训练集 (train_price) 的面积提取 ---
      建筑面积   面积_数值
0    52.3㎡   52.30
1  127.44㎡  127.44
2  118.02㎡  118.02
3  293.23㎡  293.23
4   39.85㎡   39.85
新列 '面积_数值' 的 Dtype: float64

'nan' (空值) 样本:
Empty DataFrame
Columns: [建筑面积, 面积_数值]
Index: []

--- 验证 租金训练集 (train_rent) 的面积提取 ---
       面积  面积_数值
0  36.42㎡  36.42
1  41.00㎡  41.00
2  37.36㎡  37.36
3  55.42㎡  55.42
4  49.30㎡  49.30
新列 '面积_数值' 的 Dtype: float64

'nan' (空值) 样本:
Empty DataFrame
Columns: [面积, 面积_数值]
Index: []


In [23]:
#%%
# --------------------------------------------------
# 套内面积特征提取 (仅Price数据集)
# --------------------------------------------------
print("开始提取 '套内面积' 特征...")

# 我们重用之前定义的 process_area 函数
# 传入 '套内面积' 作为 input_col，并指定 '套内面积_数值' 作为 output_col
# 这样就不会覆盖 '面积_数值' (来自'建筑面积')

try:
    # 应用到 Price 数据集
    train_price_processed = process_area(train_price_processed, '套内面积', '套内面积_数值')
    test_price_processed = process_area(test_price_processed, '套内面积', '套内面积_数值')
    
    print("\n'套内面积' 数据处理完毕！")

    # %%
    # 验证提取结果
    print("\n--- 验证 房价训练集 (train_price) 的 '套内面积' 提取 ---")
    cols_to_show_inside_area = ['套内面积', '套内面积_数值']
    print(train_price_processed[cols_to_show_inside_area].head())
    print(f"新列 '套内面积_数值' 的 Dtype: {train_price_processed['套内面积_数值'].dtype}")

    # 检查 'nan' (空值) 样本
    print("\n'nan' (空值) 样本:")
    print(train_price_processed[train_price_processed['套内面积'].isna()][cols_to_show_inside_area].head())

except NameError:
    print("错误：'process_area' 函数未定义。请确保先运行定义了该函数的代码块。")
except KeyError:
    print("错误：'套内面积' 列在 Price 数据集中不存在。")

开始提取 '套内面积' 特征...
已处理 '套内面积', 新增列 '套内面积_数值' (dtype: float64)
已处理 '套内面积', 新增列 '套内面积_数值' (dtype: float64)

'套内面积' 数据处理完毕！

--- 验证 房价训练集 (train_price) 的 '套内面积' 提取 ---
      套内面积  套内面积_数值
0      NaN     0.00
1   123.7㎡   123.70
2  101.95㎡   101.95
3  293.23㎡   293.23
4   29.94㎡    29.94
新列 '套内面积_数值' 的 Dtype: float64

'nan' (空值) 样本:
   套内面积  套内面积_数值
0   NaN      0.0
5   NaN      0.0
7   NaN      0.0
8   NaN      0.0
14  NaN      0.0


In [24]:
#房屋朝向
train_price_processed['房屋朝向'].unique()


array(['南 北', '东南', '东 南 西 北', '南', '南 西 北', '西', '北', '西 北', '东', '东南 西',
       '南 西', '东 北', '东 南 北', '东南 西北', '西 南', '东 西', '西南', '北 西', '东北',
       '南 北 西 东', '西北', '东 北 东北', '南 东', '东 南 西', '北 南', '西南 东', '南 北 东',
       '东南 北', '西 东', '南 北 西', '南 北 东 西', '北 东北', '东 西 北', '北 东南',
       '西南 东北', '东 西北', '东 南', '西南 北', '东 西 南', '南 西南', '西北 南', '东南 东北',
       '东 北 西', '北 东', '西北 东北', '北 西 东', '西 南 北', '东 东南 西 西北', '北 南 西',
       '南 东 西', '南 西南 北', '南 西北', '西南 西 南', '南 东 北', '东 东南 南', '东南 南',
       '西北 西 东', '北 南 东', '东南 西南', '西 西北', '西 北 南', '东北 西', '西南 西北',
       '东 西南 北', '东 东南', '西 东 北', '东南 南 北', '东北 东南', '东 东南 南 西南 西',
       '南 东北', '东南 西 北', '东南 南 东', '西 南 东 北', '东 东北', '西 东北', '西南 西',
       '南 西 东', '南 东南', '东南 西南 西北', '西 北 东', '西南 南', '西南 东南', '东北 南',
       '北 西南', '南 东 西南', '东北 西北', '东 东南 北', '西 北 东 南', '西 东 南', '西北 东',
       '西南 西北 东北', '东南 南 西南', '东 西南', '南 西南 西', '东 东南 南 北', '西 西北 北',
       '东 南 东南', '南 北 东北', '东 西 南 北', '南 东 东南 西南 西', '西 北 西北', '东北 北',
      

In [25]:
train_rent_processed['朝向'].unique()

array(['西', '南', '北', '西南', '南 北', '东', '东南', '南 西', '东 西', '东北', '东南 北',
       '南 西南 北', '西南 西', '西北', '南 西 北', '东南 西南', '东 东南', '南 西北', '东南 西北',
       '西南 西北', '东 南', '西 北', '南 西南', '东 北', '东 南 北', '西南 北', '南 东北',
       '东 东南 南', '东 南 西', '西 西北 北', '东南 西', '东 西 北', '东 东北',
       '南 西南 西 西北 北', '东 东南 西', '东 西北', '东南 南', '东南 南 北', '东 西南', '西北 东北',
       '东南 南 西南', '东 东南 南 西南 北', '东 南 西 北', nan, '西 西北', '东 北 东北',
       '南 西南 西', '东 西北 北', '西北 北', '北 东北', '未知', '西南 西 西北', '南 北 东北',
       '东南 西南 西', '西南 东北', '东 西南 北', '南 西南 西 西北', '东 南 西北', '东南 东北',
       '东 西 西北', '东 东南 南 西南 西', '东 东南 南 北 东北', '东 东南 南 西南', '南 西北 北',
       '东 东南 东北', '西 东北', '东 东南 北', '未知 东南', '东 东南 西 北', '西北 北 东北',
       '东 南 东北', '西南 西北 北', '东 东南 南 西', '东南 南 西 西北', '东南 西南 西北',
       '东南 南 西北 北', '东南 西北 北', '西 西北 北 东北', '东 南 西南', '东 东南 西南', '西南 西 北',
       '东南 南 西南 西 西北', '东南 西南 北', '东 东南 西南 西 西北', '东 西南 西北 北', '南 西南 西北',
       '东 西南 西北', '东 西北 东北', '东 东南 北 东北', '东 东南 南 北', '东 东南 西南 西',
       '东南 南 西南 东北'],

In [26]:
print("开始提取朝向特征...")

from sklearn.preprocessing import MultiLabelBinarizer

# 定义所有可能的基本朝向
# 我们将把 '东南' 和 '东' 视为不同的类别
all_orientations = ['东', '南', '西', '北', '东南', '西南', '东北', '西北']

def process_orientations_mlb(df, column_name):
    """
    处理朝向特征。
    格式: '南 北', '东南', '东 南 西 北', 'nan', '未知'
    
    1. 将 'nan' 和 '未知' 替换为空字符串。
    2. 按空格分割字符串，得到一个朝向列表。
    3. 使用 MultiLabelBinarizer 为 all_orientations 中的每个朝向创建哑变量。
    """
    df_copy = df.copy()
    
    # 1. 预处理：
    #    .fillna('') 将 nan 替换为空字符串
    #    .str.replace('未知', '') 将 '未知' 替换为空字符串
    s = df_copy[column_name].fillna('').str.replace('未知', '')
    
    # 2. 按空格分割字符串，得到一个列表的 Series
    #    '南 北' -> ['南', '北']
    #    '东南'  -> ['东南']
    #    ''      -> ['']
    s_split = s.str.split(' ')
    
    # 3. 初始化并应用 MultiLabelBinarizer
    #    我们传入 classes=all_orientations 来确保：
    #    a) 只为这8个方向创建列
    #    b) 即使某个朝向在数据中没出现，也会创建列（值为0）
    #    c) 忽略 '未知' 或 '' 等其他 token
    mlb = MultiLabelBinarizer(classes=all_orientations)
    
    # 4. fit_transform 会返回一个 numpy 数组
    dummies_data = mlb.fit_transform(s_split)
    
    # 5. 创建新的列名，例如 '朝向_东', '朝向_南'
    dummy_cols = [f'朝向_{c}' for c in mlb.classes_]
    
    # 6. 将数组转换为 DataFrame，并设置正确的列名和索引
    dummies_df = pd.DataFrame(dummies_data, columns=dummy_cols, index=df_copy.index)
    
    # 7. 合并回原始 DataFrame
    df_copy = pd.concat([df_copy, dummies_df], axis=1)
    
    print(f"已处理 '{column_name}': 新增 {len(dummy_cols)} 个朝向哑变量 (如 '朝向_东')")
    return df_copy

# %%
# 应用到所有四个数据帧
print("\n处理房价数据 (price) 的朝向...")
train_price_processed = process_orientations_mlb(train_price_processed, '房屋朝向')
test_price_processed = process_orientations_mlb(test_price_processed, '房屋朝向')

print("\n处理租金数据 (rent) 的朝向...")
train_rent_processed = process_orientations_mlb(train_rent_processed, '朝向')
test_rent_processed = process_orientations_mlb(test_rent_processed, '朝向')

print("\n所有朝向数据处理完毕！")

# %%
# 验证提取结果
print("\n--- 验证 房价训练集 (train_price) 的朝向提取 ---")
# 选取几个复杂的样本和新列进行验证
orientation_cols_to_show = [
    '房屋朝向', 
    '朝向_东', 
    '朝向_南', 
    '朝向_西', 
    '朝向_北', 
    '朝向_东南', 
    '朝向_西南', 
    '朝向_东北', 
    '朝向_西北'
]

# 样本1: '东 南 西 北'
print("\n样本 '东 南 西 北':")
print(train_price_processed[train_price_processed['房屋朝向'] == '东 南 西 北'][orientation_cols_to_show].head())

# 样本2: '东南' (验证 '东' 和 '南' 不会被错误标记)
print("\n样本 '东南':")
print(train_price_processed[train_price_processed['房屋朝向'] == '东南'][orientation_cols_to_show].head())

# 样本3: '东 东南'
print("\n样本 '东 东南' (来自 '东 东南 南 西南 西'):")
# 我们用 .str.contains 来找一个复杂的例子
complex_sample = train_price_processed[train_price_processed['房屋朝向'].astype(str).str.contains('东 东南', na=False)]
print(complex_sample[orientation_cols_to_show].head())

# 样本4: 'nan' (空值)
print("\n样本 'nan' (空值):")
print(train_price_processed[train_price_processed['房屋朝向'].isna()][orientation_cols_to_show].head())


print("\n--- 验证 租金训练集 (train_rent) 的朝向提取 ---")
# 样本5: '未知'
print("\n样本 '未知':")
orientation_cols_rent = [col.replace('房屋朝向', '朝向') for col in orientation_cols_to_show]
print(train_rent_processed[train_rent_processed['朝向'] == '未知'][orientation_cols_rent].head())

开始提取朝向特征...

处理房价数据 (price) 的朝向...
已处理 '房屋朝向': 新增 8 个朝向哑变量 (如 '朝向_东')
已处理 '房屋朝向': 新增 8 个朝向哑变量 (如 '朝向_东')

处理租金数据 (rent) 的朝向...
已处理 '朝向': 新增 8 个朝向哑变量 (如 '朝向_东')
已处理 '朝向': 新增 8 个朝向哑变量 (如 '朝向_东')

所有朝向数据处理完毕！

--- 验证 房价训练集 (train_price) 的朝向提取 ---

样本 '东 南 西 北':
        房屋朝向  朝向_东  朝向_南  朝向_西  朝向_北  朝向_东南  朝向_西南  朝向_东北  朝向_西北
3    东 南 西 北     1     1     1     1      0      0      0      0
113  东 南 西 北     1     1     1     1      0      0      0      0
286  东 南 西 北     1     1     1     1      0      0      0      0
711  东 南 西 北     1     1     1     1      0      0      0      0
714  东 南 西 北     1     1     1     1      0      0      0      0

样本 '东南':
   房屋朝向  朝向_东  朝向_南  朝向_西  朝向_北  朝向_东南  朝向_西南  朝向_东北  朝向_西北
2    东南     0     0     0     0      1      0      0      0
8    东南     0     0     0     0      1      0      0      0
33   东南     0     0     0     0      1      0      0      0
66   东南     0     0     0     0      1      0      0      0
87   东南     0     0     0     0      1   

In [27]:
train_price_processed['环线位置'].unique()

array(['二至三环', '五至六环', '六环外', '三至四环', '四至五环', nan, '二环内', '内环内', '内环至外环',
       '外环外', '内环至中环', '中环至外环'], dtype=object)

In [28]:
train_rent_processed['环线位置'].unique()

array(['三至四环', '二至三环', '六环外', '五至六环', '四至五环', nan, '二环内', '内环至外环', '内环内',
       '外环外', '中环至外环', '内环至中环'], dtype=object)

In [29]:
print("开始提取环线位置特征...")

def process_ring_road(df, column_name):
    """
    处理环线位置特征，创建哑变量。
    'nan' 值被视为一个特定类别 '无环线'。
    
    1. 使用 .fillna('无环线') 将 'nan' 替换为字符串 '无环线'。
    2. 使用 pd.get_dummies 创建哑变量，前缀为 '环线'。
    
    参数:
    - df: DataFrame
    - column_name: 环线位置信息所在的列名
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 将 'nan' 替换为有意义的类别 '无环线'
    s = df_copy[column_name].fillna('无环线')
    
    # 2. 创建哑变量
    #    prefix='环线' 将创建 '环线_二至三环', '环线_六环外', '环线_无环线' 等
    dummies = pd.get_dummies(s, prefix='环线')
    
    # 3. 合并新特征
    df_copy = pd.concat([df_copy, dummies], axis=1)
    
    print(f"已处理 '{column_name}': 新增 {len(dummies.columns)} 个环线哑变量 (包括 '环线_无环线')")
    return df_copy

# %%
# 应用到所有四个数据帧
print("\n处理房价数据 (price) 的环线位置...")
train_price_processed = process_ring_road(train_price_processed, '环线位置')
test_price_processed = process_ring_road(test_price_processed, '环线位置')

print("\n处理租金数据 (rent) 的环线位置...")
train_rent_processed = process_ring_road(train_rent_processed, '环线位置')
test_rent_processed = process_ring_road(test_rent_processed, '环线位置')

print("\n所有环线位置数据处理完毕！")

# %%
# 验证提取结果
print("\n--- 验证 房价训练集 (train_price) 的环线位置提取 ---")

# 1. 检查 'nan' (空值) 样本是否正确转换为 '环线_无环线'
print("\n'nan' (空值) 样本:")
# 找到 '环线位置' 原始列为 NaT/nan 的行
nan_sample_price = train_price_processed[train_price['环线位置'].isna()]
# 选取几个相关的列进行展示
cols_to_show_price = [col for col in nan_sample_price.columns if col.startswith('环线_')]
print(nan_sample_price[cols_to_show_price].head())

# 2. 检查一个常规样本
print("\n常规样本 ('二至三环'):")
sample_price = train_price_processed[train_price['环线位置'] == '二至三环']
print(sample_price[cols_to_show_price].head())

print(f"\n创建的列: {cols_to_show_price}")

开始提取环线位置特征...

处理房价数据 (price) 的环线位置...
已处理 '环线位置': 新增 12 个环线哑变量 (包括 '环线_无环线')
已处理 '环线位置': 新增 12 个环线哑变量 (包括 '环线_无环线')

处理租金数据 (rent) 的环线位置...
已处理 '环线位置': 新增 12 个环线哑变量 (包括 '环线_无环线')
已处理 '环线位置': 新增 12 个环线哑变量 (包括 '环线_无环线')

所有环线位置数据处理完毕！

--- 验证 房价训练集 (train_price) 的环线位置提取 ---

'nan' (空值) 样本:
     环线_三至四环  环线_中环至外环  环线_二环内  环线_二至三环  环线_五至六环  环线_六环外  环线_内环内  环线_内环至中环  \
33     False     False   False    False    False   False   False     False   
58     False     False   False    False    False   False   False     False   
72     False     False   False    False    False   False   False     False   
78     False     False   False    False    False   False   False     False   
101    False     False   False    False    False   False   False     False   

     环线_内环至外环  环线_四至五环  环线_外环外  环线_无环线  
33      False    False   False    True  
58      False    False   False    True  
72      False    False   False    True  
78      False    False   False    True  
101     False    False   False    Tr

In [30]:
train_price_processed['配备电梯'].unique()


array(['无', '有', nan], dtype=object)

In [32]:
train_rent_processed['电梯'].unique()

array(['无', '有', nan], dtype=object)

In [33]:
print("开始提取电梯特征...")

def process_elevator(df, column_name):
    """
    处理电梯特征 ('有', '无', nan)，创建哑变量。
    'nan' 值被视为一个特定类别 '未知'。
    
    1. 使用 .fillna('未知') 将 'nan' 替换为字符串 '未知'。
    2. 使用 pd.get_dummies 创建哑变量，前缀为 '电梯'。
    
    参数:
    - df: DataFrame
    - column_name: 电梯信息所在的列名 (例如 '配备电梯' 或 '电梯')
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 将 'nan' 替换为有意义的类别 '未知'
    s = df_copy[column_name].fillna('未知')
    
    # 2. 创建哑变量
    #    prefix='电梯' 将创建 '电梯_有', '电梯_无', '电梯_未知'
    dummies = pd.get_dummies(s, prefix='电梯')
    
    # 3. 合并新特征
    df_copy = pd.concat([df_copy, dummies], axis=1)
    
    print(f"已处理 '{column_name}': 新增 {len(dummies.columns)} 个电梯哑变量")
    return df_copy

# %%
# 应用到所有四个数据帧
print("\n处理房价数据 (price) 的电梯...")
train_price_processed = process_elevator(train_price_processed, '配备电梯')
test_price_processed = process_elevator(test_price_processed, '配备电梯')

print("\n处理租金数据 (rent) 的电梯...")
train_rent_processed = process_elevator(train_rent_processed, '电梯')
test_rent_processed = process_elevator(test_rent_processed, '电梯')

print("\n所有电梯数据处理完毕！")

# %%
# 验证提取结果
print("\n--- 验证 房价训练集 (train_price) 的电梯提取 ---")
cols_to_show_price = ['配备电梯', '电梯_有', '电梯_无', '电梯_未知']

# 1. 检查 'nan' (空值) 样本是否正确转换为 '电梯_未知'
print("\n'nan' (空值) 样本:")
# 找到 '配备电梯' 原始列为 nan 的行
nan_sample_price = train_price_processed[train_price['配备电梯'].isna()]
# 过滤出存在的列进行展示
cols_to_show_price_existing = [col for col in cols_to_show_price if col in nan_sample_price.columns]
print(nan_sample_price[cols_to_show_price_existing].head())

# 2. 检查一个常规样本 ('有')
print("\n常规样本 ('有'):")
sample_price_has = train_price_processed[train_price['配备电梯'] == '有']
print(sample_price_has[cols_to_show_price_existing].head())

print(f"\n创建的列: {[col for col in train_price_processed.columns if col.startswith('电梯_')]}")

开始提取电梯特征...

处理房价数据 (price) 的电梯...
已处理 '配备电梯': 新增 3 个电梯哑变量
已处理 '配备电梯': 新增 3 个电梯哑变量

处理租金数据 (rent) 的电梯...
已处理 '电梯': 新增 3 个电梯哑变量
已处理 '电梯': 新增 2 个电梯哑变量

所有电梯数据处理完毕！

--- 验证 房价训练集 (train_price) 的电梯提取 ---

'nan' (空值) 样本:
   配备电梯   电梯_有   电梯_无  电梯_未知
3   NaN  False  False   True
7   NaN  False  False   True
63  NaN  False  False   True
68  NaN  False  False   True
71  NaN  False  False   True

常规样本 ('有'):
   配备电梯  电梯_有   电梯_无  电梯_未知
2     有  True  False  False
4     有  True  False  False
6     有  True  False  False
8     有  True  False  False
10    有  True  False  False

创建的列: ['电梯_无', '电梯_有', '电梯_未知']


In [34]:
train_price_processed['建筑结构'].unique()


array(['混合结构', '钢混结构', nan, '砖混结构', '钢结构', '砖木结构', '未知结构', '框架结构'],
      dtype=object)

In [35]:
train_price_processed['建筑结构_comm'].unique()

array(['板楼/平房', '板楼', '塔楼/板楼/塔板结合', '板楼/塔板结合', '塔楼', '塔楼/塔板结合',
       '塔楼/板楼/平房', '塔楼/板楼', '塔板结合', '塔楼/板楼/塔板结合/平房', '板楼/塔板结合/平房',
       '塔板结合/平房', nan, '平房', '塔楼/平房', '塔楼/塔板结合/平房'], dtype=object)

In [36]:
train_rent_processed['建筑结构'].unique()

array(['塔楼/板楼', '塔楼/板楼/塔板结合', '塔楼', '板楼', '塔楼/塔板结合', '塔板结合', '板楼/塔板结合',
       '板楼/平房', nan, '塔楼/板楼/塔板结合/平房', '板楼/塔板结合/平房', '塔板结合/平房', '塔楼/板楼/平房',
       '塔楼/平房', '平房', '塔楼/塔板结合/平房'], dtype=object)

In [37]:
print("开始提取建筑结构特征...")

# 确保 MultiLabelBinarizer 已被导入
try:
    from sklearn.preprocessing import MultiLabelBinarizer
except ImportError:
    print("错误：需要 'sklearn' 库中的 'MultiLabelBinarizer'。")
    # 如果在Jupyter环境中，可以尝试安装
    # !pip install scikit-learn

def process_simple_structure(df, column_name):
    """
    处理 Price['建筑结构'] 这样的简单类别特征。
    'nan' 和 '未知结构' 将被合并为 '建筑结构_未知结构'。
    """
    df_copy = df.copy()
    
    # 1. 'nan' 和 '未知结构' 归为一类
    #    .fillna('未知结构') 将所有 nan 替换为 '未知结构'
    s = df_copy[column_name].fillna('未知结构')
    
    # 2. 创建哑变量
    #    将创建 '建筑结构_混合结构', '建筑结构_钢混结构', '建筑结构_未知结构' 等
    dummies = pd.get_dummies(s, prefix='建筑结构')
    
    # 3. 合并
    df_copy = pd.concat([df_copy, dummies], axis=1)
    
    print(f"已处理 (简单类别) '{column_name}': 新增 {len(dummies.columns)} 个哑变量。")
    return df_copy

def process_multilabel_structure(df, column_name):
    """
    处理 Price['建筑结构_comm'] 和 Rent['建筑结构'] 这样的多标签类别特征。
    按 '/' 分割，并为 '板楼', '平房', '塔楼', '塔板结合' 创建哑变量。
    """
    df_copy = df.copy()
    
    # 1. 定义所有可能的建筑类型
    all_types = ['板楼', '平房', '塔楼', '塔板结合']
    
    # 2. 预处理: nan 替换为空字符串
    s = df_copy[column_name].fillna('')
    
    # 3. 按 '/' 分割
    #    '板楼/平房' -> ['板楼', '平房']
    s_split = s.str.split('/')
    
    # 4. 初始化并应用 MultiLabelBinarizer
    mlb = MultiLabelBinarizer(classes=all_types)
    dummies_data = mlb.fit_transform(s_split)
    
    # 5. 创建列名，根据您的要求，例如 '建筑结构_板楼'
    dummy_cols = [f'建筑结构_{c}' for c in mlb.classes_]
    
    # 6. 转为 DataFrame
    dummies_df = pd.DataFrame(dummies_data, columns=dummy_cols, index=df_copy.index)
    
    # 7. 合并
    df_copy = pd.concat([df_copy, dummies_df], axis=1)
    
    print(f"已处理 (多标签) '{column_name}': 新增 {len(dummy_cols)} 个哑变量。")
    return df_copy

# %%
# --- 应用处理 ---

print("\n--- 处理 Price 数据集 ---")
# 1. 处理 Price['建筑结构'] (简单类别)
train_price_processed = process_simple_structure(train_price_processed, '建筑结构')
test_price_processed = process_simple_structure(test_price_processed, '建筑结构')

# 2. 处理 Price['建筑结构_comm'] (多标签)
train_price_processed = process_multilabel_structure(train_price_processed, '建筑结构_comm')
test_price_processed = process_multilabel_structure(test_price_processed, '建筑结构_comm')


print("\n--- 处理 Rent 数据集 ---")
# 1. 处理 Rent['建筑结构'] (多标签)
train_rent_processed = process_multilabel_structure(train_rent_processed, '建筑结构')
test_rent_processed = process_multilabel_structure(test_rent_processed, '建筑结构')

print("\n所有建筑结构数据处理完毕！")

# %%
# --- 验证提取结果 ---

print("\n--- 验证 房价训练集 (train_price) 的提取结果 ---")

# 1. 验证简单类别 (Price['建筑结构'])
print("\n验证简单类别 ('nan' 和 '未知结构'):")
# 选取 '建筑结构' 为 nan 或 '未知结构' 的样本
sample_price_simple = train_price_processed[
    (train_price['建筑结构'].isna()) | (train_price['建筑结构'] == '未知结构')
]
simple_cols = [col for col in train_price_processed.columns if col.startswith('建筑结构_') and col not in ['建筑结构_板楼', '建筑结构_平房', '建筑结构_塔楼', '建筑结构_塔板结合', '建筑结构_comm']]
print(sample_price_simple[simple_cols].head())

# 2. 验证多标签 (Price['建筑结构_comm'])
print("\n验证多标签 ('塔楼/板楼/平房'):")
sample_price_multi = train_price_processed[
    train_price['建筑结构_comm'] == '塔楼/板楼/平房'
]
multi_cols = ['建筑结构_comm', '建筑结构_板楼', '建筑结构_平房', '建筑结构_塔楼', '建筑结构_塔板结合']
print(sample_price_multi[multi_cols].head())


print("\n--- 验证 租金训练集 (train_rent) 的提取结果 ---")
# 验证多标签 (Rent['建筑结构'])
print("\n验证多标签 ('板楼/平房'):")
sample_rent_multi = train_rent_processed[
    train_rent['建筑结构'] == '板楼/平房'
]
rent_multi_cols = ['建筑结构', '建筑结构_板楼', '建筑结构_平房', '建筑结构_塔楼', '建筑结构_塔板结合']
print(sample_rent_multi[rent_multi_cols].head())

开始提取建筑结构特征...

--- 处理 Price 数据集 ---
已处理 (简单类别) '建筑结构': 新增 7 个哑变量。
已处理 (简单类别) '建筑结构': 新增 7 个哑变量。
已处理 (多标签) '建筑结构_comm': 新增 4 个哑变量。
已处理 (多标签) '建筑结构_comm': 新增 4 个哑变量。

--- 处理 Rent 数据集 ---
已处理 (多标签) '建筑结构': 新增 4 个哑变量。
已处理 (多标签) '建筑结构': 新增 4 个哑变量。

所有建筑结构数据处理完毕！

--- 验证 房价训练集 (train_price) 的提取结果 ---

验证简单类别 ('nan' 和 '未知结构'):
     建筑结构_未知结构  建筑结构_框架结构  建筑结构_混合结构  建筑结构_砖木结构  建筑结构_砖混结构  建筑结构_钢混结构  \
7         True      False      False      False      False      False   
71        True      False      False      False      False      False   
84        True      False      False      False      False      False   
159       True      False      False      False      False      False   
162       True      False      False      False      False      False   

     建筑结构_钢结构  
7       False  
71      False  
84      False  
159     False  
162     False  

验证多标签 ('塔楼/板楼/平房'):
    建筑结构_comm  建筑结构_板楼  建筑结构_平房  建筑结构_塔楼  建筑结构_塔板结合
18   塔楼/板楼/平房        1        1        1          0
215  塔楼/板楼/平房     

In [38]:
train_price_processed['装修情况'].unique()


array(['精装', '简装', nan, '毛坯', '其他'], dtype=object)

In [39]:
train_rent_processed['装修'].unique()

array(['精装修', nan], dtype=object)

In [41]:
#%%
# --------------------------------------------------
# 装修情况特征提取 (统一标准并创建哑变量)
# --------------------------------------------------
print("开始提取装修特征...")

# 定义所有统一后的装修类别
ALL_DECORATION_CATEGORIES = ['精装', '简装', '毛坯', '其他', '未知']

def process_decoration(df, column_name):
    """
    统一处理装修特征。
    - Price('装修情况'): '精装', '简装', nan, '毛坯', '其他'
    - Rent('装修'): '精装修', nan
    
    统一标准:
    1. '精装修' -> '精装'
    2. nan -> '未知'
    3. 确保 '精装', '简装', '毛坯', '其他', '未知' 都在哑变量中。
    
    参数:
    - df: DataFrame
    - column_name: '装修情况' 或 '装修'
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 复制 Series 并处理 Rent 数据中的 '精装修'
    s = df_copy[column_name]
    if column_name == '装修': # 仅 Rent 数据集
        s = s.replace('精装修', '精装')
        
    # 2. 将 nan 统一替换为 '未知'
    s = s.fillna('未知')
    
    # 3. 转换为 Categorical 类型
    #    这是关键步骤：
    #    通过 categories=ALL_DECORATION_CATEGORIES，我们告诉 pandas
    #    这个 Series 只能包含这些值。
    #    这能确保 get_dummies 会为所有 5 个类别创建列，
    #    即使某个类别在当前数据中（如 Rent 中的 '简装'）不存在。
    s_categorical = pd.Categorical(s, categories=ALL_DECORATION_CATEGORIES)
    
    # 4. 创建哑变量
    #    prefix='装修' 将创建 '装修_精装', '装修_简装', ... '装修_未知'
    dummies = pd.get_dummies(s_categorical, prefix='装修')
    
    # 5. 合并
    df_copy = pd.concat([df_copy, dummies], axis=1)
    
    print(f"已处理 '{column_name}': 新增 {len(dummies.columns)} 个统一标准的装修哑变量。")
    return df_copy

# %%
# 应用到所有四个数据帧
print("\n处理房价数据 (price) 的装修...")
train_price_processed = process_decoration(train_price_processed, '装修情况')
test_price_processed = process_decoration(test_price_processed, '装修情况')

print("\n处理租金数据 (rent) 的装修...")
train_rent_processed = process_decoration(train_rent_processed, '装修')
test_rent_processed = process_decoration(test_rent_processed, '装修')

print("\n所有装修数据处理完毕！")

# %%
# 验证提取结果
decoration_cols_to_show = ['装修_精装', '装修_简装', '装修_毛坯', '装修_其他', '装修_未知']

print(f"\n--- 验证 房价训练集 (train_price) 的装修提取 ---")
# 1. 检查 'nan' (空值) 样本
print("\n'nan' (空值) 样本:")
nan_sample_price = train_price_processed[train_price['装修情况'].isna()]
print(nan_sample_price[decoration_cols_to_show].head())

# 2. 检查 '简装' 样本
print("\n'简装' 样本:")
sample_price_simple = train_price_processed[train_price['装修情况'] == '简装']
print(sample_price_simple[decoration_cols_to_show].head())


print(f"\n--- 验证 租金训练集 (train_rent) 的装修提取 ---")
print(f"创建的列: {[col for col in train_rent_processed.columns if col.startswith('装修_')]}")

# 1. 检查 'nan' (空值) 样本
print("\n'nan' (空值) 样本:")
nan_sample_rent = train_rent_processed[train_rent['装修'].isna()]
print(nan_sample_rent[decoration_cols_to_show].head())

# 2. 检查 '精装修' 样本 (应被映射到 '装修_精装')
print("\n'精装修' 样本:")
sample_rent_luxury = train_rent_processed[train_rent['装修'] == '精装修']
print(sample_rent_luxury[decoration_cols_to_show].head())

# 3. 检查 '简装' 列 (应该存在且全为 0)
print(f"\n'装修_简装' 列在 train_rent 中的总和: {train_rent_processed['装修_简装'].sum()}")

开始提取装修特征...

处理房价数据 (price) 的装修...
已处理 '装修情况': 新增 5 个统一标准的装修哑变量。
已处理 '装修情况': 新增 5 个统一标准的装修哑变量。

处理租金数据 (rent) 的装修...
已处理 '装修': 新增 5 个统一标准的装修哑变量。
已处理 '装修': 新增 5 个统一标准的装修哑变量。

所有装修数据处理完毕！

--- 验证 房价训练集 (train_price) 的装修提取 ---

'nan' (空值) 样本:
     装修_精装  装修_精装  装修_简装  装修_简装  装修_毛坯  装修_毛坯  装修_其他  装修_其他  装修_未知  装修_未知
7    False  False  False  False  False  False  False  False   True   True
71   False  False  False  False  False  False  False  False   True   True
84   False  False  False  False  False  False  False  False   True   True
159  False  False  False  False  False  False  False  False   True   True
162  False  False  False  False  False  False  False  False   True   True

'简装' 样本:
    装修_精装  装修_精装  装修_简装  装修_简装  装修_毛坯  装修_毛坯  装修_其他  装修_其他  装修_未知  装修_未知
2   False  False   True   True  False  False  False  False  False  False
9   False  False   True   True  False  False  False  False  False  False
13  False  False   True   True  False  False  False  False  False  False
14  False  Fal

In [42]:
train_price_processed['交易时间'].unique()

array(['2021-03-29', '2020-10-29', '2020-11-24', ..., '2020-12-07',
       '2021-01-07', '2021-02-02'], dtype=object)

In [44]:
train_rent_processed['交易时间'].unique()

array(['2024-11-28', '2024-10-30', '2024-11-12', '2024-10-14',
       '2024-12-08', '2024-10-13', '2024-10-22', '2024-06-01',
       '2024-06-02', '2024-05-31', '2024-05-28', '2024-10-25',
       '2024-11-19', '2024-10-10', '2024-10-29', '2024-05-30',
       '2024-05-19', '2024-06-03', '2024-10-20', '2024-11-05',
       '2024-10-31', '2024-12-01', '2024-06-04', '2024-05-20',
       '2024-12-06', '2024-05-26', '2024-06-06', '2024-11-11',
       '2024-11-26', '2024-12-07', '2024-10-19', '2024-12-18',
       '2024-10-21', '2024-12-22', '2024-05-27', '2024-05-29',
       '2024-12-16', '2024-07-03', '2024-10-26', '2024-06-05',
       '2024-07-11', '2024-06-15', '2024-10-23', '2025-02-12',
       '2024-05-24', '2024-09-06', '2024-11-20', '2024-12-27',
       '2024-08-16', '2024-11-23', '2024-05-22', '2024-05-21',
       '2024-05-25', '2024-06-11', '2024-12-11', '2024-11-13',
       '2024-05-23', '2024-06-13', '2024-10-16', '2024-06-28',
       '2024-09-13', '2024-09-27', '2024-09-18', '2024-

In [45]:
#%%
# --------------------------------------------------
# 交易时间特征提取
# --------------------------------------------------
print("开始提取交易时间特征 (年份, 月份)...")

def process_transaction_date(df, column_name):
    """
    从 '交易时间' (格式如 '2021-03-29') 中提取 '交易年份' 和 '交易月份'。
    
    1. 使用 pd.to_datetime 转换列。
    2. 使用 .dt.year 和 .dt.month 提取特征。
    3. 'nan' 或无效日期将被转换为 0。
    
    参数:
    - df: DataFrame
    - column_name: '交易时间'
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 转换为 datetime 对象
    #    errors='coerce' 会将任何 'nan' 或无效日期字符串转换为 NaT (Not a Time)
    s_datetime = pd.to_datetime(df_copy[column_name], errors='coerce')
    
    # 2. 提取年份和月份
    #    当 s_datetime 中有 NaT 时, .dt.year 和 .dt.month 会返回 NaN (浮点型)
    df_copy['交易年份'] = s_datetime.dt.year
    df_copy['交易月份'] = s_datetime.dt.month
    
    # 3. 填充空值 (来自 NaT) 并转换为整数
    #    我们用 0 来填充缺失的年份/月份，这是一种常见的缺失值处理方式
    df_copy['交易年份'] = df_copy['交易年份'].fillna(0).astype(int)
    df_copy['交易月份'] = df_copy['交易月份'].fillna(0).astype(int)
    
    print(f"已处理 '{column_name}': 新增 '交易年份' 和 '交易月份'")
    return df_copy

# %%
# 应用到所有四个数据帧
print("\n处理房价数据 (price) 的交易时间...")
train_price_processed = process_transaction_date(train_price_processed, '交易时间')
test_price_processed = process_transaction_date(test_price_processed, '交易时间')

print("\n处理租金数据 (rent) 的交易时间...")
train_rent_processed = process_transaction_date(train_rent_processed, '交易时间')
test_rent_processed = process_transaction_date(test_rent_processed, '交易时间')

print("\n所有交易时间数据处理完毕！")

# %%
# 验证提取结果
print("\n--- 验证 房价训练集 (train_price) 的交易时间提取 ---")
cols_to_show_price = ['交易时间', '交易年份', '交易月份']
print(train_price_processed[cols_to_show_price].head())

# 检查 'nan' (空值) 样本
if train_price['交易时间'].isna().any():
    print("\n'nan' (空值) 样本:")
    nan_sample_price = train_price_processed[train_price['交易时间'].isna()]
    print(nan_sample_price[cols_to_show_price].head())
else:
    print("\n房价训练集 '交易时间' 中未发现 'nan'。")

print("\n--- 验证 租金训练集 (train_rent) 的交易时间提取 ---")
cols_to_show_rent = ['交易时间', '交易年份', '交易月份']
print(train_rent_processed[cols_to_show_rent].head())

if train_rent['交易时间'].isna().any():
    print("\n'nan' (空值) 样本:")
    nan_sample_rent = train_rent_processed[train_rent['交易时间'].isna()]
    print(nan_sample_rent[cols_to_show_rent].head())
else:
    print("\n租金训练集 '交易时间' 中未发现 'nan'。")

开始提取交易时间特征 (年份, 月份)...

处理房价数据 (price) 的交易时间...
已处理 '交易时间': 新增 '交易年份' 和 '交易月份'
已处理 '交易时间': 新增 '交易年份' 和 '交易月份'

处理租金数据 (rent) 的交易时间...
已处理 '交易时间': 新增 '交易年份' 和 '交易月份'
已处理 '交易时间': 新增 '交易年份' 和 '交易月份'

所有交易时间数据处理完毕！

--- 验证 房价训练集 (train_price) 的交易时间提取 ---
         交易时间  交易年份  交易月份
0  2021-03-29  2021     3
1  2020-10-29  2020    10
2  2020-11-24  2020    11
3  2023-03-10  2023     3
4  2019-12-12  2019    12

房价训练集 '交易时间' 中未发现 'nan'。

--- 验证 租金训练集 (train_rent) 的交易时间提取 ---
         交易时间  交易年份  交易月份
0  2024-11-28  2024    11
1  2024-10-30  2024    10
2  2024-11-12  2024    11
3  2024-10-14  2024    10
4  2024-12-08  2024    12

租金训练集 '交易时间' 中未发现 'nan'。


In [ ]:
train_price_processed[['年份','交易年份']].head()

,年份,交易年份
0,2018.0,2021
1,2017.0,2020
2,2018.0,2020
3,2020.0,2023
4,2017.0,2019


In [50]:
#%%
# --------------------------------------------------
# 重命名 '年份' 列
# --------------------------------------------------
print("开始重命名 '年份' 列为 '房屋登记时间'...")

old_col_name = '年份'
new_col_name = '房屋登记时间'

try:
    # 房价数据集
    train_price_processed = train_price_processed.rename(columns={old_col_name: new_col_name})
    test_price_processed = test_price_processed.rename(columns={old_col_name: new_col_name})
    print(f"已重命名 'train_price_processed' 中的 '{old_col_name}' -> '{new_col_name}'")
    print(f"已重命名 'test_price_processed' 中的 '{old_col_name}' -> '{new_col_name}'")
    
    # 租金数据集
    train_rent_processed = train_rent_processed.rename(columns={old_col_name: new_col_name})
    test_rent_processed = test_rent_processed.rename(columns={old_col_name: new_col_name})
    print(f"已重命名 'train_rent_processed' 中的 '{old_col_name}' -> '{new_col_name}'")
    print(f"已重命名 'test_rent_processed' 中的 '{old_col_name}' -> '{new_col_name}'")
    
    print("\n所有列重命名完毕！")
    
    # %%
    # 验证重命名结果
    print("\n--- 验证重命名后的列名 ---")
    print(f"'train_price_processed' 中是否存在 '{new_col_name}': {new_col_name in train_price_processed.columns}")
    print(f"'train_rent_processed' 中是否存在 '{new_col_name}': {new_col_name in train_rent_processed.columns}")
    print(f"'train_price_processed' 中是否存在 '{old_col_name}': {old_col_name in train_price_processed.columns}")
    
except KeyError:
    print(f"错误：原始列 '{old_col_name}' 在某个 DataFrame 中不存在。请检查之前的代码。")

开始重命名 '年份' 列为 '房屋登记时间'...
已重命名 'train_price_processed' 中的 '年份' -> '房屋登记时间'
已重命名 'test_price_processed' 中的 '年份' -> '房屋登记时间'
已重命名 'train_rent_processed' 中的 '年份' -> '房屋登记时间'
已重命名 'test_rent_processed' 中的 '年份' -> '房屋登记时间'

所有列重命名完毕！

--- 验证重命名后的列名 ---
'train_price_processed' 中是否存在 '房屋登记时间': True
'train_rent_processed' 中是否存在 '房屋登记时间': True
'train_price_processed' 中是否存在 '年份': False


In [51]:
train_price_processed['物业类别'].unique()

array(['普通住宅/平房', '普通住宅/商业/底商', '普通住宅/写字楼/商业/底商/地下仓储/库房', '普通住宅/别墅',
       '车库/普通住宅/别墅/写字楼/商业办公类', '车库/普通住宅/工业厂房/底商/地下仓储/库房',
       '车库/普通住宅/公寓/公寓（住宅）/公寓/住宅/公寓/公寓', '普通住宅/别墅/商业/底商', '车库/普通住宅/商业/底商',
       '普通住宅/写字楼/商业', '普通住宅/商业', '普通住宅/商业/平房', '普通住宅/商业办公类/公寓/住宅', '普通住宅',
       '普通住宅/写字楼/商业/商业办公类', '车库/普通住宅/底商', '车库/普通住宅', '车库/普通住宅/商业',
       '车库/普通住宅/写字楼/商业/商业办公类/公寓/公寓/住宅/公寓/公寓', '车库/普通住宅/写字楼/商业/底商/商业办公类',
       '车库/普通住宅/商业办公类', '普通住宅/写字楼/商业办公类', '普通住宅/平房/商业办公类/公寓/住宅',
       '普通住宅/底商', '普通住宅/商业/底商/商业办公类', '普通住宅/写字楼', '普通住宅/公寓/住宅',
       '普通住宅/别墅/商业/底商/商业办公类', '普通住宅/公寓/酒店式公寓/公寓（住宅）/公寓/住宅', '普通住宅/别墅/商业',
       '车库/普通住宅/写字楼/商业/底商/商业办公类/公寓/公寓/住宅/公寓/公寓/地下仓储/库房',
       '车库/普通住宅/写字楼/底商/商业办公类', '普通住宅/写字楼/商业办公类/公寓', '普通住宅/四合院',
       '普通住宅/商业/商业办公类', '车库/普通住宅/底商/商业办公类/公寓/公寓（住宅）/公寓/住宅/公寓/公寓',
       '车库/普通住宅/别墅/商业/工业厂房/地下仓储/库房', '车库/普通住宅/工业厂房/商业办公类',
       '车库/普通住宅/底商/商业办公类', '车库/普通住宅/四合院/商业/底商', '普通住宅/别墅/写字楼/商业办公类',
       '车库/普通住宅/写字楼/商业/商业办公类/公寓（住宅）/公寓/住宅', '普通住宅/别墅/商业办公类',
     

In [52]:
train_rent_processed['物业类别'].unique()

array(['普通住宅/底商', '普通住宅/商业/底商', '车库/普通住宅/写字楼/商业/底商',
       '车库/普通住宅/底商/商业办公类/公寓/公寓（住宅）/公寓/住宅/公寓/公寓', '普通住宅',
       '普通住宅/商业办公类/公寓/住宅', '车库/普通住宅/商业', '车库/普通住宅/商业/底商/商业办公类',
       '车库/普通住宅/写字楼/商业/商业办公类/公寓/公寓/住宅/公寓/公寓', '普通住宅/商业',
       '普通住宅/商业/工业厂房/商业办公类', '普通住宅/底商/商业办公类', '车库/普通住宅/商业/底商',
       '普通住宅/写字楼/商业办公类/公寓',
       '车库/普通住宅/商业/底商/商业办公类/公寓/酒店式公寓/住宅式公寓/公寓（住宅）/公寓/住宅/公寓/公寓/地下仓储/库房',
       '车库/普通住宅/四合院/商业/底商', '车库/普通住宅/别墅',
       '车库/普通住宅/写字楼/商业/底商/商业办公类/公寓/公寓/住宅/公寓/公寓/地下仓储/库房', '普通住宅/写字楼/工业厂房',
       '普通住宅/写字楼/商业/底商/商业办公类', '普通住宅/四合院/平房/底商', nan, '普通住宅/别墅',
       '普通住宅/平房', '车库/普通住宅/商业/底商/商业办公类/公寓/住宅', '商业/商业办公类',
       '普通住宅/写字楼/商业办公类/公寓/酒店式公寓/住宅式公寓/公寓（住宅）/公寓/住宅',
       '车库/普通住宅/写字楼/商业/底商/商业办公类', '车库/普通住宅', '普通住宅/商业/底商/商业办公类',
       '普通住宅/平房/底商', '普通住宅/写字楼/商业/底商/地下仓储/库房',
       '普通住宅/写字楼/商业/商业办公类/公寓/公寓/住宅', '普通住宅/公寓/住宅/地下仓储/库房', '普通住宅/写字楼/商业',
       '普通住宅/商业/底商/公寓/公寓（住宅）/公寓/住宅/公寓/公寓',
       '普通住宅/写字楼/商业办公类/酒店式公寓/公寓/公寓/商务型公寓', '普通住宅/公寓/住宅',
       '普通住宅/写字楼/商业/商业办公类',

In [53]:
#%%
# --------------------------------------------------
# 物业类别特征提取 (标准化与多标签哑变量)
# --------------------------------------------------
print("开始提取物业类别特征...")

from sklearn.preprocessing import MultiLabelBinarizer

def normalize_property_type(s_col):
    """
    辅助函数：标准化物业类别字符串。
    1. 替换别名 ('公寓（住宅）' -> '公寓')
    2. 处理 '住宅' -> '公寓' (避免替换 '普通住宅')
    3. 移除重复项
    
    返回: 一个包含干净类别列表的 Series
    """
    # 1. Handle nan
    s = s_col.fillna('')
    
    # 2. 定义别名替换
    #    (从长到短排序，防止 '公寓' 替换 '公寓（住宅）' 的一部分)
    replacements = {
        '单身公寓（住宅）': '公寓',
        '公寓（住宅）': '公寓',
        '酒店式公寓': '公寓',
        '住宅式公寓': '公寓',
        '商务型公寓': '公寓',
        '商务公寓': '公寓',
        '库房': '地下仓储',
    }
    
    for old, new in replacements.items():
        s = s.str.replace(old, new)
        
    # 3. 定义一个内部函数，用于分割、替换 '住宅' 并获取唯一集合
    def clean_parts(text):
        if not text:
            return []
        parts = text.split('/')
        new_parts = set()
        for part in parts:
            if part == '住宅':
                new_parts.add('公寓')
            elif part: # 避免添加空字符串
                new_parts.add(part)
        return list(new_parts) # 返回一个列表

    # 4. 应用此内部函数
    #    这将返回一个 Series，其中每个元素都是一个列表, e.g., ['公寓', '普通住宅']
    return s.apply(clean_parts)

# --- 关键步骤：在所有数据集中找到所有唯一的、标准化的类别 ---

print("正在分析所有数据集中的物业类别...")
# 1. 标准化所有四个数据集中的列
norm_price_train = normalize_property_type(train_price['物业类别'])
norm_price_test = normalize_property_type(test_price['物业类别'])
norm_rent_train = normalize_property_type(train_rent['物业类别'])
norm_rent_test = normalize_property_type(test_rent['物业类别'])

# 2. 将所有列表合并到一个 Series 中
all_lists = pd.concat([norm_price_train, norm_price_test, norm_rent_train, norm_rent_test])

# 3. 遍历所有列表，找到所有唯一的类别
all_categories = set()
all_lists.apply(all_categories.update)

# 4. 移除空字符串并排序，创建最终的类别列表
all_categories.discard('')
ALL_PROPERTY_TYPES = sorted(list(all_categories))

print(f"已找到 {len(ALL_PROPERTY_TYPES)} 个统一的物业类别。")
# print(ALL_PROPERTY_TYPES) # (取消注释以查看所有类别)

# --- 定义主处理函数 ---

def process_property_type(df, column_name, all_types_list):
    """
    使用 MultiLabelBinarizer 处理标准化的物业类别列表。
    
    参数:
    - df: DataFrame
    - column_name: '物业类别'
    - all_types_list: 预先计算好的 ALL_PROPERTY_TYPES 列表
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 标准化当前 DataFrame 的列 (返回一个列表的 Series)
    normalized_lists = normalize_property_type(df_copy[column_name])
    
    # 2. 初始化 Binarizer，强制使用所有类别
    mlb = MultiLabelBinarizer(classes=all_types_list)
    
    # 3. 转换数据
    dummies_data = mlb.fit_transform(normalized_lists)
    
    # 4. 创建列名 (例如 '物业类别_公寓')
    dummy_cols = [f'物业类别_{c}' for c in mlb.classes_]
    
    # 5. 转为 DataFrame
    dummies_df = pd.DataFrame(dummies_data, columns=dummy_cols, index=df_copy.index)
    
    # 6. 合并
    df_copy = pd.concat([df_copy, dummies_df], axis=1)
    
    print(f"已处理 '{column_name}': 新增 {len(dummy_cols)} 个物业类别哑变量。")
    return df_copy

# %%
# --- 应用处理 ---
print("\n处理房价数据 (price) 的物业类别...")
train_price_processed = process_property_type(train_price_processed, '物业类别', ALL_PROPERTY_TYPES)
test_price_processed = process_property_type(test_price_processed, '物业类别', ALL_PROPERTY_TYPES)

print("\n处理租金数据 (rent) 的物业类别...")
train_rent_processed = process_property_type(train_rent_processed, '物业类别', ALL_PROPERTY_TYPES)
test_rent_processed = process_property_type(test_rent_processed, '物业类别', ALL_PROPERTY_TYPES)

print("\n所有物业类别数据处理完毕！")

# %%
# --- 验证提取结果 ---
print("\n--- 验证 房价训练集 (train_price) 的物业类别提取 ---")

# 1. 验证一个复杂的 '公寓' 样本
# 原始数据中包含 '普通住宅/公寓/公寓（住宅）/公寓/住宅' 的行
complex_apartment_sample = train_price_processed[
    train_price['物业类别'].astype(str).str.contains('公寓（住宅）', na=False)
]
cols_to_show = ['物业类别_普通住宅', '物业类别_公寓', '物业类别_住宅']
# 筛选出实际存在的列进行显示
cols_to_show_existing_price = [col for col in cols_to_show if col in complex_apartment_sample.columns]
print("\n复杂公寓样本 (应标记 '普通住宅' 和 '公寓', 不应有 '住宅'):")
print(complex_apartment_sample[cols_to_show_existing_price].head())

# 2. 验证 '库房' 样本
# 原始数据中包含 '库房' 的行
storage_sample = train_price_processed[
    train_price['物业类别'].astype(str).str.contains('库房', na=False)
]
cols_to_show = ['物业类别_地下仓储', '物业类别_库房']
cols_to_show_existing_storage = [col for col in cols_to_show if col in storage_sample.columns]
print("\n'库房' 样本 (应标记 '地下仓储'):")
print(storage_sample[cols_to_show_existing_storage].head())

print("\n--- 验证 租金训练集 (train_rent) 的列是否一致 ---")
rent_cols = [col for col in train_rent_processed.columns if col.startswith('物业类别_')]
print(f"租金训练集创建了 {len(rent_cols)} 个物业类别列。")

if len(rent_cols) == len(ALL_PROPERTY_TYPES):
    print("✓ 验证通过：租金数据集的列数与总类别数一致。")
else:
    print("✗ 验证失败：租金数据集的列数与总类别数不一致。")

开始提取物业类别特征...
正在分析所有数据集中的物业类别...
已找到 18 个统一的物业类别。

处理房价数据 (price) 的物业类别...
已处理 '物业类别': 新增 18 个物业类别哑变量。
已处理 '物业类别': 新增 18 个物业类别哑变量。

处理租金数据 (rent) 的物业类别...
已处理 '物业类别': 新增 18 个物业类别哑变量。
已处理 '物业类别': 新增 18 个物业类别哑变量。

所有物业类别数据处理完毕！

--- 验证 房价训练集 (train_price) 的物业类别提取 ---

复杂公寓样本 (应标记 '普通住宅' 和 '公寓', 不应有 '住宅'):
     物业类别_普通住宅  物业类别_公寓
8            1        1
51           1        1
79           1        1
96           1        1
174          1        1

'库房' 样本 (应标记 '地下仓储'):
    物业类别_地下仓储
2           1
6           1
7           1
61          1
83          1

--- 验证 租金训练集 (train_rent) 的列是否一致 ---
租金训练集创建了 18 个物业类别列。
✓ 验证通过：租金数据集的列数与总类别数一致。


In [54]:
train_price_processed['建筑年代'].unique()

array(['1955-2000年', '2005年', '2011-2012年', '2009-2011年', '2003-2018年',
       '2007-2010年', '2011-2014年', '1997-1998年', '2004-2006年',
       '2006-2011年', '2006-2010年', '2007-2009年', '1980-2000年',
       '2002-2006年', '2010-2012年', '1997-2002年', '2000-2017年',
       '1989-2008年', '1954-1995年', '1994-2012年', '2008-2009年',
       '1999-2001年', '2014-2015年', '1997-2003年', '2012-2015年',
       '2005-2014年', '1960-1996年', '1998-2003年', '2006-2013年',
       '1982-1996年', '1996-2003年', '1963-2004年', '2003-2012年',
       '2012-2018年', '2017-2019年', '2005-2008年', '1987-1990年', '1992年',
       '2003-2015年', '2002-2004年', '2014-2016年', '1995-1996年', '2008年',
       '1987-2007年', '1978-2000年', '1999-2002年', '2004-2007年',
       '2015-2018年', '2001-2008年', '1979-2002年', '2012-2016年',
       '2015-2017年', '2009-2012年', '1981-1989年', '1999-2009年',
       '2006-2016年', '1976-1989年', '1955-1996年', '1995-2008年', '2002年',
       '2000-2006年', '2002-2005年', '1996-1999年', '1982-1985年',
       '2008-2014年'

In [55]:
train_rent_processed['建筑年代'].unique()

array(['1963-2001年', '1988-2002年', '2004-2009年', '2015-2018年',
       '1980-1996年', '1985-1993年', '1997-1998年', '2002-2006年',
       '2012-2015年', '1970-1997年', '2002-2008年', '1997-2000年',
       '2008-2009年', '2007-2008年', '1994-2000年', '1999-2001年',
       '1989-2000年', '1980-1999年', '2006-2007年', '2004-2007年', '2008年',
       '1956-1998年', '1997-2009年', '2014-2018年', '1998-2000年',
       '1999-2009年', '1990-2001年', '1990-1997年', '1996-2007年',
       '2005-2017年', '2005-2008年', '1994-2002年', '2014-2016年',
       '1955-1996年', '1967-1998年', '1998-1999年', '1993-1997年',
       '2010-2012年', '1987-2003年', nan, '1990-1998年', '1979年',
       '1980-1990年', '2019-2022年', '1986-1990年', '2006-2009年',
       '2005-2014年', '2012-2013年', '1985-2000年', '1979-2000年',
       '1980-2001年', '2006-2011年', '2002-2004年', '2008-2014年',
       '1980-1998年', '1982-1996年', '2001-2002年', '1996-1998年',
       '1978-2000年', '1978-2002年', '1981-1992年', '2003-2006年',
       '2009-2010年', '2007-2009年', '2015-2016年

In [56]:
#%%
# --------------------------------------------------
# 建筑年代特征提取
# --------------------------------------------------
print("开始提取建筑年代特征...")

# 确保 numpy 已经导入
import numpy as np

def process_build_year(df, column_name):
    """
    处理 '建筑年代' (格式如 '2005年' 或 '1955-2000年')。
    
    1. 提取第一个年份 (year1)。
    2. 提取第二个年份 (year2), 如果存在。
    3. 如果 'nan' 或无法解析，结果为 0。
    4. 如果只有 year1 (如 '2005年')，新值为 year1。
    5. 如果有 year1 和 year2 (如 '1955-2000年')，新值为 (year1 + year2) / 2 并四舍五入。
    
    参数:
    - df: DataFrame
    - column_name: '建筑年代'
    
    返回:
    - 带有新特征列 '建筑年份' 的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 转换为字符串，'nan' -> 'nan'
    s = df_copy[column_name].astype(str)
    
    # 2. 定义正则表达式
    #    ^(\d{4})        : 捕获组 1 - 匹配开头的4位数字 (第一个年份)
    #    (?:-(\d{4}))?   : 一个可选的非捕获组
    #        -           : 匹配连字符
    #        (\d{4})     : 捕获组 2 - 匹配4位数字 (第二个年份)
    #    年?$            : 匹配可选的 '年' 字符直到结尾
    pattern = r'^(\d{4})(?:-(\d{4}))?年?$'
    
    # 3. 提取年份
    #    这将返回一个有两列的 DataFrame (捕获组1, 捕获组2)
    #    '2005年' -> ('2005', nan)
    #    '1955-2000年' -> ('1955', '2000')
    #    'nan' -> (nan, nan)
    extracted_df = s.str.extract(pattern)
    
    # 4. 将提取的字符串转换为数值
    #    errors='coerce' 会将 nan 和无效字符串转换为 NaN (数值型)
    year1 = pd.to_numeric(extracted_df[0], errors='coerce')
    year2 = pd.to_numeric(extracted_df[1], errors='coerce')
    
    # 5. 应用逻辑
    #    np.where(condition, value_if_true, value_if_false)
    #    condition: year2.isna() (第二个年份是否为空？)
    #    value_if_true: year1 (是, 说明是单一
    #    年份，直接用 year1)
    #    value_if_false: (year1 + year2) / 2 (否, 说明是年份范围, 计算平均值)
    result = np.where(year2.isna(), year1, (year1 + year2) / 2)
    
    # 6. 清理结果
    #    pd.Series(result) : 将 numpy 数组转回 Series
    #    .round()          : 对平均值进行四舍五入
    #    .fillna(0)        : 将所有 NaN (来自 'nan' 字符串) 替换为 0
    #    .astype(int)      : 转换为整数
    df_copy['建筑年份'] = pd.Series(result).round().fillna(0).astype(int)
    
    print(f"已处理 '{column_name}': 新增 '建筑年份'")
    return df_copy

# %%
# 应用到所有四个数据帧
print("\n处理房价数据 (price) 的建筑年代...")
train_price_processed = process_build_year(train_price_processed, '建筑年代')
test_price_processed = process_build_year(test_price_processed, '建筑年代')

print("\n处理租金数据 (rent) 的建筑年代...")
train_rent_processed = process_build_year(train_rent_processed, '建筑年代')
test_rent_processed = process_build_year(test_rent_processed, '建筑年代')

print("\n所有建筑年代数据处理完毕！")

# %%
# 验证提取结果
print("\n--- 验证 房价训练集 (train_price) 的建筑年代提取 ---")
cols_to_show = ['建筑年代', '建筑年份']

# 1. 检查 'nan' (空值) 样本
print("\n'nan' (空值) 样本 (应为 0):")
nan_sample_price = train_price_processed[train_price['建筑年代'].isna()]
print(nan_sample_price[cols_to_show].head())

# 2. 检查 '单一' 年份样本
print("\n'单一' 年份样本 (如 '2005年'):")
sample_price_single = train_price_processed[train_price['建筑年代'] == '2005年']
print(sample_price_single[cols_to_show].head())

# 3. 检查 '年份范围' 样本
print("\n'年份范围' 样本 (如 '1955-2000年', 应为 (1955+2000)/2 = 1977.5 -> 1978):")
sample_price_range = train_price_processed[train_price['建筑年代'] == '1955-2000年']
print(sample_price_range[cols_to_show].head())

开始提取建筑年代特征...

处理房价数据 (price) 的建筑年代...
已处理 '建筑年代': 新增 '建筑年份'
已处理 '建筑年代': 新增 '建筑年份'

处理租金数据 (rent) 的建筑年代...
已处理 '建筑年代': 新增 '建筑年份'
已处理 '建筑年代': 新增 '建筑年份'

所有建筑年代数据处理完毕！

--- 验证 房价训练集 (train_price) 的建筑年代提取 ---

'nan' (空值) 样本 (应为 0):
    建筑年代  建筑年份
287  NaN     0
372  NaN     0
420  NaN     0
424  NaN     0
506  NaN     0

'单一' 年份样本 (如 '2005年'):
      建筑年代  建筑年份
1    2005年  2005
432  2005年  2005
571  2005年  2005
641  2005年  2005
774  2005年  2005

'年份范围' 样本 (如 '1955-2000年', 应为 (1955+2000)/2 = 1977.5 -> 1978):
            建筑年代  建筑年份
0     1955-2000年  1978
778   1955-2000年  1978
1346  1955-2000年  1978
1362  1955-2000年  1978
1419  1955-2000年  1978


In [57]:
train_price_processed[['房屋总数','楼栋总数']].head()

,房屋总数,楼栋总数
0,1317户,19栋
1,2317户,40栋
2,1554户,20栋
3,66户,27栋
4,1685户,19栋


In [58]:
train_rent_processed[['房屋总数','楼栋总数']].head()


,房屋总数,楼栋总数
0,1731户,19栋
1,1931户,9栋
2,1891户,12栋
3,3026户,10栋
4,3031户,27栋


In [59]:
train_price_processed['房屋总数'].unique()

array(['1317户', '2317户', '1554户', ..., '1455户', '1511户', '1002户'],
      dtype=object)

In [60]:
#%%
# --------------------------------------------------
# 房屋总数 和 楼栋总数 特征提取
# --------------------------------------------------
print("开始提取 房屋总数 和 楼栋总数 特征...")

def process_unit_count(df, input_col, output_col):
    """
    从带单位的字符串 (如 '120户', '10栋') 中提取数值。
    
    1. 提取字符串开头的数字。
    2. 将新列转换为 int 类型。
    
    参数:
    - df: DataFrame
    - input_col: 信息所在的列名 (例如 '房屋总数')
    - output_col: 新建的数值列的名称
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 统一转为字符串，处理 np.nan
    s = df_copy[input_col].astype(str)
    
    # 2. 提取数字
    #    正则表达式 r'^(\d+)'
    #    ^     匹配字符串的开头
    #    (\d+) 是一个捕获组, 匹配一个或多个数字
    #    这会匹配 '120' from '120户'
    pattern = r'^(\d+)'
    extracted = s.str.extract(pattern, expand=False)
    
    # 3. 清理并转换为 int
    #    .fillna('0')：将所有 NaN (来自 'nan' 字符串或未匹配项) 替换为 '0'
    #    .astype(int)：将列从字符串转换为整数
    df_copy[output_col] = extracted.fillna('0').astype(int)
    
    print(f"已处理 '{input_col}', 新增 '{output_col}' (dtype: {df_copy[output_col].dtype})")
    return df_copy

# %%
# --- 应用处理 ---

# 1. 处理 '房屋总数'
print("\n处理 '房屋总数'...")
train_price_processed = process_unit_count(train_price_processed, '房屋总数', '房屋总数_数值')
test_price_processed = process_unit_count(test_price_processed, '房屋总数', '房屋总数_数值')
train_rent_processed = process_unit_count(train_rent_processed, '房屋总数', '房屋总数_数值')
test_rent_processed = process_unit_count(test_rent_processed, '房屋总数', '房屋总数_数值')

# 2. 处理 '楼栋总数'
print("\n处理 '楼栋总数'...")
train_price_processed = process_unit_count(train_price_processed, '楼栋总数', '楼栋总数_数值')
test_price_processed = process_unit_count(test_price_processed, '楼栋总数', '楼栋总数_数值')
train_rent_processed = process_unit_count(train_rent_processed, '楼栋总数', '楼栋总数_数值')
test_rent_processed = process_unit_count(test_rent_processed, '楼栋总数', '楼栋总数_数值')

print("\n所有 房屋总数 和 楼栋总数 数据处理完毕！")

# %%
# --- 验证提取结果 ---
print("\n--- 验证 房价训练集 (train_price) 的提取结果 ---")
cols_to_show_house = ['房屋总数', '房屋总数_数值']
cols_to_show_building = ['楼栋总数', '楼栋总数_数值']

print("\n房屋总数 样本:")
print(train_price_processed[cols_to_show_house].head())

print("\n楼栋总数 样本:")
print(train_price_processed[cols_to_show_building].head())

# 检查 'nan' (空值) 样本
if train_price['房屋总数'].isna().any():
    print("\n'房屋总数' 的 'nan' (空值) 样本 (应为 0):")
    nan_sample_price = train_price_processed[train_price['房屋总数'].isna()]
    print(nan_sample_price[cols_to_show_house].head())

开始提取 房屋总数 和 楼栋总数 特征...

处理 '房屋总数'...
已处理 '房屋总数', 新增 '房屋总数_数值' (dtype: int32)
已处理 '房屋总数', 新增 '房屋总数_数值' (dtype: int32)
已处理 '房屋总数', 新增 '房屋总数_数值' (dtype: int32)
已处理 '房屋总数', 新增 '房屋总数_数值' (dtype: int32)

处理 '楼栋总数'...
已处理 '楼栋总数', 新增 '楼栋总数_数值' (dtype: int32)
已处理 '楼栋总数', 新增 '楼栋总数_数值' (dtype: int32)
已处理 '楼栋总数', 新增 '楼栋总数_数值' (dtype: int32)
已处理 '楼栋总数', 新增 '楼栋总数_数值' (dtype: int32)

所有 房屋总数 和 楼栋总数 数据处理完毕！

--- 验证 房价训练集 (train_price) 的提取结果 ---

房屋总数 样本:
    房屋总数  房屋总数_数值
0  1317户     1317
1  2317户     2317
2  1554户     1554
3    66户       66
4  1685户     1685

楼栋总数 样本:
  楼栋总数  楼栋总数_数值
0  19栋       19
1  40栋       40
2  20栋       20
3  27栋       27
4  19栋       19

'房屋总数' 的 'nan' (空值) 样本 (应为 0):
     房屋总数  房屋总数_数值
1524  NaN        0
1773  NaN        0
1899  NaN        0
2035  NaN        0
2054  NaN        0


In [62]:
train_price_processed['绿 化 率'].unique()

array(['30%', '40.1%', '60%', '40%', '35%', '25%', '14.99%', '15%',
       '30.2%', '20%', '50%', '10%', '0.9%', '45%', '31%', '42%', '32%',
       '19%', '31.2%', '0.01%', '29.99%', '26%', nan, '36%', '80%', '33%',
       '19.99%', '32.5%', '64%', '1%', '28%', '30.6%', '48%', '30.1%',
       '18.75%', '38%', '21%', '12%', '24%', '41%', '27%', '3%', '50.1%',
       '76%', '18%', '43%', '5%', '35.1%', '37.4%', '8%', '22%', '46%',
       '24.5%', '2.6%', '31.42%', '16%', '11.2%', '2%', '0.1%', '33.6%',
       '52%', '16.1%', '10500%', '30.8%', '44.58%', '37%', '2.5%',
       '32.43%', '35.2%', '56.6%', '35.74%', '30.02%', '44%', '37.49%',
       '43.8%', '3.5%', '6%', '18.64%', '49.1%', '25.21%', '30.05%',
       '30.53%', '41.2%', '35.7%', '25.1%', '31.5%', '30.97%', '2.1%',
       '37.1%', '23%', '0.21%', '30.01%', '36.4%', '29%', '55%', '34.2%',
       '39%', '24.05%', '34%', '36.54%', '34.8%', '36.14%', '34.52%',
       '0.8%', '45.5%', '32.8%', '0.3%', '17.5%', '0.2%', '31.4%',
    

In [63]:
train_rent_processed['绿 化 率'].unique()

array(['30%', '35%', '25%', '10%', '14.99%', '20%', '33.6%', '22%', nan,
       '16%', '0.01%', '31.2%', '26%', '36%', '24%', '45%', '31%', '40%',
       '33%', '29.99%', '60%', '18%', '32%', '48%', '41%', '50.1%', '15%',
       '30.2%', '30.1%', '19.99%', '21%', '80%', '0.9%', '50%', '18.75%',
       '76%', '46%', '2.6%', '28%', '1%', '64%', '8%', '35.1%', '38%',
       '30.6%', '24.5%', '5%', '3%', '42%', '31.42%', '27%', '19%',
       '40.1%', '11.2%', '2%', '12%', '0.1%', '24.99%', '37%', '10500%',
       '16.1%', '52%', '32.43%', '30.8%', '44.58%', '35.74%', '56.6%',
       '2.5%', '35.2%', '31.5%', '24.05%', '44%', '30.02%', '27.5%',
       '43.8%', '37.1%', '30.97%', '23%', '25.21%', '35.7%', '32.5%',
       '18.64%', '55%', '34.8%', '36.4%', '30.01%', '41.2%', '17.5%',
       '34%', '2.1%', '30.53%', '32.8%', '36.14%', '29%', '36.54%',
       '0.21%', '3.5%', '25.1%', '45.5%', '31.4%', '43%', '39%', '37.49%',
       '34.52%', '34.2%', '37.5%', '65%', '45.62%', '12.5%', '49%',
 

In [64]:
#%%
# --------------------------------------------------
# 绿化率特征提取 (带异常值处理)
# --------------------------------------------------
print("开始提取 绿化率 特征...")

# 确保 numpy 已经导入
import numpy as np
import pandas as pd

def process_greening_rate(df, input_col, output_col, fill_mean=None):
    """
    处理 '绿 化 率' (如 '30%', '10500%')。
    
    1. 提取数字 (float)。
    2. 将 'nan' 和异常值 (如 > 100%) 设为 np.nan。
    3. 如果是训练集 (fill_mean is None):
        a. 计算有效数据 (0-100) 的平均值。
        b. 用此平均值填充所有 np.nan。
        c. 返回处理后的 df 和计算出的平均值。
    4. 如果是测试集 (fill_mean is provided):
        a. 用 'fill_mean' (来自训练集) 填充所有 np.nan。
        b. 返回处理后的 df 和 None。
    """
    df_copy = df.copy()
    
    # 1. 统一转为字符串，处理 np.nan
    s = df_copy[input_col].astype(str)
    
    # 2. 提取数字 (包括小数)
    pattern = r'(\d+\.?\d*)'
    extracted = s.str.extract(pattern, expand=False)
    
    # 3. 转换为 float，'nan' 和 '未知' 会变为 NaN
    numeric = pd.to_numeric(extracted, errors='coerce')
    
    # 4. 处理异常值：将大于 100% 的值设为 NaN，以便计算均值时排除它们
    numeric[numeric > 100] = np.nan
    
    calculated_mean = fill_mean
    
    if fill_mean is None:
        # 5a. (训练集) 计算有效值 (0-100 之间) 的平均值
        calculated_mean = numeric.mean()
        # 安全检查：如果所有值都是 nan 或 异常值
        if pd.isna(calculated_mean):
            print(f"警告: '{input_col}' 在训练集中没有有效数据。将使用 0 填充。")
            calculated_mean = 0
    
    # 5b. (训练集和测试集) 填充所有 NaN (包括原始nan和>100的异常值)
    df_copy[output_col] = numeric.fillna(calculated_mean)
    
    if fill_mean is None:
        print(f"已处理 (训练集) '{input_col}', 新增 '{output_col}'. "
              f"计算出的有效均值为: {calculated_mean:.2f}%")
        return df_copy, calculated_mean
    else:
        print(f"已处理 (测试集) '{input_col}', 新增 '{output_col}'. "
              f"使用训练集均值: {calculated_mean:.2f}% 填充。")
        return df_copy, None

# %%
# --- 应用处理 ---

input_col_name = '绿 化 率' # 注意空格
output_col_name = '绿化率%'

try:
    # 1. 处理 Price 数据集 (训练集 + 测试集)
    print("\n处理 Price 数据集 '绿 化 率'...")
    # 先处理训练集，获取均值
    train_price_processed, price_mean = process_greening_rate(
        train_price_processed, input_col_name, output_col_name, fill_mean=None
    )
    # 再处理测试集，传入均值
    test_price_processed, _ = process_greening_rate(
        test_price_processed, input_col_name, output_col_name, fill_mean=price_mean
    )

    # 2. 处理 Rent 数据集 (训练集 + 测试集)
    print("\n处理 Rent 数据集 '绿 化 率'...")
    # 先处理训练集，获取均值
    train_rent_processed, rent_mean = process_greening_rate(
        train_rent_processed, input_col_name, output_col_name, fill_mean=None
    )
    # 再处理测试集，传入均值
    test_rent_processed, _ = process_greening_rate(
        test_rent_processed, input_col_name, output_col_name, fill_mean=rent_mean
    )

    print("\n所有 绿化率 数据处理完毕！")

except KeyError as e:
    print(f"错误：找不到列 {e}。请确保列名完全正确，包括空格。")
except NameError as e:
    print(f"错误：{e}。请确保 'train_price_processed' 等 DataFrame 已被正确定义。")

# %%
# --- 验证提取结果 ---
print("\n--- 验证 房价训练集 (train_price) 的绿化率提取 ---")
cols_to_show = [input_col_name, output_col_name]

# 1. 检查 'nan' (空值) 样本 (应为均值)
if 'train_price' in locals() and train_price[input_col_name].isna().any():
    print(f"\n'nan' (空值) 样本 (应填充为均值 {price_mean:.2f}):")
    nan_sample_price = train_price_processed[train_price[input_col_name].isna()]
    print(nan_sample_price[cols_to_show].head())

# 2. 检查 '异常值' 样本 (如 '10500%') (应为均值)
if 'train_price' in locals():
    outlier_sample_price = train_price_processed[train_price[input_col_name] == '10500%']
    if not outlier_sample_price.empty:
        print(f"\n'异常值' (10500%) 样本 (应填充为均值 {price_mean:.2f}):")
        print(outlier_sample_price[cols_to_show].head())

# 3. 检查 '常规' 样本
if 'train_price' in locals():
    print("\n'常规' 样本 (如 '30%'):")
    regular_sample_price = train_price_processed[train_price[input_col_name] == '30%']
    print(regular_sample_price[cols_to_show].head())

开始提取 绿化率 特征...

处理 Price 数据集 '绿 化 率'...
已处理 (训练集) '绿 化 率', 新增 '绿化率%'. 计算出的有效均值为: 33.18%
已处理 (测试集) '绿 化 率', 新增 '绿化率%'. 使用训练集均值: 33.18% 填充。

处理 Rent 数据集 '绿 化 率'...
已处理 (训练集) '绿 化 率', 新增 '绿化率%'. 计算出的有效均值为: 33.10%
已处理 (测试集) '绿 化 率', 新增 '绿化率%'. 使用训练集均值: 33.10% 填充。

所有 绿化率 数据处理完毕！

--- 验证 房价训练集 (train_price) 的绿化率提取 ---

'nan' (空值) 样本 (应填充为均值 33.18):
    绿 化 率       绿化率%
241   NaN  33.181266
251   NaN  33.181266
283   NaN  33.181266
539   NaN  33.181266
635   NaN  33.181266

'异常值' (10500%) 样本 (应填充为均值 33.18):
        绿 化 率       绿化率%
16527  10500%  33.181266
16588  10500%  33.181266
16888  10500%  33.181266
16925  10500%  33.181266
17136  10500%  33.181266

'常规' 样本 (如 '30%'):
   绿 化 率  绿化率%
0    30%  30.0
1    30%  30.0
2    30%  30.0
11   30%  30.0
13   30%  30.0


In [66]:
#'容 积 率'
train_price_processed['容 积 率'].head()

0    3.00
1    1.73
2    1.70
3    1.00
4    1.58
Name: 容 积 率, dtype: float64

In [67]:
train_rent_processed['容 积 率'].head()

0    2.5
1    1.2
2    2.7
3    2.8
4    1.7
Name: 容 积 率, dtype: float64

In [70]:
#'物 业 费'
print(train_price_processed['物 业 费'].unique())

['1.3-1.65元/月/㎡' '0.65元/月/㎡' '1.98-2.98元/月/㎡' ... '19元/月/㎡'
 '0.45-1.2元/月/㎡' '0.48-0.5元/月/㎡']


In [71]:
#%%
# --------------------------------------------------
# 物业费 特征提取
# --------------------------------------------------
print("开始提取 物业费 特征...")

# 确保 numpy 已经导入
import numpy as np
import pandas as pd

def process_property_fee(df, input_col, output_col):
    """
    处理 '物 业 费' (格式如 '1.3-1.65元/月/㎡', '0.65元/月/㎡')。
    
    1. 提取第一个数字 (num1)。
    2. 提取第二个数字 (num2), 如果存在。
    3. 如果 'nan' 或无法解析，结果为 0。
    4. 如果只有 num1 (如 '0.65元/月/㎡')，新值为 num1。
    5. 如果有 num1 和 num2 (如 '1.3-1.65元/月/㎡')，新值为 (num1 + num2) / 2。
    
    参数:
    - df: DataFrame
    - input_col: '物 业 费' (带空格)
    - output_col: '物业费_数值'
    
    返回:
    - 带有新特征列 '物业费_数值' 的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 转换为字符串，'nan' -> 'nan'
    s = df_copy[input_col].astype(str)
    
    # 2. 定义正则表达式
    #    (\d+\.?\d*)   : 捕获组 1 - 匹配第一个数字 (float or int)
    #    (?:-(\d+\.?\d*))? : 可选的 ' - ' 和 捕获组 2 (第二个数字)
    #    我们只在字符串开头附近查找，不使用 ^ 来允许 "约 1.3" 这样的格式
    pattern = r'(\d+\.?\d*)(?:-(\d+\.?\d*))?'
    
    # 3. 提取数字
    #    返回一个有两列的 DataFrame (捕获组1, 捕获组2)
    #    '1.3-1.65...' -> ('1.3', '1.65')
    #    '0.65...'     -> ('0.65', nan)
    #    'nan'         -> (nan, nan)
    extracted_df = s.str.extract(pattern)
    
    # 4. 将提取的字符串转换为数值
    num1 = pd.to_numeric(extracted_df[0], errors='coerce')
    num2 = pd.to_numeric(extracted_df[1], errors='coerce')
    
    # 5. 应用逻辑
    #    np.where(condition, value_if_true, value_if_false)
    #    如果 num2 是 NaT/nan, 返回 num1, 否则返回 (num1 + num2) / 2
    result = np.where(num2.isna(), num1, (num1 + num2) / 2)
    
    # 6. 清理结果
    #    .fillna(0)  : 将所有 NaN (来自 'nan' 字符串) 替换为 0
    #    结果为 float 类型，这是正确的，因为费用可以是小数
    df_copy[output_col] = pd.Series(result).fillna(0)
    
    print(f"已处理 '{input_col}', 新增 '{output_col}' (dtype: {df_copy[output_col].dtype})")
    return df_copy

# %%
# --- 应用处理 ---

# !! 注意 !! 这里的列名 '物 业 费' 包含空格，与您提供的一致。
input_col_name = '物 业 费' 
output_col_name = '物业费_数值'

try:
    # 1. 处理 Price 数据集
    print("\n处理 Price 数据集 '物 业 费'...")
    train_price_processed = process_property_fee(
        train_price_processed, input_col_name, output_col_name
    )
    test_price_processed = process_property_fee(
        test_price_processed, input_col_name, output_col_name
    )
    
    # 2. 处理 Rent 数据集
    print("\n处理 Rent 数据集 '物 业 费'...")
    train_rent_processed = process_property_fee(
        train_rent_processed, input_col_name, output_col_name
    )
    test_rent_processed = process_property_fee(
        test_rent_processed, input_col_name, output_col_name
    )

    print("\n所有 物业费 数据处理完毕！")

except KeyError as e:
    print(f"错误：找不到列 {e}。请确保列名完全正确，包括空格。")
    print("如果您的列名是 '物业费' (没有空格)，请修改 'input_col_name' 变量。")
except NameError as e:
    print(f"错误：{e}。请确保 'train_price_processed' 等 DataFrame 已被正确定义。")
    print("这通常意味着您需要从头开始重新运行您的脚本。")


# %%
# --- 验证提取结果 ---
print("\n--- 验证 房价训练集 (train_price) 的物业费提取 ---")
cols_to_show_fee = [input_col_name, output_col_name]

# 验证代码也需要 try-except 来防止 NameError
try:
    # 1. 检查 'nan' (空值) 样本 (应为 0)
    if train_price[input_col_name].isna().any():
        print(f"\n'nan' (空值) 样本 (应填充为 0.0):")
        nan_sample_price = train_price_processed[train_price[input_col_name].isna()]
        print(nan_sample_price[cols_to_show_fee].head())
    
    # 2. 检查 '单一值' 样本
    single_val_sample = train_price_processed[train_price[input_col_name] == '0.65元/月/㎡']
    if not single_val_sample.empty:
        print(f"\n'单一值' (0.65) 样本 (应为 0.65):")
        print(single_val_sample[cols_to_show_fee].head())
        
    # 3. 检查 '范围' 样本 (1.3 + 1.65) / 2 = 1.475
    range_sample = train_price_processed[train_price[input_col_name] == '1.3-1.65元/月/㎡']
    if not range_sample.empty:
        print(f"\n'范围' (1.3-1.65) 样本 (应为 1.475):")
        print(range_sample[cols_to_show_fee].head())
        
except NameError:
    print("...验证跳过，因为 'train_price' 或 'train_price_processed' 未在环境中定义。")

开始提取 物业费 特征...

处理 Price 数据集 '物 业 费'...
已处理 '物 业 费', 新增 '物业费_数值' (dtype: float64)
已处理 '物 业 费', 新增 '物业费_数值' (dtype: float64)

处理 Rent 数据集 '物 业 费'...
已处理 '物 业 费', 新增 '物业费_数值' (dtype: float64)
已处理 '物 业 费', 新增 '物业费_数值' (dtype: float64)

所有 物业费 数据处理完毕！

--- 验证 房价训练集 (train_price) 的物业费提取 ---

'nan' (空值) 样本 (应填充为 0.0):
    物 业 费  物业费_数值
130   NaN     0.0
256   NaN     0.0
388   NaN     0.0
635   NaN     0.0
658   NaN     0.0

'单一值' (0.65) 样本 (应为 0.65):
         物 业 费  物业费_数值
1    0.65元/月/㎡    0.65
242  0.65元/月/㎡    0.65
354  0.65元/月/㎡    0.65
641  0.65元/月/㎡    0.65
896  0.65元/月/㎡    0.65

'范围' (1.3-1.65) 样本 (应为 1.475):
              物 业 费  物业费_数值
0     1.3-1.65元/月/㎡   1.475
778   1.3-1.65元/月/㎡   1.475
1346  1.3-1.65元/月/㎡   1.475
1362  1.3-1.65元/月/㎡   1.475
1419  1.3-1.65元/月/㎡   1.475


In [72]:
train_rent_processed['车位'].unique()

array([nan, '租用车位', '免费使用'], dtype=object)

In [74]:
train_price_processed['停车位'].head()

0     300.0
1    1550.0
2     324.0
3     500.0
4    1800.0
Name: 停车位, dtype: float64

In [76]:
train_rent_processed['停车位'].head()

0    450.0
1    150.0
2    965.0
3    500.0
4    400.0
Name: 停车位, dtype: float64

In [78]:
#停车费用
train_price_processed['停车费用'].unique()

array(['暂无', '150', '1200', '460', '600', '1000', '200',
       '地上150元/月/位，地下2元/时/位，地下固定车位450元/月/位', '170', '450', '650', '140',
       '700', '地上免费  地下400元/月', '210', '300', '500',
       '地上150元，地上机械180，地下300', '无', '400', '750', '1800', '300.00', '550',
       '地上300元/月/位，地下500元/月/位', '350', '80', '地上150元/月/位，地下500元/月/位',
       '地上150元/月/位，地下300元/月/位', '100', '1000～1500', '地上150；地下450', '1500',
       '800', '850', '地上450元/月/位，地450元/月/位0', '160',
       '地上134元/月/位，地下280元/月/位', '1100', '950', '地上120元/月，地下200元/月', '70',
       '150.00', '201', '420-480', '1250', '150.0', '90', '150元/月',
       '地下400，地上免费', '地上150，地下380', '151', '地上150，地下300', '1150', '60',
       '120', '155', '150元/月地上，260元/月地下', '目前免费', '25', '1400', '270',
       '地上200月/元/位，地下300元/月/', '800～1000', '1300', '地下300，地上150',
       '150；350', '固定的80，不固定60', '380', nan, '地上150',
       '地上150元/月/位  地下350元/月/位', '390', '180', '420', '150元/月/位', '230',
       '100月/月/位', '330', '150元', '1600', '地下300元/月/位', '地上150元，升降

In [79]:
#%%
# --------------------------------------------------
# 停车费用 特征提取
# --------------------------------------------------
print("开始提取 停车费用 特征...")

import pandas as pd
import numpy as np

def process_parking_fee(df, input_col, output_col):
    """
    处理 '停车费用' (格式如 '150', '地上150元...', '1000～1500', '暂无', '免费').
    
    1. 提取字符串中出现的第一个数字 (float or int)。
    2. 如果 'nan', '暂无', '免费', '未知' 或任何不含数字的字符串，
       .str.extract 会返回 NaN。
    3. 将所有 NaN 填充为 0。
    
    参数:
    - df: DataFrame
    - input_col: '停车费用'
    - output_col: '停车费用_数值'
    
    返回:
    - 带有新特征列 '停车费用_数值' 的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 转换为字符串，'nan' -> 'nan'
    #    也处理 `120.0` (float) -> '120.0' (str)
    s = df_copy[input_col].astype(str)
    
    # 2. 定义正则表达式
    #    (\d+\.?\d*) : 捕获组 1 - 匹配第一个出现的数字 (float or int)
    pattern = r'(\d+\.?\d*)'
    
    # 3. 提取数字
    extracted = s.str.extract(pattern, expand=False)
    
    # 4. 清理结果
    #    .fillna(0)  : 将所有 NaN (来自 'nan', '暂无', '免费' 等) 替换为 0
    #    .astype(float) : 转换为 float 类型
    df_copy[output_col] = extracted.fillna('0').astype(float)
    
    print(f"已处理 '{input_col}', 新增 '{output_col}' (dtype: {df_copy[output_col].dtype})")
    return df_copy

# %%
# --- 应用处理 ---
input_col_name = '停车费用'
output_col_name = '停车费用_数值'

try:
    # 1. 处理 Price 数据集
    print("\n处理 Price 数据集 '停车费用'...")
    train_price_processed = process_parking_fee(
        train_price_processed, input_col_name, output_col_name
    )
    test_price_processed = process_parking_fee(
        test_price_processed, input_col_name, output_col_name
    )
    
    # 2. 处理 Rent 数据集
    print("\n处理 Rent 数据集 '停车费用'...")
    train_rent_processed = process_parking_fee(
        train_rent_processed, input_col_name, output_col_name
    )
    test_rent_processed = process_parking_fee(
        test_rent_processed, input_col_name, output_col_name
    )

    print("\n所有 停车费用 数据处理完毕！")

except KeyError as e:
    print(f"错误：找不到列 {e}。")
except NameError as e:
    print(f"错误：{e}。请确保 'train_price_processed' 等 DataFrame 已被正确定义。")
    print("这通常意味着您需要从头开始重新运行您的脚本。")


# %%
# --- 验证提取结果 ---
print("\n--- 验证 房价训练集 (train_price) 的停车费用提取 ---")
cols_to_show = [input_col_name, output_col_name]

# 验证代码也需要 try-except 来防止 NameError
try:
    # 1. 检查 '暂无' 样本 (应为 0)
    if (train_price[input_col_name] == '暂无').any():
        print(f"\n'暂无' 样本 (应填充为 0.0):")
        sample = train_price_processed[train_price[input_col_name] == '暂无']
        print(sample[cols_to_show].head())
    
    # 2. 检查 'nan' (空值) 样本 (应为 0)
    if train_price[input_col_name].isna().any():
        print(f"\n'nan' (空值) 样本 (应填充为 0.0):")
        sample = train_price_processed[train_price[input_col_name].isna()]
        print(sample[cols_to_show].head())

    # 3. 检查 '免费' 样本 (应为 0)
    if (train_price[input_col_name] == '免费').any():
        print(f"\n'免费' 样本 (应填充为 0.0):")
        sample = train_price_processed[train_price[input_col_name] == '免费']
        print(sample[cols_to_show].head())

    # 4. 检查 '地上免费 地下400...' 样本 (应为 400)
    if (train_price[input_col_name] == '地上免费  地下400元/月').any():
        print(f"\n'地上免费  地下400元/月' 样本 (应为 400.0):")
        sample = train_price_processed[train_price[input_col_name] == '地上免费  地下400元/月']
        print(sample[cols_to_show].head())
        
    # 5. 检查 '1000～1500' 样本 (应为 1000)
    if (train_price[input_col_name] == '1000～1500').any():
        print(f"\n'1000～1500' 样本 (应为 1000.0):")
        sample = train_price_processed[train_price[input_col_name] == '1000～1500']
        print(sample[cols_to_show].head())

    # 6. 检查 '地上150元/月/位...' 样本 (应为 150)
    val = '地上150元/月/位，地下2元/时/位，地下固定车位450元/月/位'
    if (train_price[input_col_name] == val).any():
        print(f"\n'地上150元/月/位...' 样本 (应为 150.0):")
        sample = train_price_processed[train_price[input_col_name] == val]
        print(sample[cols_to_show].head())

except NameError:
    print("...验证跳过，因为 'train_price' 或 'train_price_processed' 未在环境中定义。")
except KeyError:
    print(f"...验证跳过，因为原始 'train_price' 中缺少 '{input_col_name}'。")

开始提取 停车费用 特征...

处理 Price 数据集 '停车费用'...
已处理 '停车费用', 新增 '停车费用_数值' (dtype: float64)
已处理 '停车费用', 新增 '停车费用_数值' (dtype: float64)

处理 Rent 数据集 '停车费用'...
已处理 '停车费用', 新增 '停车费用_数值' (dtype: float64)
已处理 '停车费用', 新增 '停车费用_数值' (dtype: float64)

所有 停车费用 数据处理完毕！

--- 验证 房价训练集 (train_price) 的停车费用提取 ---

'暂无' 样本 (应填充为 0.0):
   停车费用  停车费用_数值
0    暂无      0.0
3    暂无      0.0
5    暂无      0.0
20   暂无      0.0
37   暂无      0.0

'nan' (空值) 样本 (应填充为 0.0):
     停车费用  停车费用_数值
1524  NaN      0.0
1773  NaN      0.0
1899  NaN      0.0
2035  NaN      0.0
2054  NaN      0.0

'免费' 样本 (应填充为 0.0):
      停车费用  停车费用_数值
88107   免费      0.0
88227   免费      0.0
88659   免费      0.0
89039   免费      0.0
89110   免费      0.0

'地上免费  地下400元/月' 样本 (应为 400.0):
                停车费用  停车费用_数值
32    地上免费  地下400元/月    400.0
364   地上免费  地下400元/月    400.0
728   地上免费  地下400元/月    400.0
1053  地上免费  地下400元/月    400.0
1320  地上免费  地下400元/月    400.0

'1000～1500' 样本 (应为 1000.0):
          停车费用  停车费用_数值
163  1000～1500   1000.0
253  1000～1500   

In [80]:
 # '供水', '供暖', '供电', '燃气费', '供热费'
print(train_price_processed['供水'].unique())
print(train_price_processed['供暖'].unique())
print(train_price_processed['供电'].unique())
print(train_price_processed['燃气费'].unique())
print(train_price_processed['供热费'].unique())


['民水' '商水/民水' '商水' nan]
['集中供暖' '自采暖' '集中供暖/自采暖' nan '自采暖/无供暖' '集中供暖/自采暖/无供暖' '无供暖']
['民电' '商电/民电' '商电' nan]
['2.61元/m³' '2.61-2.63元/m³' nan '2.6-2.61元/m³' '2.61-2.68元/m³' '2.63元/m³'
 '2.61-6元/m³' '2.46元/m³' '2.2-2.46元/m³' '2.95元/m³' '2.47元/m³'
 '2.15-2.48元/m³' '2.14-2.52元/m³' '2.15元/m³' '2.46-2.95元/m³'
 '2.15-2.46元/m³' '2.28-2.49元/m³' '3.45元/m³' '2.24-3.5元/m³' '2.47-2.58元/m³'
 '2.24-2.47元/m³' '2.24元/m³' '2.45元/m³' '2.53元/m³' '2.2-2.47元/m³'
 '2.36元/m³' '2.46-2.48元/m³' '2.37-2.45元/m³' '2.45-2.46元/m³' '2.48元/m³'
 '2-2.24元/m³' '2.24-2.43元/m³' '1.7元/m³' '2.35-2.46元/m³' '2.24-2.65元/m³'
 '2.98元/m³' '3.2元/m³' '2.4-2.47元/m³' '2.64元/m³' '2元/m³' '1.96-1.98元/m³'
 '1.97-2元/m³' '1.98元/m³' '1.97元/m³' '1.96元/m³' '1.96-2元/m³'
 '1.96-1.97元/m³' '1.98-2.8元/m³' '1.98-2元/m³' '1.2-1.98元/m³'
 '1.97-1.98元/m³' '1.72-1.96元/m³' '1.72-1.97元/m³' '1.74-1.98元/m³'
 '1.73元/m³' '2-2.01元/m³' '1.72元/m³' '1.72-1.98元/m³' '1.78-3.45元/m³'
 '1.98-2.02元/m³' '2.9-3.5元/m³' '1.68-5元/m³' '1.9-1.98元/m³' '1.96-2.01元/m³'
 '1.95元/m³' 

In [82]:
#%%
# --------------------------------------------------
# 供水、供暖、供电 特征提取 (多标签哑变量)
# --------------------------------------------------
print("开始提取 供水、供暖、供电 特征...")

try:
    from sklearn.preprocessing import MultiLabelBinarizer
except ImportError:
    print("错误：需要 'sklearn' 库中的 'MultiLabelBinarizer'。")
    # !pip install scikit-learn

# 1. 定义所有唯一的、统一的类别 (基于您提供的 unique() 输出)
#    我们添加 '未知' 来处理 'nan'
ALL_SUPPLY_WATER = sorted(['民水', '商水', '未知'])
ALL_SUPPLY_HEAT = sorted(['集中供暖', '自采暖', '无供暖', '未知'])
ALL_SUPPLY_ELEC = sorted(['民电', '商电', '未知'])

def process_multi_label(df, column_name, prefix, all_categories_list):
    """
    通用函数：处理按 '/' 分割的多标签类别，并创建哑变量。
    'nan' 被处理为 '未知'。
    
    参数:
    - df: DataFrame
    - column_name: 要处理的列名
    - prefix: 新哑变量列的前缀
    - all_categories_list: 包含所有可能类别的列表 (确保列一致)
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 将 'nan' 替换为 '未知'
    s = df_copy[column_name].fillna('未知')
    
    # 2. 按 '/' 分割
    #    '商水/民水' -> ['商水', '民水']
    #    '未知' -> ['未知']
    s_split = s.str.split('/')
    
    # 3. 初始化并应用 MultiLabelBinarizer
    #    我们传入 classes=... 来确保所有 df 都有相同的列
    mlb = MultiLabelBinarizer(classes=all_categories_list)
    dummies_data = mlb.fit_transform(s_split)
    
    # 4. 创建列名，例如 '供水_民水', '供水_商水', '供水_未知'
    dummy_cols = [f'{prefix}_{c}' for c in mlb.classes_]
    
    # 5. 转为 DataFrame
    dummies_df = pd.DataFrame(dummies_data, columns=dummy_cols, index=df_copy.index)
    
    # 6. 合并
    df_copy = pd.concat([df_copy, dummies_df], axis=1)
    
    print(f"已处理 '{column_name}': 新增 {len(dummy_cols)} 个 '{prefix}_' 哑变量。")
    return df_copy

# %%
# --- 应用 供水 ---
print("\n--- 处理 '供水' ---")
train_price_processed = process_multi_label(train_price_processed, '供水', '供水', ALL_SUPPLY_WATER)
test_price_processed = process_multi_label(test_price_processed, '供水', '供水', ALL_SUPPLY_WATER)
train_rent_processed = process_multi_label(train_rent_processed, '供水', '供水', ALL_SUPPLY_WATER)
test_rent_processed = process_multi_label(test_rent_processed, '供水', '供水', ALL_SUPPLY_WATER)

# --- 应用 供暖 ---
print("\n--- 处理 '供暖' ---")
train_price_processed = process_multi_label(train_price_processed, '供暖', '供暖', ALL_SUPPLY_HEAT)
test_price_processed = process_multi_label(test_price_processed, '供暖', '供暖', ALL_SUPPLY_HEAT)
train_rent_processed = process_multi_label(train_rent_processed, '供暖', '供暖', ALL_SUPPLY_HEAT)
test_rent_processed = process_multi_label(test_rent_processed, '供暖', '供暖', ALL_SUPPLY_HEAT)

# --- 应用 供电 ---
print("\n--- 处理 '供电' ---")
train_price_processed = process_multi_label(train_price_processed, '供电', '供电', ALL_SUPPLY_ELEC)
test_price_processed = process_multi_label(test_price_processed, '供电', '供电', ALL_SUPPLY_ELEC)
train_rent_processed = process_multi_label(train_rent_processed, '供电', '供电', ALL_SUPPLY_ELEC)
test_rent_processed = process_multi_label(test_rent_processed, '供电', '供电', ALL_SUPPLY_ELEC)

print("\n所有 供水、供暖、供电 数据处理完毕！")

# %%
# --- 验证 供水、供暖、供电 ---
print("\n--- 验证 房价训练集 (train_price) 的提取结果 ---")

# 1. 验证 '供水' ('商水/民水' 和 'nan')
cols_water = ['供水', '供水_民水', '供水_商水', '供水_未知']
print("\n'供水' 样本 ('商水/民水'):")
print(train_price_processed[train_price['供水'] == '商水/民水'][cols_water].head())
print("\n'供水' 样本 ('nan'):")
print(train_price_processed[train_price['供水'].isna()][cols_water].head())

# 2. 验证 '供暖' ('自采暖/无供暖' 和 'nan')
cols_heat = ['供暖', '供暖_集中供暖', '供暖_自采暖', '供暖_无供暖', '供暖_未知']
print("\n'供暖' 样本 ('自采暖/无供暖'):")
print(train_price_processed[train_price['供暖'] == '自采暖/无供暖'][cols_heat].head())
print("\n'供暖' 样本 ('nan'):")
print(train_price_processed[train_price['供暖'].isna()][cols_to_show].head())


#%%
# --------------------------------------------------
# 燃气费、供热费 特征提取 (数值型)
# --------------------------------------------------
print("\n开始提取 燃气费 和 供热费 特征...")

# 我们将重用 'process_property_fee' 函数，因为它实现了完全相同的逻辑
# (提取1或2个数字，如果是2个则取平均，nan 转为 0)

try:
    # 1. 处理 '燃气费'
    print("\n处理 '燃气费'...")
    train_price_processed = process_property_fee(train_price_processed, '燃气费', '燃气费_数值')
    test_price_processed = process_property_fee(test_price_processed, '燃气费', '燃气费_数值')
    train_rent_processed = process_property_fee(train_rent_processed, '燃气费', '燃气费_数值')
    test_rent_processed = process_property_fee(test_rent_processed, '燃气费', '燃气费_数值')

    # 2. 处理 '供热费'
    print("\n处理 '供热费'...")
    train_price_processed = process_property_fee(train_price_processed, '供热费', '供热费_数值')
    test_price_processed = process_property_fee(test_price_processed, '供热费', '供热费_数值')
    train_rent_processed = process_property_fee(train_rent_processed, '供热费', '供热费_数值')
    test_rent_processed = process_property_fee(test_rent_processed, '供热费', '供热费_数值')

    print("\n所有 燃气费 和 供热费 数据处理完毕！")

except NameError:
    print("错误：'process_property_fee' 函数未定义。")
    print("请确保您已经运行了之前定义 'process_property_fee' (用于处理 '物 业 费') 的代码块。")
except KeyError as e:
    print(f"错误：找不到列 {e}。")


# %%
# --- 验证 燃气费、供热费 ---
print("\n--- 验证 房价训练集 (train_price) 的 燃气费 提取 ---")
cols_gas = ['燃气费', '燃气费_数值']

# 1. 验证 'nan' (空值) 样本 (应为 0)
print("\n'燃气费' 样本 ('nan'):")
print(train_price_processed[train_price['燃气费'].isna()][cols_gas].head())

# 2. 验证 '范围' 样本 (2.61 + 2.63) / 2 = 2.62
print("\n'燃气费' 样本 ('2.61-2.63元/m³'):")
print(train_price_processed[train_price['燃气费'] == '2.61-2.63元/m³'][cols_gas].head())

# 3. 验证 '单一值' 样本
print("\n'燃气费' 样本 ('2.61元/m³'):")
print(train_price_processed[train_price['燃气费'] == '2.61元/m³'][cols_gas].head())

开始提取 供水、供暖、供电 特征...

--- 处理 '供水' ---
已处理 '供水': 新增 3 个 '供水_' 哑变量。
已处理 '供水': 新增 3 个 '供水_' 哑变量。
已处理 '供水': 新增 3 个 '供水_' 哑变量。
已处理 '供水': 新增 3 个 '供水_' 哑变量。

--- 处理 '供暖' ---
已处理 '供暖': 新增 4 个 '供暖_' 哑变量。
已处理 '供暖': 新增 4 个 '供暖_' 哑变量。
已处理 '供暖': 新增 4 个 '供暖_' 哑变量。
已处理 '供暖': 新增 4 个 '供暖_' 哑变量。

--- 处理 '供电' ---
已处理 '供电': 新增 3 个 '供电_' 哑变量。
已处理 '供电': 新增 3 个 '供电_' 哑变量。
已处理 '供电': 新增 3 个 '供电_' 哑变量。
已处理 '供电': 新增 3 个 '供电_' 哑变量。

所有 供水、供暖、供电 数据处理完毕！

--- 验证 房价训练集 (train_price) 的提取结果 ---

'供水' 样本 ('商水/民水'):
      供水  供水_民水  供水_商水  供水_未知
1  商水/民水      1      1      0
2  商水/民水      1      1      0
4  商水/民水      1      1      0
6  商水/民水      1      1      0
7  商水/民水      1      1      0

'供水' 样本 ('nan'):
      供水  供水_民水  供水_商水  供水_未知
287  NaN      0      0      1
372  NaN      0      0      1
420  NaN      0      0      1
424  NaN      0      0      1
506  NaN      0      0      1

'供暖' 样本 ('自采暖/无供暖'):
            供暖  供暖_集中供暖  供暖_自采暖  供暖_无供暖  供暖_未知
69417  自采暖/无供暖        0       1       1      0
69429  自采暖/无供暖      

In [81]:
print(train_rent_processed['供水'].unique())
print(train_rent_processed['供暖'].unique())
print(train_rent_processed['供电'].unique())
print(train_rent_processed['燃气费'].unique())
print(train_rent_processed['供热费'].unique())


['民水' '商水/民水' '商水' nan]
['集中供暖' '集中供暖/自采暖' '自采暖' nan '集中供暖/自采暖/无供暖' '自采暖/无供暖' '无供暖']
['民电' '商电/民电' '商电' nan]
['2.61元/m³' '2.61-2.63元/m³' '2.6-2.61元/m³' '2.61-6元/m³' nan '2.63元/m³'
 '2.95元/m³' '2.15-2.46元/m³' '2.46元/m³' '2.45-2.46元/m³' '2.47元/m³'
 '2.24元/m³' '2.46-2.95元/m³' '2.14-2.52元/m³' '2.28-2.49元/m³' '2.2-2.46元/m³'
 '2.15元/m³' '2.24-3.5元/m³' '2.15-2.48元/m³' '2.24-2.47元/m³' '2.48元/m³'
 '2.45元/m³' '2.36元/m³' '2.24-2.65元/m³' '2.2-2.47元/m³' '2-2.24元/m³'
 '2.53元/m³' '1.7元/m³' '3.2元/m³' '2.47-2.58元/m³' '2.37-2.45元/m³'
 '2.4-2.47元/m³' '3.45元/m³' '2.24-2.43元/m³' '2.64元/m³' '2.46-2.48元/m³'
 '2.98元/m³' '1.96-1.98元/m³' '1.97元/m³' '1.96元/m³' '1.98元/m³'
 '1.97-1.98元/m³' '1.96-2元/m³' '2元/m³' '1.98-2.8元/m³' '2-2.01元/m³'
 '1.98-2元/m³' '1.74-1.98元/m³' '1.97-2元/m³' '1.96-1.97元/m³' '1.72-1.97元/m³'
 '1.73元/m³' '1.72-1.98元/m³' '1.72-1.96元/m³' '1.9-3.5元/m³' '1.95-1.98元/m³'
 '1.2-1.98元/m³' '3.7元/m³' '1.97-2.32元/m³' '1.89元/m³' '1.98-1.99元/m³'
 '1.11元/m³' '1.72元/m³' '1元/m³' '1.74元/m³' '1.99元/m³' '1.9-1.98元

In [83]:
train_price_processed['产权描述'].unique()

array(['商品房/已购公房/央产房/私产', '商品房/一类经济适用房/私产', '商品房/限价商品房', '商品房',
       '商品房/已购公房/私产', '商品房/私产', '商品房/使用权', '商品房/一类经济适用房/二类经济适用房',
       '商品房/已购公房/使用权', '商品房/已购公房/二类经济适用房/私产', '商品房/已购公房/央产房',
       '商品房/已购公房/央产房/一类经济适用房/二类经济适用房/私产', '商品房/已购公房/一类经济适用房/二类经济适用房/私产',
       '商品房/二类经济适用房/定向安置房', '商品房/限价商品房/自住型商品房/定向安置房', '商品房/已购公房',
       '商品房/已购公房/一类经济适用房', '商品房/自住型商品房', '商品房/使用权/私产/校产',
       '商品房/已购公房/央产房/二类经济适用房/使用权/私产', '商品房/使用权/私产',
       '商品房/一类经济适用房/二类经济适用房/私产', '商品房/已购公房/央产房/二类经济适用房/私产',
       '商品房/一类经济适用房/二类经济适用房/定向安置房',
       '商品房/已购公房/央产房/一类经济适用房/二类经济适用房/私产/军产/校产', '商品房/已购公房/二类经济适用房',
       '商品房/二类经济适用房', '商品房/已购公房/央产房/二类经济适用房', '商品房/已购公房/使用权/私产',
       '商品房/二类经济适用房/乡产/私产/定向安置房', '商品房/已购公房/央产房/二类经济适用房/私产/定向安置房',
       '商品房/已购公房/央产房/使用权/私产/军产', '商品房/已购公房/央产房/二类经济适用房/使用权', '商品房/乡产/私产',
       '商品房/一类经济适用房', '商品房/公租房',
       '商品房/已购公房/央产房/一类经济适用房/二类经济适用房/使用权/乡产/定向安置房', '商品房/使用权/私产/公租房',
       '商品房/自住型商品房/公租房', '商品房/已购公房/二类经济适用房/私产/军产', '商品房/定向安置房',
       '商品房/已购公房/私产/军产

In [84]:
train_rent_processed['产权描述'].unique()

array(['商品房/已购公房/央产房/二类经济适用房/私产', '商品房/已购公房/使用权/私产', '商品房/私产',
       '商品房/自住型商品房', '商品房/已购公房/一类经济适用房/二类经济适用房/私产', '商品房', '商品房/已购公房/央产房',
       '商品房/使用权/私产', '商品房/已购公房', '商品房/已购公房/央产房/一类经济适用房/二类经济适用房/私产',
       '商品房/已购公房/央产房/二类经济适用房/私产/军产', '商品房/已购公房/央产房/二类经济适用房/使用权/私产',
       '商品房/已购公房/央产房/使用权', '商品房/已购公房/央产房/私产', '商品房/自住型商品房/公租房',
       '商品房/已购公房/私产', '商品房/一类经济适用房/二类经济适用房/私产', '商品房/已购公房/使用权/私产/军产',
       '商品房/已购公房/一类经济适用房/二类经济适用房/私产/定向安置房', '商品房/已购公房/二类经济适用房/私产',
       '商品房/已购公房/使用权/私产/定向安置房', nan, '商品房/一类经济适用房/二类经济适用房',
       '商品房/央产房/使用权/私产', '商品房/已购公房/二类经济适用房', '商品房/限价商品房',
       '商品房/自住型商品房/定向安置房', '商品房/已购公房/二类经济适用房/使用权/私产',
       '商品房/已购公房/央产房/二类经济适用房/使用权',
       '商品房/已购公房/央产房/一类经济适用房/二类经济适用房/使用权/乡产/定向安置房', '商品房/一类经济适用房/私产',
       '商品房/已购公房/央产房/二类经济适用房/私产/定向安置房',
       '商品房/已购公房/央产房/一类经济适用房/二类经济适用房/私产/军产/校产', '商品房/限价商品房/自住型商品房',
       '商品房/一类经济适用房', '商品房/二类经济适用房', '商品房/二类经济适用房/乡产/私产/定向安置房',
       '商品房/已购公房/央产房/使用权/私产/军产', '商品房/已购公房/私产/公租房', '商品房/二类经济适用房/定向安置房',
   

In [85]:
#%%
# --------------------------------------------------
# 产权描述 特征提取 (标准化与多标签哑变量)
# --------------------------------------------------
print("开始提取 产权描述 特征...")

from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import numpy as np

# 1. 定义一个辅助函数来标准化类别
def normalize_rights(s_col):
    """
    辅助函数：标准化产权描述的 Series。
    1. 'nan' -> '未知'
    2. '一类/二类经济适用房' -> '经济适用房'
    3. '拆迁还建房'/'回迁房' -> '动迁安置房'
    4. '售后公房' -> '已购公房'
    5. 按 '/' 分割
    6. 移除每个列表中的重复项
    
    返回: 一个包含干净类别列表的 Series
    """
    # 1. Handle nan
    s = s_col.fillna('未知')
    
    # 2. 定义别名替换
    replacements = {
        '一类经济适用房': '经济适用房',
        '二类经济适用房': '经济适用房',
        '拆迁还建房': '动迁安置房',
        '回迁房': '动迁安置房',
        '售后公房': '已购公房'
    }
    
    # 3. 应用替换
    for old, new in replacements.items():
        s = s.str.replace(old, new, regex=False)
        
    # 4. 按 '/' 分割
    s_split = s.str.split('/')
    
    # 5. 应用 set() 到每个列表以移除重复项
    #    例如 '经济适用房/经济适用房' -> ['经济适用房']
    return s_split.apply(lambda x: list(set(x)))

# %%
# --- 关键步骤：在所有数据集中找到所有唯一的、标准化的类别 ---

try:
    print("正在分析所有数据集中的产权类别...")
    # 1. 标准化所有四个数据集中的列
    norm_price_train = normalize_rights(train_price['产权描述'])
    norm_price_test = normalize_rights(test_price['产权描述'])
    norm_rent_train = normalize_rights(train_rent['产权描述'])
    norm_rent_test = normalize_rights(test_rent['产权描述'])

    # 2. 将所有列表合并到一个 Series 中
    all_lists = pd.concat([norm_price_train, norm_price_test, norm_rent_train, norm_rent_test])

    # 3. 遍历所有列表，找到所有唯一的类别
    all_categories = set()
    all_lists.apply(all_categories.update)

    # 4. 移除空字符串并排序，创建最终的类别列表
    all_categories.discard('')
    ALL_PROPERTY_RIGHTS = sorted(list(all_categories))

    print(f"已找到 {len(ALL_PROPERTY_RIGHTS)} 个统一的产权类别。")
    # print(ALL_PROPERTY_RIGHTS) # (取消注释以查看所有类别)

except NameError as e:
    print(f"错误：{e}。 'train_price' 等原始 DataFrame 未定义。")
    print("请确保从头运行脚本来加载数据。")

# %%
# --- 定义主处理函数 ---

def process_property_rights(df, column_name, all_types_list):
    """
    使用 MultiLabelBinarizer 处理标准化的产权描述列表。
    
    参数:
    - df: DataFrame
    - column_name: '产权描述'
    - all_types_list: 预先计算好的 ALL_PROPERTY_RIGHTS 列表
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 标准化当前 DataFrame 的列 (返回一个列表的 Series)
    normalized_lists = normalize_rights(df_copy[column_name])
    
    # 2. 初始化 Binarizer，强制使用所有类别
    mlb = MultiLabelBinarizer(classes=all_types_list)
    
    # 3. 转换数据
    dummies_data = mlb.fit_transform(normalized_lists)
    
    # 4. 创建列名 (例如 '产权_商品房')
    dummy_cols = [f'产权_{c}' for c in mlb.classes_]
    
    # 5. 转为 DataFrame
    dummies_df = pd.DataFrame(dummies_data, columns=dummy_cols, index=df_copy.index)
    
    # 6. 合并
    df_copy = pd.concat([df_copy, dummies_df], axis=1)
    
    print(f"已处理 '{column_name}': 新增 {len(dummy_cols)} 个 '产权_' 哑变量。")
    return df_copy

# %%
# --- 应用处理 ---

try:
    print("\n处理房价数据 (price) 的产权描述...")
    train_price_processed = process_property_rights(train_price_processed, '产权描述', ALL_PROPERTY_RIGHTS)
    test_price_processed = process_property_rights(test_price_processed, '产权描述', ALL_PROPERTY_RIGHTS)

    print("\n处理租金数据 (rent) 的产权描述...")
    train_rent_processed = process_property_rights(train_rent_processed, '产权描述', ALL_PROPERTY_RIGHTS)
    test_rent_processed = process_property_rights(test_rent_processed, '产权描述', ALL_PROPERTY_RIGHTS)

    print("\n所有 产权描述 数据处理完毕！")

except NameError as e:
    print(f"错误：{e}。 'train_price_processed' 或 'ALL_PROPERTY_RIGHTS' 未定义。")
    print("请确保从头运行脚本来加载数据和定义类别。")
except KeyError as e:
    print(f"错误：找不到列 {e}。")


# %%
# --- 验证提取结果 ---
print("\n--- 验证 房价训练集 (train_price) 的产权描述提取 ---")

try:
    # 1. 验证一个复杂的 '经济适用房' 样本
    sample_text = '商品房/一类经济适用房/二类经济适用房/私产'
    complex_sample = train_price_processed[train_price['产权描述'] == sample_text]
    
    # 定义要检查的列
    cols_to_check_econ = ['产权_商品房', '产权_经济适用房', '产权_私产']
    # 检查标准化前的列是否*不*存在 (如果它们不在 ALL_PROPERTY_RIGHTS 中)
    if '产权_一类经济适用房' not in ALL_PROPERTY_RIGHTS:
        print("✓ 验证通过：'产权_一类经济适用房' 已被正确标准化。")
    
    if not complex_sample.empty:
        print(f"\n复杂样本 ('{sample_text}'):")
        print(complex_sample[cols_to_check_econ].head())

    # 2. 验证 'nan' (空值) 样本 (应标记 '产权_未知')
    nan_sample_price = train_price_processed[train_price['产权描述'].isna()]
    if not nan_sample_price.empty:
        print("\n'nan' (空值) 样本 (应标记 '产权_未知'):")
        print(nan_sample_price[['产权_未知']].head())

    # 3. 验证 '售后公房' -> '已购公房'
    sample_text_shou = '商品房/售后公房'
    shou_sample = train_price_processed[train_price['产权描述'] == sample_text_shou]
    cols_to_check_shou = ['产权_商品房', '产权_已购公房']
    if '产权_售后公房' not in ALL_PROPERTY_RIGHTS:
        print("✓ 验证通过：'产权_售后公房' 已被正确标准化。")

    if not shou_sample.empty:
        print(f"\n'售后公房' 样本 ('{sample_text_shou}'):")
        print(shou_sample[cols_to_check_shou].head())

except NameError:
    print("...验证跳过，因为 'train_price' 或 'train_price_processed' 未在环境中定义。")

开始提取 产权描述 特征...
正在分析所有数据集中的产权类别...
已找到 21 个统一的产权类别。

处理房价数据 (price) 的产权描述...
已处理 '产权描述': 新增 21 个 '产权_' 哑变量。
已处理 '产权描述': 新增 21 个 '产权_' 哑变量。

处理租金数据 (rent) 的产权描述...
已处理 '产权描述': 新增 21 个 '产权_' 哑变量。
已处理 '产权描述': 新增 21 个 '产权_' 哑变量。

所有 产权描述 数据处理完毕！

--- 验证 房价训练集 (train_price) 的产权描述提取 ---
✓ 验证通过：'产权_一类经济适用房' 已被正确标准化。

复杂样本 ('商品房/一类经济适用房/二类经济适用房/私产'):
     产权_商品房  产权_经济适用房  产权_私产
93        1         1      1
141       1         1      1
205       1         1      1
340       1         1      1
411       1         1      1

'nan' (空值) 样本 (应标记 '产权_未知'):
      产权_未知
1524      1
1773      1
1899      1
2035      1
2054      1
✓ 验证通过：'产权_售后公房' 已被正确标准化。

'售后公房' 样本 ('商品房/售后公房'):
       产权_商品房  产权_已购公房
69443       1        1
69787       1        1
69987       1        1
70058       1        1
70200       1        1


In [90]:
train_price_processed['区县'].fillna(0, inplace=True)
train_rent_processed['区县'].fillna(0, inplace=True)
test_price_processed['区县'].fillna(0, inplace=True)
test_rent_processed['区县'].fillna(0, inplace=True)


In [ ]:
train_price_processed['板块_comm'].fillna(0, inplace=True)
train_rent_processed['板块'].fillna(0, inplace=True)
test_price_processed['板块_comm'].fillna(0, inplace=True)
test_rent_processed['板块'].fillna(0, inplace=True)


In [94]:
train_price_processed['梯户比例'].unique()

array(['一梯三户', '一梯两户', '一梯五户', nan, '两梯十一户', '一梯六户', '两梯四户', '三梯八户',
       '一梯一户', '两梯两户', '两梯八户', '两梯二十五户', '三梯十三户', '两梯二十户', '两梯六户',
       '五梯二十四户', '六梯十户', '三梯十二户', '两梯二十四户', '一梯十二户', '四梯十二户', '两梯十五户',
       '八梯二十九户', '一梯四户', '一梯九户', '三梯二十四户', '四梯九户', '四梯二十七户', '五梯九户',
       '四梯八户', '三梯十一户', '五梯十五户', '五梯二十户', '四梯七户', '三梯九户', '两梯十户',
       '八梯三十八户', '两梯二十一户', '六梯十八户', '四梯十六户', '两梯七户', '五梯六十户', '两梯二十七户',
       '两梯五户', '三梯六十五户', '三梯二十一户', '四梯五十五户', '四梯十户', '四梯十一户', '四梯三十九户',
       '两梯十九户', '两梯十四户', '四梯五十二户', '三梯二十五户', '四梯十四户', '三梯三十七户', '三梯三十九户',
       '四梯一户', '三梯十户', '三梯二十户', '两梯三户', '两梯十二户', '四梯三十三户', '一梯八户', '一梯十户',
       '三梯二十二户', '三梯六户', '六梯八户', '两梯二十八户', '三梯三十户', '两梯十三户', '四梯六户',
       '三梯二十七户', '两梯九户', '三梯两户', '五梯八户', '三梯十五户', '一梯十四户', '四梯五户', '四梯两户',
       '八梯六十二户', '一梯七户', '三梯三十二户', '四梯三十一户', '三梯三十三户', '两梯一户', '两梯二十三户',
       '三梯二十八户', '四梯三十五户', '三梯七户', '一梯十一户', '八梯三十七户', '三梯十七户', '三梯十九户',
       '四梯三十四户', '三梯三户', '五梯五十户', '一梯十五户', '三梯三十六户', '四梯五十四户', '两梯四十户',
    

In [95]:
#%%
# --------------------------------------------------
# 梯户比例 特征提取 (仅 Price)
# --------------------------------------------------
import re
import numpy as np
import pandas as pd
print("开始提取 梯户比例 特征...")

def chinese_to_int(s):
    """
    将中文数字字符串 (如 '一', '十一', '二十', '一百零三') 转换为整数。
    """
    if not isinstance(s, str): return 0
    
    digit_map = {'一': 1, '二': 2, '两': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9, '十': 10, '零': 0}
    
    s = s.strip()
    num = 0
    
    # Handle '百' (e.g., '一百零三', '一百九十一')
    if '百' in s:
        parts = s.split('百')
        # '一百...'
        if parts[0] in digit_map:
            num += digit_map.get(parts[0], 0) * 100
        # '百...' (Not in data, but safe)
        elif parts[0] == '': 
            num += 100
        s = parts[1] # '零三' or '九十一' or ''
    
    # Handle '十' (e.g., '十一', '二十', '二十五', '九十一')
    if '十' in s:
        parts = s.split('十')
        # '十一'
        if parts[0] == '': 
            num += 10
        # '二十', '九十'
        else: 
            num += digit_map.get(parts[0], 0) * 10
        s = parts[1] # '一' or '' or '五'
    
    # Handle ones digit (e.g., '一', '三' from '零三')
    s = s.replace('零', '')
    if s:
        num += digit_map.get(s, 0)
        
    return num

def process_elevator_ratio(df, column_name):
    """
    处理 '梯户比例' (格式如 '一梯三户', '两梯十一户', '五梯一百零三户').
    
    1. 提取 'X' (梯) 和 'Y' (户)
    2. 将中文数字 X, Y 转换为整数
    3. 计算 '梯户比例_数值' = 户 / 梯
    4. 'nan', '未知', 0梯 都将导致 0.0
    """
    df_copy = df.copy()
    
    # 1. 转换为字符串
    s = df_copy[column_name].astype(str)
    
    # 2. 定义一个 apply 函数
    def calculate_ratio(s_val):
        if not isinstance(s_val, str):
            return 0.0 # Handle nan
        
        # 使用正则表达式匹配
        match = re.search(r'(.+)梯(.+)户', s_val)
        
        if not match:
            return 0.0 # Handle 'nan' string etc.
            
        elevators_str = match.group(1)
        households_str = match.group(2)
        
        # 转换
        elevators = chinese_to_int(elevators_str)
        households = chinese_to_int(households_str)
        
        # 计算比例：户数 / 梯数
        if elevators == 0:
            return 0.0 # Avoid division by zero
        else:
            return households / elevators
    
    # 3. 应用函数
    df_copy['梯户比例_数值'] = s.apply(calculate_ratio)
    
    print(f"已处理 '{column_name}', 新增 '梯户比例_数值' (dtype: {df_copy['梯户比例_数值'].dtype})")
    return df_copy

# %%
# --- 应用处理 ---
# 根据您的请求，我们只处理 Price 数据集
input_col_name = '梯户比例'

try:
    # 1. 处理 Price 数据集
    print("\n处理 Price 数据集 '梯户比例'...")
    train_price_processed = process_elevator_ratio(
        train_price_processed, input_col_name
    )
    test_price_processed = process_elevator_ratio(
        test_price_processed, input_col_name
    )
    
    print("\nPrice 数据集 '梯户比例' 处理完毕。")
    
    # 2. 检查 Rent 数据集
    if 'train_rent_processed' in locals() and input_col_name in train_rent_processed.columns:
        print(f"提示: '{input_col_name}' 列也存在于 Rent 数据集中，但本次未按要求处理。")
    
except KeyError as e:
    print(f"错误：找不到列 {e}。")
except NameError as e:
    print(f"错误：{e}。请确保 'train_price_processed' 等 DataFrame 已被正确定义。")
    print("这通常意味着您需要从头开始重新运行您的脚本。")

# %%
# --- 验证提取结果 ---
print("\n--- 验证 房价训练集 (train_price) 的梯户比例提取 ---")
cols_to_show = [input_col_name, '梯户比例_数值']

# 验证代码也需要 try-except 来防止 NameError
try:
    # 1. 检查 'nan' (空值) 样本 (应为 0)
    if train_price[input_col_name].isna().any():
        print(f"\n'nan' (空值) 样本 (应填充为 0.0):")
        sample = train_price_processed[train_price[input_col_name].isna()]
        print(sample[cols_to_show].head())
    
    # 2. 检查 '一梯两户' 样本 (应为 2 / 1 = 2.0)
    sample_1_2 = train_price_processed[train_price[input_col_name] == '一梯两户']
    if not sample_1_2.empty:
        print(f"\n'一梯两户' 样本 (应为 2.0):")
        print(sample_1_2[cols_to_show].head())
        
    # 3. 检查 '两梯十一户' 样本 (应为 11 / 2 = 5.5)
    sample_2_11 = train_price_processed[train_price[input_col_name] == '两梯十一户']
    if not sample_2_11.empty:
        print(f"\n'两梯十一户' 样本 (应为 5.5):")
        print(sample_2_11[cols_to_show].head())

    # 4. 检查 '五梯一百零三户' 样本 (应为 103 / 5 = 20.6)
    sample_5_103 = train_price_processed[train_price[input_col_name] == '五梯一百零三户']
    if not sample_5_103.empty:
        print(f"\n'五梯一百零三户' 样本 (应为 20.6):")
        print(sample_5_103[cols_to_show].head())
        
    # 5. 检查 '一梯一户' 样本 (应为 1 / 1 = 1.0)
    sample_1_1 = train_price_processed[train_price[input_col_name] == '一梯一户']
    if not sample_1_1.empty:
        print(f"\n'一梯一户' 样本 (应为 1.0):")
        print(sample_1_1[cols_to_show].head())

except NameError:
    print("...验证跳过，因为 'train_price' 或 'train_price_processed' 未在环境中定义。")
except KeyError:
    print(f"...验证跳过，因为原始 'train_price' 中缺少 '{input_col_name}'。")

开始提取 梯户比例 特征...

处理 Price 数据集 '梯户比例'...
已处理 '梯户比例', 新增 '梯户比例_数值' (dtype: float64)
已处理 '梯户比例', 新增 '梯户比例_数值' (dtype: float64)

Price 数据集 '梯户比例' 处理完毕。

--- 验证 房价训练集 (train_price) 的梯户比例提取 ---

'nan' (空值) 样本 (应填充为 0.0):
   梯户比例  梯户比例_数值
3   NaN      0.0
7   NaN      0.0
71  NaN      0.0
84  NaN      0.0
89  NaN      0.0

'一梯两户' 样本 (应为 2.0):
    梯户比例  梯户比例_数值
1   一梯两户      2.0
10  一梯两户      2.0
11  一梯两户      2.0
13  一梯两户      2.0
15  一梯两户      2.0

'两梯十一户' 样本 (应为 5.5):
      梯户比例  梯户比例_数值
4    两梯十一户      5.5
154  两梯十一户      5.5
210  两梯十一户      5.5
325  两梯十一户      5.5
966  两梯十一户      5.5

'五梯一百零三户' 样本 (应为 20.6):
          梯户比例  梯户比例_数值
31608  五梯一百零三户     20.6
43575  五梯一百零三户     20.6
43578  五梯一百零三户     20.6
43580  五梯一百零三户     20.6
44993  五梯一百零三户     20.6

'一梯一户' 样本 (应为 1.0):
    梯户比例  梯户比例_数值
9   一梯一户      1.0
37  一梯一户      1.0
49  一梯一户      1.0
81  一梯一户      1.0
95  一梯一户      1.0


In [96]:
train_price_processed['别墅类型'].unique()

array([nan, '独栋', '联排', '叠拼', '双拼'], dtype=object)

In [97]:
#%%
# --------------------------------------------------
# 别墅类型 特征提取 (仅 Price)
# --------------------------------------------------
print("开始提取 别墅类型 特征...")

import pandas as pd
import numpy as np

def process_villa_type(df, column_name):
    """
    处理 '别墅类型' (格式如 '独栋', '联排', nan)。
    
    1. 将 'nan' 替换为 '非别墅'。
    2. 使用 pd.get_dummies 创建哑变量，前缀为 '别墅'。
    
    参数:
    - df: DataFrame
    - column_name: '别墅类型'
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 将 'nan' 替换为 '非别墅'
    #    这样 get_dummies 就会自动创建一个 '别墅_非别墅' 列
    s = df_copy[column_name].fillna('非别墅')
    
    # 2. 创建哑变量
    #    将创建 '别墅_独栋', '别墅_联排', '别墅_非别墅' 等
    dummies = pd.get_dummies(s, prefix='别墅')
    
    # 3. 合并
    df_copy = pd.concat([df_copy, dummies], axis=1)
    
    print(f"已处理 '{column_name}': 新增 {len(dummies.columns)} 个 '别墅_' 哑变量。")
    return df_copy

# %%
# --- 应用处理 ---
# 根据您的请求，我们只处理 Price 数据集
input_col_name = '别墅类型'

try:
    # 1. 处理 Price 数据集
    print("\n处理 Price 数据集 '别墅类型'...")
    train_price_processed = process_villa_type(
        train_price_processed, input_col_name
    )
    test_price_processed = process_villa_type(
        test_price_processed, input_col_name
    )
    
    print("\nPrice 数据集 '别墅类型' 处理完毕。")
    
except KeyError as e:
    print(f"错误：找不到列 {e}。")
except NameError as e:
    print(f"错误：{e}。请确保 'train_price_processed' 等 DataFrame 已被正确定义。")
    print("这通常意味着您需要从头开始重新运行您的脚本。")

# %%
# --- 验证提取结果 ---
print("\n--- 验证 房价训练集 (train_price) 的别墅类型提取 ---")

# 定义所有预期的新列
cols_to_show = ['别墅类型', '别墅_独栋', '别墅_联排', '别墅_叠拼', '别墅_双拼', '别墅_非别墅']
# 筛选出实际存在的列进行显示
cols_to_show_existing = [col for col in cols_to_show if col in train_price_processed.columns]


# 验证代码也需要 try-except 来防止 NameError
try:
    # 1. 检查 'nan' (空值) 样本 (应标记 '别墅_非别墅')
    if train_price[input_col_name].isna().any():
        print(f"\n'nan' (空值) 样本 (应标记 '别墅_非别墅'):")
        sample = train_price_processed[train_price[input_col_name].isna()]
        print(sample[cols_to_show_existing].head())
    
    # 2. 检查 '独栋' 样本 (应标记 '别墅_独栋')
    sample_du = train_price_processed[train_price[input_col_name] == '独栋']
    if not sample_du.empty:
        print(f"\n'独栋' 样本 (应标记 '别墅_独栋'):")
        print(sample_du[cols_to_show_existing].head())
        
except NameError:
    print("...验证跳过，因为 'train_price' 或 'train_price_processed' 未在环境中定义。")
except KeyError:
    print(f"...验证跳过，因为原始 'train_price' 中缺少 '{input_col_name}'。")

print(f"\n创建的列: {[col for col in train_price_processed.columns if col.startswith('别墅_')]}")

开始提取 别墅类型 特征...

处理 Price 数据集 '别墅类型'...
已处理 '别墅类型': 新增 5 个 '别墅_' 哑变量。
已处理 '别墅类型': 新增 5 个 '别墅_' 哑变量。

Price 数据集 '别墅类型' 处理完毕。

--- 验证 房价训练集 (train_price) 的别墅类型提取 ---

'nan' (空值) 样本 (应标记 '别墅_非别墅'):
  别墅类型  别墅_独栋  别墅_联排  别墅_叠拼  别墅_双拼  别墅_非别墅
0  NaN  False  False  False  False    True
1  NaN  False  False  False  False    True
2  NaN  False  False  False  False    True
4  NaN  False  False  False  False    True
5  NaN  False  False  False  False    True

'独栋' 样本 (应标记 '别墅_独栋'):
     别墅类型  别墅_独栋  别墅_联排  别墅_叠拼  别墅_双拼  别墅_非别墅
3      独栋   True  False  False  False   False
714    独栋   True  False  False  False   False
761    独栋   True  False  False  False   False
763    独栋   True  False  False  False   False
1667   独栋   True  False  False  False   False

创建的列: ['别墅_双拼', '别墅_叠拼', '别墅_独栋', '别墅_联排', '别墅_非别墅']


In [98]:
train_price_processed.columns

Index(['区域', '板块', '环线', 'Price', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向',
       '建筑结构',
       ...
       '产权_经济适用房', '产权_自住型商品房', '产权_限价商品房', '产权_集资房', '梯户比例_数值', '别墅_双拼',
       '别墅_叠拼', '别墅_独栋', '别墅_联排', '别墅_非别墅'],
      dtype='object', length=189)

In [99]:
train_price_processed['交易权属'].unique()

array(['商品房', '已购公房', '限价商品房', '一类经济适用房', '央产房', '私产', '二类经济适用房', '定向安置房',
       '房改房', '经济适用房', '拆迁还建房', '集资房', '动迁安置房', '售后公房', '回迁房'],
      dtype=object)

In [102]:
train_price_processed['上次交易'].head()

0    2013-07-31
1    2010-12-10
2    2013-10-28
3    2017-04-12
4    2017-02-28
Name: 上次交易, dtype: object

In [103]:
train_price_processed['房屋用途'].unique()

array(['普通住宅', '别墅', '商业办公类', '车库', '公寓/住宅', '酒店式公寓', '公寓', '公寓/公寓',
       '四合院', '商务型公寓', '公寓（住宅）', '住宅式公寓', '商住两用', '新式里弄', '老公寓', '花园洋房',
       '底商', '商业', '商务公寓', '写字楼', nan], dtype=object)

In [106]:
train_price_processed['房屋年限'].unique()

array(['满五年', '满两年', '未满两年', nan], dtype=object)

In [107]:
#%%
# --------------------------------------------------
# 房屋年限 特征提取 (仅 Price, 带层级逻辑)
# --------------------------------------------------
print("开始提取 房屋年限 特征...")

import pandas as pd
import numpy as np

def process_deed_limit(df, column_name):
    """
    处理 '房屋年限' (格式如 '满五年', '满两年', nan)。
    
    1. '满五年' 逻辑上包含 '满两年'。
    2. 'nan' 被视为 '未知'。
    
    创建以下列:
    - '年限_满两年': 1 if '满两年' OR '满五年', 0 otherwise
    - '年限_满五年': 1 if '满五年', 0 otherwise
    - '年限_未满两年': 1 if '未满两年', 0 otherwise
    - '年限_未知': 1 if nan, 0 otherwise
    
    参数:
    - df: DataFrame
    - column_name: '房屋年限'
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 获取原始 Series
    s = df_copy[column_name]
    
    # 2. 创建 '年限_满两年'
    #    使用 .isin() 检查是否为 '满两年' 或 '满五年'
    df_copy['年限_满两年'] = s.isin(['满两年', '满五年']).astype(int)
    
    # 3. 创建 '年限_满五年'
    df_copy['年限_满五年'] = (s == '满五年').astype(int)
    
    # 4. 创建 '年限_未满两年'
    df_copy['年限_未满两年'] = (s == '未满两年').astype(int)
    
    # 5. 创建 '年限_未知'
    df_copy['年限_未知'] = s.isna().astype(int)
    
    print(f"已处理 '{column_name}': 新增 '年限_满两年', '年限_满五年', '年限_未满两年', '年限_未知'。")
    return df_copy

# %%
# --- 应用处理 ---
# 根据您的请求，我们只处理 Price 数据集
input_col_name = '房屋年限'

try:
    # 1. 处理 Price 数据集
    print("\n处理 Price 数据集 '房屋年限'...")
    train_price_processed = process_deed_limit(
        train_price_processed, input_col_name
    )
    test_price_processed = process_deed_limit(
        test_price_processed, input_col_name
    )
    
    print("\nPrice 数据集 '房屋年限' 处理完毕。")
    
except KeyError as e:
    print(f"错误：找不到列 {e}。")
except NameError as e:
    print(f"错误：{e}。请确保 'train_price_processed' 等 DataFrame 已被正确定义。")
    print("这通常意味着您需要从头开始重新运行您的脚本。")

# %%
# --- 验证提取结果 ---
print("\n--- 验证 房价训练集 (train_price) 的房屋年限提取 ---")

# 定义所有预期的新列
cols_to_show = ['房屋年限', '年限_满两年', '年限_满五年', '年限_未满两年', '年限_未知']
cols_to_show_existing = [col for col in cols_to_show if col in train_price_processed.columns]


# 验证代码也需要 try-except 来防止 NameError
try:
    # 1. 检查 '满五年' 样本 (应为: 满两年=1, 满五年=1)
    sample_5y = train_price_processed[train_price[input_col_name] == '满五年']
    if not sample_5y.empty:
        print(f"\n'满五年' 样本 (应为 满两年=1, 满五年=1):")
        print(sample_5y[cols_to_show_existing].head())

    # 2. 检查 '满两年' 样本 (应为: 满两年=1, 满五年=0)
    sample_2y = train_price_processed[train_price[input_col_name] == '满两年']
    if not sample_2y.empty:
        print(f"\n'满两年' 样本 (应为 满两年=1, 满五年=0):")
        print(sample_2y[cols_to_show_existing].head())
        
    # 3. 检查 '未满两年' 样本 (应为: 未满两年=1)
    sample_lt2y = train_price_processed[train_price[input_col_name] == '未满两年']
    if not sample_lt2y.empty:
        print(f"\n'未满两年' 样本 (应为 未满两年=1):")
        print(sample_lt2y[cols_to_show_existing].head())
        
    # 4. 检查 'nan' (空值) 样本 (应为: 未知=1)
    if train_price[input_col_name].isna().any():
        print(f"\n'nan' (空值) 样本 (应为 未知=1):")
        sample_nan = train_price_processed[train_price[input_col_name].isna()]
        print(sample_nan[cols_to_show_existing].head())
        
except NameError:
    print("...验证跳过，因为 'train_price' 或 'train_price_processed' 未在环境中定义。")
except KeyError:
    print(f"...验证跳过，因为原始 'train_price' 中缺少 '{input_col_name}'。")

print(f"\n创建的列: {[col for col in train_price_processed.columns if col.startswith('年限_')]}")

开始提取 房屋年限 特征...

处理 Price 数据集 '房屋年限'...
已处理 '房屋年限': 新增 '年限_满两年', '年限_满五年', '年限_未满两年', '年限_未知'。
已处理 '房屋年限': 新增 '年限_满两年', '年限_满五年', '年限_未满两年', '年限_未知'。

Price 数据集 '房屋年限' 处理完毕。

--- 验证 房价训练集 (train_price) 的房屋年限提取 ---

'满五年' 样本 (应为 满两年=1, 满五年=1):
  房屋年限  年限_满两年  年限_满五年  年限_未满两年  年限_未知
0  满五年       1       1        0      0
1  满五年       1       1        0      0
2  满五年       1       1        0      0
3  满五年       1       1        0      0
4  满五年       1       1        0      0

'满两年' 样本 (应为 满两年=1, 满五年=0):
   房屋年限  年限_满两年  年限_满五年  年限_未满两年  年限_未知
21  满两年       1       0        0      0
28  满两年       1       0        0      0
34  满两年       1       0        0      0
36  满两年       1       0        0      0
38  满两年       1       0        0      0

'未满两年' 样本 (应为 未满两年=1):
     房屋年限  年限_满两年  年限_满五年  年限_未满两年  年限_未知
106  未满两年       0       0        1      0
133  未满两年       0       0        1      0
226  未满两年       0       0        1      0
234  未满两年       0       0        1      0
257  未满两年       0   

In [108]:
train_price_processed['产权所属'].unique()


array(['非共有', '共有'], dtype=object)

In [109]:
#%%
# --------------------------------------------------
# 产权所属 特征提取 (仅 Price)
# --------------------------------------------------
print("开始提取 产权所属 特征...")

import pandas as pd
import numpy as np

def process_ownership_type(df, column_name):
    """
    处理 '产权所属' (格式如 '共有', '非共有', nan)。
    
    1. 将 'nan' 替换为 '未知'。
    2. 使用 pd.get_dummies 创建哑变量，前缀为 '产权所属'。
    
    参数:
    - df: DataFrame
    - column_name: '产权所属'
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 将 'nan' 替换为 '未知'
    s = df_copy[column_name].fillna('未知')
    
    # 2. 创建哑变量
    #    将创建 '产权所属_共有', '产权所属_非共有', '产权所属_未知'
    dummies = pd.get_dummies(s, prefix='产权所属')
    
    # 3. 合并
    df_copy = pd.concat([df_copy, dummies], axis=1)
    
    print(f"已处理 '{column_name}': 新增 {len(dummies.columns)} 个 '产权所属_' 哑变量。")
    return df_copy

# %%
# --- 应用处理 ---
# 根据您的请求，我们只处理 Price 数据集
input_col_name = '产权所属'

try:
    # 1. 处理 Price 数据集
    print("\n处理 Price 数据集 '产权所属'...")
    train_price_processed = process_ownership_type(
        train_price_processed, input_col_name
    )
    test_price_processed = process_ownership_type(
        test_price_processed, input_col_name
    )
    
    print("\nPrice 数据集 '产权所属' 处理完毕。")
    
except KeyError as e:
    print(f"错误：找不到列 {e}。")
except NameError as e:
    print(f"错误：{e}。请确保 'train_price_processed' 等 DataFrame 已被正确定义。")
    print("这通常意味着您需要从头开始重新运行您的脚本。")

# %%
# --- 验证提取结果 ---
print("\n--- 验证 房价训练集 (train_price) 的产权所属提取 ---")

# 定义所有预期的新列
cols_to_show = ['产权所属', '产权所属_共有', '产权所属_非共有']
# 检查 '未知' 列是否被创建
if '产权所属_未知' in train_price_processed.columns:
    cols_to_show.append('产权所属_未知')

# 验证代码也需要 try-except 来防止 NameError
try:
    # 1. 检查 '共有' 样本
    sample_shared = train_price_processed[train_price[input_col_name] == '共有']
    if not sample_shared.empty:
        print(f"\n'共有' 样本 (应标记 '产权所属_共有'):")
        print(sample_shared[cols_to_show].head())
        
    # 2. 检查 '非共有' 样本
    sample_not_shared = train_price_processed[train_price[input_col_name] == '非共有']
    if not sample_not_shared.empty:
        print(f"\n'非共有' 样本 (应标记 '产权所属_非共有'):")
        print(sample_not_shared[cols_to_show].head())

    # 3. 检查 'nan' (空值) 样本 (如果存在)
    if train_price[input_col_name].isna().any():
        print(f"\n'nan' (空值) 样本 (应标记 '产权所属_未知'):")
        sample_nan = train_price_processed[train_price[input_col_name].isna()]
        print(sample_nan[cols_to_show].head())
    elif '产权所属_未知' not in train_price_processed.columns:
        print(f"\n'产权所属' 列中没有 'nan' 值，未创建 '产权所属_未知'。")

        
except NameError:
    print("...验证跳过，因为 'train_price' 或 'train_price_processed' 未在环境中定义。")
except KeyError:
    print(f"...验证跳过，因为原始 'train_price' 中缺少 '{input_col_name}'。")

print(f"\n创建的列: {[col for col in train_price_processed.columns if col.startswith('产权所属_')]}")

开始提取 产权所属 特征...

处理 Price 数据集 '产权所属'...
已处理 '产权所属': 新增 2 个 '产权所属_' 哑变量。
已处理 '产权所属': 新增 2 个 '产权所属_' 哑变量。

Price 数据集 '产权所属' 处理完毕。

--- 验证 房价训练集 (train_price) 的产权所属提取 ---

'共有' 样本 (应标记 '产权所属_共有'):
   产权所属  产权所属_共有  产权所属_非共有
9    共有     True     False
11   共有     True     False
21   共有     True     False
37   共有     True     False
39   共有     True     False

'非共有' 样本 (应标记 '产权所属_非共有'):
  产权所属  产权所属_共有  产权所属_非共有
0  非共有    False      True
1  非共有    False      True
2  非共有    False      True
3  非共有    False      True
4  非共有    False      True

'产权所属' 列中没有 'nan' 值，未创建 '产权所属_未知'。

创建的列: ['产权所属_共有', '产权所属_非共有']


In [110]:
train_price_processed['抵押信息'].unique()


array([nan])

In [ ]:
train_price_processed[ '房屋优势'].unique()

array(['装修、房本满五年', '地铁、装修、房本满五年', '、房本满五年', '地铁、房本满五年', '房本满五年',
       '装修、房本满五年、', '房本满五年、', '装修、房本满两年、', '地铁、房本满两年', '地铁、装修、房本满两年',
       '装修、房本满两年', '地铁、装修、房本满五年、', '房本满两年', '地铁、房本满两年、', '房本满两年、', nan,
       '地铁、房本满五年、', '、房本满五年、', '、房本满两年', '地铁、', '地铁、装修', '地铁、装修、房本满两年、',
       '地铁、装修、', '装修', '地铁', '、房本满两年、', '装修、', '、'], dtype=object)

In [114]:
#只提取地铁就行
#%%
# --------------------------------------------------
# 房屋优势 特征提取 (仅 Price - 只提取地铁)
# --------------------------------------------------
print("开始提取 房屋优势 特征 (仅地铁)...")

import pandas as pd
import numpy as np

def process_subway_advantage(df, column_name):
    """
    处理 '房屋优势' (格式如 '地铁、装修、房本满五年', '、房本满五年、', nan)。
    
    只创建一个新列 '房屋优势_地铁':
    - 1: 如果字符串中包含 '地铁'
    - 0: 如果字符串中不包含 '地铁' 或为 'nan'
    
    参数:
    - df: DataFrame
    - column_name: '房屋优势'
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 'nan' 替换为空字符串 ''
    s = df_copy[column_name].fillna('')
    
    # 2. 检查是否包含 '地铁'
    #    .str.contains('地铁') 会返回 True/False
    #    .astype(int) 将 True 转换为 1, False 转换为 0
    df_copy['房屋优势_地铁'] = s.str.contains('地铁').astype(int)
    
    print(f"已处理 '{column_name}': 新增 '房屋优势_地铁' 二元特征列。")
    return df_copy

# %%
# --- 应用处理 ---
# 根据您的请求，我们只处理 Price 数据集
input_col_name = '房屋优势'

try:
    # 1. 处理 Price 数据集
    print("\n处理 Price 数据集 '房屋优势' (仅地铁)...")
    train_price_processed = process_subway_advantage(
        train_price_processed, input_col_name
    )
    test_price_processed = process_subway_advantage(
        test_price_processed, input_col_name
    )
    
    print("\nPrice 数据集 '房屋优势' 处理完毕。")
    
except KeyError as e:
    print(f"错误：找不到列 {e}。")
except NameError as e:
    print(f"错误：{e}。请确保 'train_price_processed' 等 DataFrame 已被正确定义。")
    print("这通常意味着您需要从头开始重新运行您的脚本。")

# %%
# --- 验证提取结果 ---
print("\n--- 验证 房价训练集 (train_price) 的房屋优势(地铁)提取 ---")

cols_to_show = [input_col_name, '房屋优势_地铁']

# 验证代码也需要 try-except 来防止 NameError
try:
    # 1. 检查 'nan' (空值) 样本 (应为 0)
    if train_price[input_col_name].isna().any():
        print(f"\n'nan' (空值) 样本 (应为 0):")
        sample = train_price_processed[train_price[input_col_name].isna()]
        print(sample[cols_to_show].head())
    
    # 2. 检查 '、房本满五年、' 样本 (应为 0)
    sample_dirty = train_price_processed[train_price[input_col_name] == '、房本满五年、']
    if not sample_dirty.empty:
        print(f"\n'、房本满五年、' 样本 (应为 0):")
        print(sample_dirty[cols_to_show].head())
        
    # 3. 检查 '地铁、装修、房本满两年' 样本 (应为 1)
    sample_full = train_price_processed[train_price[input_col_name] == '地铁、装修、房本满两年']
    if not sample_full.empty:
        print(f"\n'地铁、装修、房本满两年' 样本 (应为 1):")
        print(sample_full[cols_to_show].head())
        
    # 4. 检查 '装修、房本满五年' 样本 (应为 0)
    sample_no_subway = train_price_processed[train_price[input_col_name] == '装修、房本满五年']
    if not sample_no_subway.empty:
        print(f"\n'装修、房本满五年' 样本 (应为 0):")
        print(sample_no_subway[cols_to_show].head())
        
except NameError:
    print("...验证跳过，因为 'train_price' 或 'train_price_processed' 未在环境中定义。")
except KeyError:
    print(f"...验证跳过，因为原始 'train_price' 中缺少 '{input_col_name}'。")

开始提取 房屋优势 特征 (仅地铁)...

处理 Price 数据集 '房屋优势' (仅地铁)...
已处理 '房屋优势': 新增 '房屋优势_地铁' 二元特征列。
已处理 '房屋优势': 新增 '房屋优势_地铁' 二元特征列。

Price 数据集 '房屋优势' 处理完毕。

--- 验证 房价训练集 (train_price) 的房屋优势(地铁)提取 ---

'nan' (空值) 样本 (应为 0):
    房屋优势  房屋优势_地铁
106  NaN        0
300  NaN        0
352  NaN        0
543  NaN        0
634  NaN        0

'、房本满五年、' 样本 (应为 0):
        房屋优势  房屋优势_地铁
122  、房本满五年、        0
365  、房本满五年、        0
369  、房本满五年、        0
428  、房本满五年、        0
439  、房本满五年、        0

'地铁、装修、房本满两年' 样本 (应为 1):
            房屋优势  房屋优势_地铁
34   地铁、装修、房本满两年        1
38   地铁、装修、房本满两年        1
47   地铁、装修、房本满两年        1
51   地铁、装修、房本满两年        1
128  地铁、装修、房本满两年        1

'装修、房本满五年' 样本 (应为 0):
        房屋优势  房屋优势_地铁
0   装修、房本满五年        0
1   装修、房本满五年        0
6   装修、房本满五年        0
14  装修、房本满五年        0
18  装修、房本满五年        0


In [ ]:
train_price_processed['核心卖点'].unique()

array(['此房是南北通透小板楼，户型方正，格局合理', '南北通透商品房自住装修无个税', '房子满五年，商品房，三居室两个卫生间',
       ..., '此房源南北通透好楼层全天采光交通便利', '房子老本能正常过户户型方正采光好视野好交通便利',
       '房本满五税费低，南北通透格局，配套齐全，交通便利。'], dtype=object)

In [117]:
train_rent_processed['租赁方式'].unique()

array(['整租', '合租'], dtype=object)

In [121]:
#%%
# --------------------------------------------------
# 租赁方式 特征提取 (仅 Rent)
# --------------------------------------------------
print("开始提取 租赁方式 特征...")

import pandas as pd
import numpy as np

def process_lease_type(df, column_name):
    """
    处理 '租赁方式' (格式如 '整租', '合租', nan)。
    
    1. 将 'nan' 替换为 '未知'。
    2. 使用 pd.get_dummies 创建哑变量，前缀为 '租赁方式'。
    
    参数:
    - df: DataFrame
    - column_name: '租赁方式'
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 将 'nan' 替换为 '未知'
    s = df_copy[column_name].fillna('未知')
    
    # 2. 创建哑变量
    #    将创建 '租赁方式_整租', '租赁方式_合租', '租赁方式_未知'
    dummies = pd.get_dummies(s, prefix='租赁方式')
    
    # 3. 合并
    df_copy = pd.concat([df_copy, dummies], axis=1)
    
    print(f"已处理 '{column_name}': 新增 {len(dummies.columns)} 个 '租赁方式_' 哑变量。")
    return df_copy

# %%
# --- 应用处理 (仅 Rent) ---
input_col_name_lease = '租赁方式'

try:
    # 1. 处理 Rent 数据集
    print("\n处理 Rent 数据集 '租赁方式'...")
    train_rent_processed = process_lease_type(
        train_rent_processed, input_col_name_lease
    )
    test_rent_processed = process_lease_type(
        test_rent_processed, input_col_name_lease
    )
    
    print("\nRent 数据集 '租赁方式' 处理完毕。")
    
except KeyError as e:
    print(f"错误：找不到列 {e}。")
except NameError as e:
    print(f"错误：{e}。请确保 'train_rent_processed' 等 DataFrame 已被正确定义。")
    print("这通常意味着您需要从头开始重新运行您的脚本。")

# %%
# --- 验证 租赁方式 提取结果 ---
print("\n--- 验证 租金训练集 (train_rent) 的租赁方式提取 ---")

# 定义所有预期的新列
cols_to_show = ['租赁方式', '租赁方式_整租', '租赁方式_合租']
if '租赁方式_未知' in train_rent_processed.columns:
    cols_to_show.append('租赁方式_未知')

try:
    # 1. 检查 '合租' 样本
    sample_share = train_rent_processed[train_rent[input_col_name_lease] == '合租']
    if not sample_share.empty:
        print(f"\n'合租' 样本 (应标记 '租赁方式_合租'):")
        print(sample_share[cols_to_show].head())
        
    # 2. 检查 'nan' (空值) 样本 (如果存在)
    if train_rent[input_col_name_lease].isna().any():
        print(f"\n'nan' (空值) 样本 (应标记 '租赁方式_未知'):")
        sample_nan = train_rent_processed[train_rent[input_col_name_lease].isna()]
        print(sample_nan[cols_to_show].head())
        
except NameError:
    print("...验证跳过，因为 'train_rent' 或 'train_rent_processed' 未在环境中定义。")

#%%
# --------------------------------------------------
# 付款方式 特征提取 (仅 Rent)
# --------------------------------------------------
print("\n开始提取 付款方式 特征...")

def process_payment_type(df, column_name):
    """
    处理 '付款方式' (格式如 '季付价', '年付价', nan, 'https://...').
    
    1. 将 'nan' 替换为 '未知'。
    2. 将 'https://...' 链接替换为 '未知'。
    3. 使用 pd.get_dummies 创建哑变量，前缀为 '付款方式'。
    
    参数:
    - df: DataFrame
    - column_name: '付款方式'
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 将 'nan' 替换为 '未知'
    s = df_copy[column_name].fillna('未知')
    
    # 2. 将 'https://...' 链接替换为 '未知'
    #    .str.startswith('http') 会标记所有 http 和 https
    #    我们使用 .loc 来定位这些行，并将它们的值设为 '未知'
    s.loc[s.str.startswith('http', na=False)] = '未知'
    
    # 3. 创建哑变量
    dummies = pd.get_dummies(s, prefix='付款方式')
    
    # 4. 合并
    df_copy = pd.concat([df_copy, dummies], axis=1)
    
    print(f"已处理 '{column_name}': 新增 {len(dummies.columns)} 个 '付款方式_' 哑变量。")
    return df_copy

# %%
# --- 应用处理 (仅 Rent) ---
input_col_name_payment = '付款方式'

try:
    # 1. 处理 Rent 数据集
    print("\n处理 Rent 数据集 '付款方式'...")
    train_rent_processed = process_payment_type(
        train_rent_processed, input_col_name_payment
    )
    test_rent_processed = process_payment_type(
        test_rent_processed, input_col_name_payment
    )
    
    print("\nRent 数据集 '付款方式' 处理完毕。")
    
except KeyError as e:
    print(f"错误：找不到列 {e}。")
except NameError as e:
    print(f"错误：{e}。请确保 'train_rent_processed' 等 DataFrame 已被正确定义。")

# %%
# --- 验证 付款方式 提取结果 ---
print("\n--- 验证 租金训练集 (train_rent) 的付款方式提取 ---")

# 定义所有预期的新列
cols_to_show_pay = [
    '付款方式', 
    '付款方式_季付价', 
    '付款方式_年付价', 
    '付款方式_半年付价', 
    '付款方式_月付价', 
    '付款方式_双月付价', 
    '付款方式_未知'
]
cols_to_show_pay_existing = [col for col in cols_to_show_pay if col in train_rent_processed.columns]


try:
    # 1. 检查 'https://...' 样本 (应标记 '付款方式_未知')
    sample_link = train_rent_processed[
        train_rent[input_col_name_payment].astype(str).str.startswith('http', na=False)
    ]
    if not sample_link.empty:
        print(f"\n'https://...' 链接样本 (应标记 '付款方式_未知'):")
        print(sample_link[cols_to_show_pay_existing].head())
        
    # 2. 检查 'nan' (空值) 样本 (应标记 '付款方式_未知')
    if train_rent[input_col_name_payment].isna().any():
        print(f"\n'nan' (空值) 样本 (应标记 '付款方式_未知'):")
        sample_nan = train_rent_processed[train_rent[input_col_name_payment].isna()]
        print(sample_nan[cols_to_show_pay_existing].head())

    # 3. 检查 '季付价' 样本
    sample_quarter = train_rent_processed[train_rent[input_col_name_payment] == '季付价']
    if not sample_quarter.empty:
        print(f"\n'季付价' 样本 (应标记 '付款方式_季付价'):")
        print(sample_quarter[cols_to_show_pay_existing].head())
        
except NameError:
    print("...验证跳过，因为 'train_rent' 或 'train_rent_processed' 未在环境中定义。")

开始提取 租赁方式 特征...

处理 Rent 数据集 '租赁方式'...
已处理 '租赁方式': 新增 2 个 '租赁方式_' 哑变量。
已处理 '租赁方式': 新增 2 个 '租赁方式_' 哑变量。

Rent 数据集 '租赁方式' 处理完毕。

--- 验证 租金训练集 (train_rent) 的租赁方式提取 ---

'合租' 样本 (应标记 '租赁方式_合租'):
      租赁方式  租赁方式_整租  租赁方式_合租
19090   合租    False     True
20971   合租    False     True
21168   合租    False     True
21410   合租    False     True
21693   合租    False     True

开始提取 付款方式 特征...

处理 Rent 数据集 '付款方式'...
已处理 '付款方式': 新增 6 个 '付款方式_' 哑变量。
已处理 '付款方式': 新增 6 个 '付款方式_' 哑变量。

Rent 数据集 '付款方式' 处理完毕。

--- 验证 租金训练集 (train_rent) 的付款方式提取 ---

'https://...' 链接样本 (应标记 '付款方式_未知'):
                                 付款方式  付款方式_季付价  付款方式_年付价  付款方式_半年付价  \
17072  https://image1.ljcdn.com/rent-     False     False      False   
22223  https://image1.ljcdn.com/rent-     False     False      False   
38164  https://img.ljcdn.com/usercent     False     False      False   
38550  https://img.ljcdn.com/usercent     False     False      False   
38934  https://img.ljcdn.com/usercent     False     False      False   


In [122]:
train_rent_processed['车位'].unique()

array([nan, '租用车位', '免费使用'], dtype=object)

In [125]:
#%%
# --------------------------------------------------
# 车位 特征提取 (仅 Rent)
# --------------------------------------------------
print("开始提取 车位 特征...")

import pandas as pd
import numpy as np

def process_parking_spot_rent(df, column_name):
    """
    处理 '车位' (格式如 '租用车位', '免费使用', nan)。
    
    1. 将 'nan' 替换为 '无车位'。
    2. 使用 pd.get_dummies 创建哑变量，前缀为 '车位'。
    
    参数:
    - df: DataFrame
    - column_name: '车位'
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 将 'nan' 替换为 '无车位'
    s = df_copy[column_name].fillna('无车位')
    
    # 2. 创建哑变量
    #    将创建 '车位_租用车位', '车位_免费使用', '车位_无车位'
    dummies = pd.get_dummies(s, prefix='车位')
    
    # 3. 合并
    df_copy = pd.concat([df_copy, dummies], axis=1)
    
    print(f"已处理 '{column_name}': 新增 {len(dummies.columns)} 个 '车位_' 哑变量。")
    return df_copy

# %%
# --- 应用处理 (仅 Rent) ---
input_col_name_parking = '车位'

try:
    # 1. 处理 Rent 数据集
    print("\n处理 Rent 数据集 '车位'...")
    train_rent_processed = process_parking_spot_rent(
        train_rent_processed, input_col_name_parking
    )
    test_rent_processed = process_parking_spot_rent(
        test_rent_processed, input_col_name_parking
    )
    
    print("\nRent 数据集 '车位' 处理完毕。")
    
except KeyError as e:
    print(f"错误：找不到列 {e}。")
except NameError as e:
    print(f"错误：{e}。请确保 'train_rent_processed' 等 DataFrame 已被正确定义。")
    print("这通常意味着您需要从头开始重新运行您的脚本。")

# %%
# --- 验证 车位 提取结果 ---
print("\n--- 验证 租金训练集 (train_rent) 的车位提取 ---")

# 定义所有预期的新列
cols_to_show = ['车位', '车位_租用车位', '车位_免费使用', '车位_无车位']
cols_to_show_existing = [col for col in cols_to_show if col in train_rent_processed.columns]


try:
    # 1. 检查 'nan' (空值) 样本 (应标记 '车位_无车位')
    if train_rent[input_col_name_parking].isna().any():
        print(f"\n'nan' (空值) 样本 (应标记 '车位_无车位'):")
        sample_nan = train_rent_processed[train_rent[input_col_name_parking].isna()]
        print(sample_nan[cols_to_show_existing].head())
        
    # 2. 检查 '租用车位' 样本
    sample_rent = train_rent_processed[train_rent[input_col_name_parking] == '租用车位']
    if not sample_rent.empty:
        print(f"\n'租用车位' 样本 (应标记 '车位_租用车位'):")
        print(sample_rent[cols_to_show_existing].head())

    # 3. 检查 '免费使用' 样本
    sample_free = train_rent_processed[train_rent[input_col_name_parking] == '免费使用']
    if not sample_free.empty:
        print(f"\n'免费使用' 样本 (应标记 '车位_免费使用'):")
        print(sample_free[cols_to_show_existing].head())
        
except NameError:
    print("...验证跳过，因为 'train_rent' 或 'train_rent_processed' 未在环境中定义。")
except KeyError:
     print(f"...验证跳过，因为原始 'train_rent' 中缺少 '{input_col_name_parking}'。")

开始提取 车位 特征...

处理 Rent 数据集 '车位'...
已处理 '车位': 新增 3 个 '车位_' 哑变量。
已处理 '车位': 新增 3 个 '车位_' 哑变量。

Rent 数据集 '车位' 处理完毕。

--- 验证 租金训练集 (train_rent) 的车位提取 ---

'nan' (空值) 样本 (应标记 '车位_无车位'):
    车位  车位_租用车位  车位_免费使用  车位_无车位
0  NaN    False    False    True
1  NaN    False    False    True
4  NaN    False    False    True
5  NaN    False    False    True
6  NaN    False    False    True

'租用车位' 样本 (应标记 '车位_租用车位'):
      车位  车位_租用车位  车位_免费使用  车位_无车位
2   租用车位     True    False   False
3   租用车位     True    False   False
15  租用车位     True    False   False
16  租用车位     True    False   False
17  租用车位     True    False   False

'免费使用' 样本 (应标记 '车位_免费使用'):
      车位  车位_租用车位  车位_免费使用  车位_无车位
11  免费使用    False     True   False
12  免费使用    False     True   False
40  免费使用    False     True   False
44  免费使用    False     True   False
64  免费使用    False     True   False


In [ ]:

#用水', '用电', '燃气', '采暖', '租期'
print(train_rent_processed['用水'].unique())
print(train_rent_processed['用电'].unique())
print(train_rent_processed['燃气'].unique())
print(train_rent_processed['采暖'].unique())
print(train_rent_processed['租期'].unique())


['民水' '商水' nan]
['民电' '商电' nan]
['有' '无' nan]
['集中供暖' '自采暖' nan]
['1年' nan '5~12个月' '1~2年' '3年以内' '1年以内' '2年以内' '3年以上' '3~12个月' '3个月以上'
 '2~5个月' '6~12个月' '1~12个月' '2年' '1~3年' '4~12个月' '1年以上' '1~3个月' '3~36个月'
 '2~36个月' '8~12个月' '2~3个月' '3~24个月' '1~4个月' '11~36个月' '6~24个月' '6~36个月'
 '8~24个月' '9~10个月' '9~12个月' '7~12个月' '11~12个月' '7个月' '1~36个月' '6个月以上'
 '8个月' '1~6个月' '8个月以上' '2~24个月' '2~12个月' '1个月以上' '6个月' '3~8个月' '1~24个月'
 '5个月以上' '4~6个月' '3个月' '2~8个月' '6~9个月' '4个月以上' '11个月' '3~11个月' '3~10个月'
 '4~7个月' '8~36个月' '6~11个月' '7~24个月' '2个月以上' '1~2个月' '9个月' '10~11个月'
 '6~7个月' '9~24个月' '5个月' '3~6个月' '4~5个月' '2~6个月' '11~24个月' '2个月' '4个月'
 '2~9个月' '10~24个月' '5~24个月' '4~24个月' '10~36个月' '11个月以上' '10个月以上' '10~12个月'
 '10个月' '1个月以内' '1个月' '3~5个月' '1~8个月' '3~4个月' '9个月以上' '1~11个月']


In [127]:
#%%
# --------------------------------------------------
# 燃气 特征提取 (仅 Rent)
# --------------------------------------------------
print("开始提取 燃气 特征...")

import pandas as pd
import numpy as np

def process_gas_rent(df, column_name):
    """
    处理 '燃气' (格式如 '有', '无', nan)。
    
    1. 将 'nan' 替换为 '未知'。
    2. 使用 pd.get_dummies 创建哑变量，前缀为 '燃气'。
    
    参数:
    - df: DataFrame
    - column_name: '燃气'
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 将 'nan' 替换为 '未知'
    s = df_copy[column_name].fillna('未知')
    
    # 2. 创建哑变量
    #    将创建 '燃气_有', '燃气_无', '燃气_未知'
    dummies = pd.get_dummies(s, prefix='燃气')
    
    # 3. 合并
    df_copy = pd.concat([df_copy, dummies], axis=1)
    
    print(f"已处理 '{column_name}': 新增 {len(dummies.columns)} 个 '燃气_' 哑变量。")
    return df_copy

# %%
# --- 应用处理 (仅 Rent) ---
input_col_name_gas = '燃气'

try:
    # 1. 处理 Rent 数据集
    print("\n处理 Rent 数据集 '燃气'...")
    train_rent_processed = process_gas_rent(
        train_rent_processed, input_col_name_gas
    )
    test_rent_processed = process_gas_rent(
        test_rent_processed, input_col_name_gas
    )
    
    print("\nRent 数据集 '燃气' 处理完毕。")
    
except KeyError as e:
    print(f"错误：找不到列 {e}。")
except NameError as e:
    print(f"错误：{e}。请确保 'train_rent_processed' 等 DataFrame 已被正确定义。")
    print("这通常意味着您需要从头开始重新运行您的脚本。")

# %%
# --- 验证 燃气 提取结果 ---
print("\n--- 验证 租金训练集 (train_rent) 的燃气提取 ---")

# 定义所有预期的新列
cols_to_show = ['燃气', '燃气_有', '燃气_无', '燃气_未知']
cols_to_show_existing = [col for col in cols_to_show if col in train_rent_processed.columns]


try:
    # 1. 检查 'nan' (空值) 样本 (应标记 '燃气_未知')
    if train_rent[input_col_name_gas].isna().any():
        print(f"\n'nan' (空值) 样本 (应标记 '燃气_未知'):")
        sample_nan = train_rent_processed[train_rent[input_col_name_gas].isna()]
        print(sample_nan[cols_to_show_existing].head())
        
    # 2. 检查 '有' 样本
    sample_rent = train_rent_processed[train_rent[input_col_name_gas] == '有']
    if not sample_rent.empty:
        print(f"\n'有' 样本 (应标记 '燃气_有'):")
        print(sample_rent[cols_to_show_existing].head())
        
except NameError:
    print("...验证跳过，因为 'train_rent' 或 'train_rent_processed' 未在环境中定义。")
except KeyError:
     print(f"...验证跳过，因为原始 'train_rent' 中缺少 '{input_col_name_gas}'。")

开始提取 燃气 特征...

处理 Rent 数据集 '燃气'...
已处理 '燃气': 新增 3 个 '燃气_' 哑变量。
已处理 '燃气': 新增 3 个 '燃气_' 哑变量。

Rent 数据集 '燃气' 处理完毕。

--- 验证 租金训练集 (train_rent) 的燃气提取 ---

'nan' (空值) 样本 (应标记 '燃气_未知'):
       燃气   燃气_有   燃气_无  燃气_未知
388   NaN  False  False   True
2121  NaN  False  False   True
3225  NaN  False  False   True
4521  NaN  False  False   True
5203  NaN  False  False   True

'有' 样本 (应标记 '燃气_有'):
  燃气  燃气_有   燃气_无  燃气_未知
0  有  True  False  False
1  有  True  False  False
2  有  True  False  False
4  有  True  False  False
5  有  True  False  False


In [129]:
#%%
# --------------------------------------------------
# 租期 特征提取 (仅 Rent - 数值型)
# --------------------------------------------------
print("\n开始提取 租期 特征 (数值型)...")

import pandas as pd
import numpy as np
import re

def process_lease_term_numeric_rent(df, column_name):
    """
    处理 '租期' (格式如 '1年', '5~12个月', '1~2年', nan)。
    
    1. 提取第一个数字 (num)。
    2. 提取单位 ('年' 或 '月')。
    3. 如果单位是 '年', num = num * 12。
    4. 新列 '租期_月数' = num。
    5. 'nan' 或无法解析的字符串将为 0。
    
    参数:
    - df: DataFrame
    - column_name: '租期'
    
    返回:
    - 带有新特征列 '租期_月数' 的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 转换为字符串，'nan' -> 'nan'
    s = df_copy[column_name].astype(str)
    
    # 2. 定义一个 apply 函数
    def parse_term(s_val):
        if s_val == 'nan':
            return 0
        
        # 正则表达式:
        # (\d+)     : 捕获组 1 - 匹配第一个数字 (num)
        # (?:~.*?)? : 可选的非捕获组，匹配 '~' 和后面的任何内容 (如 '5~12' 中的 '~12')
        # (年|月)   : 捕获组 2 - 匹配单位 '年' 或 '月'
        match = re.search(r'(\d+)(?:~.*?)?(年|月)', s_val)
        
        if not match:
            return 0 # 无法匹配 (如 '未知' 或其他格式)
            
        try:
            num = int(match.group(1))
            unit = match.group(2)
            
            if unit == '年':
                return num * 12
            elif unit == '月':
                return num
            else:
                return 0
                
        except (ValueError, TypeError):
            return 0
    
    # 3. 应用函数
    df_copy['租期_月数'] = s.apply(parse_term)
    
    print(f"已处理 '{column_name}': 新增 '租期_月数' (dtype: {df_copy['租期_月数'].dtype})")
    return df_copy

# %%
# --- 应用处理 (仅 Rent) ---
input_col_name_lease_term = '租期'

try:
    # 1. 处理 Rent 数据集
    print("\n处理 Rent 数据集 '租期' (数值型)...")
    train_rent_processed = process_lease_term_numeric_rent(
        train_rent_processed, input_col_name_lease_term
    )
    test_rent_processed = process_lease_term_numeric_rent(
        test_rent_processed, input_col_name_lease_term
    )
    
    print("\nRent 数据集 '租期' 处理完毕。")
    
except KeyError as e:
    print(f"错误：找不到列 {e}。")
except NameError as e:
    print(f"错误：{e}。请确保 'train_rent_processed' 等 DataFrame 已被正确定义。")
    print("这通常意味着您需要从头开始重新运行您的脚本。")

# %%
# --- 验证 租期 (数值型) 提取结果 ---
print("\n--- 验证 租金训练集 (train_rent) 的租期(数值型)提取 ---")

# 定义所有预期的新列
cols_to_show = ['租期', '租期_月数']


try:
    # 1. 检查 'nan' (空值) 样本 (应为 0)
    if train_rent[input_col_name_lease_term].isna().any():
        print(f"\n'nan' (空值) 样本 (应为 0):")
        sample = train_rent_processed[train_rent[input_col_name_lease_term].isna()]
        print(sample[cols_to_show].head())
        
    # 2. 检查 '1年' 样本 (应为 12)
    sample = train_rent_processed[train_rent[input_col_name_lease_term] == '1年']
    if not sample.empty:
        print(f"\n'1年' 样本 (应为 12):")
        print(sample[cols_to_show].head())

    # 3. 检查 '5~12个月' 样本 (应为 5)
    sample = train_rent_processed[train_rent[input_col_name_lease_term] == '5~12个月']
    if not sample.empty:
        print(f"\n'5~12个月' 样本 (应为 5):")
        print(sample[cols_to_show].head())

    # 4. 检查 '1~2年' 样本 (应为 12)
    sample = train_rent_processed[train_rent[input_col_name_lease_term] == '1~2年']
    if not sample.empty:
        print(f"\n'1~2年' 样本 (应为 12):")
        print(sample[cols_to_show].head())

    # 5. 检查 '3年以上' 样本 (应为 36)
    sample = train_rent_processed[train_rent[input_col_name_lease_term] == '3年以上']
    if not sample.empty:
        print(f"\n'3年以上' 样本 (应为 36):")
        print(sample[cols_to_show].head())

    # 6. 检查 '3~36个月' 样本 (应为 3)
    sample = train_rent_processed[train_rent[input_col_name_lease_term] == '3~36个月']
    if not sample.empty:
        print(f"\n'3~36个月' 样本 (应为 3):")
        print(sample[cols_to_show].head())
        
except NameError:
    print("...验证跳过，因为 'train_rent' 或 'train_rent_processed' 未在环境中定义。")
except KeyError:
     print(f"...验证跳过，因为原始 'train_rent' 中缺少 '{input_col_name_lease_term}'。")


开始提取 租期 特征 (数值型)...

处理 Rent 数据集 '租期' (数值型)...
已处理 '租期': 新增 '租期_月数' (dtype: int64)
已处理 '租期': 新增 '租期_月数' (dtype: int64)

Rent 数据集 '租期' 处理完毕。

--- 验证 租金训练集 (train_rent) 的租期(数值型)提取 ---

'nan' (空值) 样本 (应为 0):
     租期  租期_月数
1   NaN      0
2   NaN      0
18  NaN      0
21  NaN      0
32  NaN      0

'1年' 样本 (应为 12):
    租期  租期_月数
0   1年     12
4   1年     12
7   1年     12
8   1年     12
13  1年     12

'5~12个月' 样本 (应为 5):
         租期  租期_月数
3    5~12个月      5
234  5~12个月      5
250  5~12个月      5
263  5~12个月      5
362  5~12个月      5

'1~2年' 样本 (应为 12):
      租期  租期_月数
5   1~2年     12
6   1~2年     12
10  1~2年     12
16  1~2年     12
25  1~2年     12

'3年以上' 样本 (应为 36):
       租期  租期_月数
36   3年以上     36
105  3年以上     36
148  3年以上     36
160  3年以上     36
185  3年以上     36

'3~36个月' 样本 (应为 3):
           租期  租期_月数
854    3~36个月      3
11600  3~36个月      3
22630  3~36个月      3
44669  3~36个月      3
57414  3~36个月      3


In [126]:
train_rent_processed['配套设施'].unique()

array(['洗衣机、空调、衣柜、热水器、床、宽带', '洗衣机、空调、衣柜、电视、热水器、床、宽带',
       '洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、宽带', '洗衣机、空调、热水器、床、宽带',
       '洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、天然气', '洗衣机、空调、衣柜、电视、冰箱、热水器、床、宽带',
       '洗衣机、空调、冰箱、床、宽带', '暖气、宽带、天然气', '洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气',
       '洗衣机、空调、冰箱、热水器、床、宽带', '洗衣机、空调、电视、热水器、床、宽带', '空调、电视、暖气、天然气',
       '洗衣机、空调、衣柜、电视、冰箱、热水器、床、天然气', '洗衣机、空调、电视、床、宽带',
       '洗衣机、空调、衣柜、冰箱、热水器、床、暖气、天然气', '洗衣机、空调、衣柜、冰箱、热水器、床、宽带',
       '洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、宽带、天然', '空调、床、暖气', '床', '空调、热水器、暖气、天然气',
       '空调、衣柜、暖气、天然气', '洗衣机、衣柜、电视、冰箱、热水器、床、暖气、天然气',
       '洗衣机、空调、衣柜、电视、冰箱、床、暖气、天然气', '洗衣机、空调、电视、冰箱、热水器、床、宽带',
       '洗衣机、空调、衣柜、冰箱、热水器、床、暖气、宽带', '洗衣机、空调、衣柜、电视、冰箱、床、宽带',
       '空调、衣柜、冰箱、床、暖气、天然气', '衣柜、床', '洗衣机、空调、衣柜、冰箱、床、宽带',
       '洗衣机、空调、电视、冰箱、热水器、床、暖气', '洗衣机、空调、衣柜、冰箱、热水器、床、暖气',
       '空调、衣柜、电视、冰箱、床、暖气、天然气', '空调、热水器、床、暖气、宽带', '洗衣机、空调、衣柜、热水器、床、暖气',
       '空调、衣柜、热水器、床、暖气、天然气', '空调、', nan, '洗衣机、空调、衣柜、电视、冰箱、床、暖气',
       '空调、衣柜、电视、冰箱、热水器、床、暖气、天然气', '洗衣机、空调、床、暖气', '洗衣机、空调、衣柜、床、宽带',
       '洗衣机

In [132]:
#%%
# --------------------------------------------------
# 配套设施 特征提取 (仅 Rent)
# --------------------------------------------------
print("开始提取 配套设施 特征...")

from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import numpy as np

def normalize_facilities(s_col):
    """
    辅助函数：清理并分割配套设施字符串。
    1. nan -> ''
    2. '天然' -> '天然气' (修复拼写错误)
    3. 按 '、' 分割
    4. 移除空格和空字符串 (如来自 '空调、')
    5. 返回一个干净的列表
    """
    s = s_col.fillna('')
    
    # 1. 修复 '天然' 拼写错误 (在分割前)
    s = s.str.replace('天然', '天然气', regex=False)
    
    # 2. 按 '、' 分割
    s_split = s.str.split('、')
    
    # 3. 清理每个列表 (移除空格和空字符串)
    def clean_list(facility_list):
        cleaned_set = set()
        for item in facility_list:
            cleaned_item = item.strip()
            if cleaned_item: # 过滤掉空字符串
                cleaned_set.add(cleaned_item)
        return list(cleaned_set)
        
    return s_split.apply(clean_list)

# %%
# --- 步骤 1: 找出所有独特的设施 (并集 N) ---
#    (我们必须在 train_rent 和 test_rent 的 *原始* 数据上执行此操作
#     以确保在所有数据中获得完整的并集)

try:
    print("正在分析所有 Rent 数据集中的配套设施 (寻找并集 N)...")
    
    # 1. 标准化所有四个数据集中的列
    norm_rent_train = normalize_facilities(train_rent['配套设施'])
    norm_rent_test = normalize_facilities(test_rent['配套设施'])
    
    # 2. 将所有列表合并到一个 Series 中
    all_lists = pd.concat([norm_rent_train, norm_rent_test])

    # 3. 遍历所有列表，找到所有唯一的类别
    all_facilities_set = set()
    all_lists.apply(all_facilities_set.update)
    
    # 4. 移除空字符串并排序，创建最终的类别列表
    all_facilities_set.discard('')
    ALL_FACILITIES_LIST = sorted(list(all_facilities_set))
    N_FACILITIES = len(ALL_FACILITIES_LIST)
    
    print(f"已找到 {N_FACILITIES} 个独特的设施 (N = {N_FACILITIES})。")
    print(f"基准设施列表: {ALL_FACILITIES_LIST}")

except NameError as e:
    print(f"错误：{e}。 'train_rent' 或 'test_rent' 未定义。")
    print("请确保从头运行脚本来加载数据。")
    # 如果发生错误，设置一个默认值以允许代码继续
    ALL_FACILITIES_LIST = ['洗衣机', '空调', '衣柜', '电视', '冰箱', '热水器', '床', '暖气', '宽带', '天然气']
    N_FACILITIES = len(ALL_FACILITIES_LIST)
    print(f"警告：使用默认的 {N_FACILITIES} 个设施列表。")

# %%
# --- 步骤 2: 定义主处理函数 (创建哑变量和指数) ---

def process_facilities_rent(df, column_name, all_categories_list):
    """
    处理 '配套设施'。
    
    1. (目标 1) 创建一系列 '配套_' 哑变量。
    2. (目标 2) 创建 '配套设施齐全指数' (k / N)。
    
    参数:
    - df: DataFrame
    - column_name: '配套设施'
    - all_categories_list: 预先计算好的 ALL_FACILITIES_LIST
    
    返回:
    - 带有新特征列的 DataFrame
    """
    df_copy = df.copy()
    
    # 1. 获取标准化的列表 Series
    normalized_lists = normalize_facilities(df_copy[column_name])
    
    # 2. 初始化 Binarizer，强制使用所有类别
    mlb = MultiLabelBinarizer(classes=all_categories_list)
    
    # 3. 转换数据
    dummies_data = mlb.fit_transform(normalized_lists)
    
    # 4. (目标 1) 创建哑变量 DataFrame
    dummy_cols = [f'配套_{c}' for c in mlb.classes_]
    dummies_df = pd.DataFrame(dummies_data, columns=dummy_cols, index=df_copy.index)
    
    # 5. (目标 2) 计算齐全指数
    N_FACILITIES = len(all_categories_list)
    if N_FACILITIES > 0:
        # k = 该行拥有的设施总数 (即 0/1 矩阵的行和)
        k = dummies_df.sum(axis=1)
        # 计算指数 k / N
        df_copy['配套设施齐全指数'] = k / N_FACILITIES
    else:
        df_copy['配套设施齐全指数'] = 0.0 # 避免除以 0
            
    # 6. 合并哑变量
    df_copy = pd.concat([df_copy, dummies_df], axis=1)
    
    print(f"已处理 '{column_name}':")
    print(f"  - 新增 {len(dummy_cols)} 个 '配套_' 哑变量。")
    print(f"  - 新增 '配套设施齐全指数' (基准 N = {N_FACILITIES})。")
    
    return df_copy

# %%
# --- 应用处理 (仅 Rent) ---
input_col_name_facilities = '配套设施'

try:
    # 1. 处理 Rent 数据集
    print("\n处理 Rent 数据集 '配套设施'...")
    train_rent_processed = process_facilities_rent(
        train_rent_processed, input_col_name_facilities, ALL_FACILITIES_LIST
    )
    test_rent_processed = process_facilities_rent(
        test_rent_processed, input_col_name_facilities, ALL_FACILITIES_LIST
    )
    
    print("\nRent 数据集 '配套设施' 处理完毕。")
    
except KeyError as e:
    print(f"错误：找不到列 {e}。")
except NameError as e:
    print(f"错误：{e}。请确保 'train_rent_processed' 等 DataFrame 已被正确定义。")
    print("这通常意味着您需要从头开始重新运行您的脚本。")

# %%
# --- 验证 配套设施 提取结果 ---
print("\n--- 验证 租金训练集 (train_rent) 的配套设施提取 ---")

# (假设 N=10, 列表为 ['床', '天然气', '电视', '暖气', '热水器', '冰箱', '洗衣机', '宽带', '空调', '衣柜'])
cols_to_show = [
    '配套设施',
    '配套设施齐全指数',
    '配套_床',
    '配套_天然气',
    '配套_空调',
]

try:
    # 1. 检查 'nan' (空值) 样本 (应为 0.0)
    if train_rent[input_col_name_facilities].isna().any():
        print(f"\n'nan' (空值) 样本 (指数应为 0.0):")
        sample = train_rent_processed[train_rent[input_col_name_facilities].isna()]
        print(sample[cols_to_show].head())
        
    # 2. 检查 '床' 样本 (指数应为 1/N)
    sample_bed = train_rent_processed[train_rent[input_col_name_facilities] == '床']
    if not sample_bed.empty:
        print(f"\n'床' 样本 (指数应为 1 / {N_FACILITIES} = {1/N_FACILITIES:.2f}):")
        print(sample_bed[cols_to_show].head())

    # 3. 检查 '空调、' 脏数据 (指数应为 1/N)
    sample_dirty_ac = train_rent_processed[train_rent[input_col_name_facilities] == '空调、']
    if not sample_dirty_ac.empty:
        print(f"\n'空调、' (脏数据) 样本 (指数应为 1 / {N_FACILITIES} = {1/N_FACILITIES:.2f}):")
        print(sample_dirty_ac[cols_to_show].head())
        
    # 4. 检查 '天然' (拼写错误) 样本
    sample_typo_gas = train_rent_processed[
        train_rent[input_col_name_facilities] == '洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、宽带、天然'
    ]
    if not sample_typo_gas.empty:
        print(f"\n'天然' (拼写错误) 样本 (应正确标记 '配套_天然气' 且指数为 {N_FACILITIES}/{N_FACILITIES}):")
        cols_gas = ['配套设施', '配套设施齐全指数', '配套_天然气']
        print(sample_typo_gas[cols_gas].head())

    # 5. 检查一个全覆盖的样本 (指数应为 1.0)
    sample_full_str = '洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、宽带、天然气'
    sample_full = train_rent_processed[
        (train_rent['配套设施'] == sample_full_str) | 
        (train_rent['配套设施'] == '洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、天然气') # 修复过的版本
    ]
    if not sample_full.empty:
        print(f"\n'全设施' 样本 (指数应为 {N_FACILITIES}/{N_FACILITIES} = 1.0):")
        print(sample_full[cols_to_show].head())

except NameError:
    print("...验证跳过，因为 'train_rent' 或 'train_rent_processed' 未在环境中定义。")
except KeyError:
     print(f"...验证跳过，因为原始 'train_rent' 中缺少 '{input_col_name_facilities}'。")

开始提取 配套设施 特征...
正在分析所有 Rent 数据集中的配套设施 (寻找并集 N)...
已找到 11 个独特的设施 (N = 11)。
基准设施列表: ['冰箱', '天然气', '天然气气', '宽带', '床', '暖气', '洗衣机', '热水器', '电视', '空调', '衣柜']

处理 Rent 数据集 '配套设施'...
已处理 '配套设施':
  - 新增 11 个 '配套_' 哑变量。
  - 新增 '配套设施齐全指数' (基准 N = 11)。
已处理 '配套设施':
  - 新增 11 个 '配套_' 哑变量。
  - 新增 '配套设施齐全指数' (基准 N = 11)。

Rent 数据集 '配套设施' 处理完毕。

--- 验证 租金训练集 (train_rent) 的配套设施提取 ---

'nan' (空值) 样本 (指数应为 0.0):
    配套设施  配套设施齐全指数  配套_床  配套_天然气  配套_空调
120  NaN       0.0     0       0      0
231  NaN       0.0     0       0      0
240  NaN       0.0     0       0      0
283  NaN       0.0     0       0      0
309  NaN       0.0     0       0      0

'床' 样本 (指数应为 1 / 11 = 0.09):
    配套设施  配套设施齐全指数  配套_床  配套_天然气  配套_空调
51     床  0.090909     1       0      0
52     床  0.090909     1       0      0
185    床  0.090909     1       0      0
202    床  0.090909     1       0      0
232    床  0.090909     1       0      0

'空调、' (脏数据) 样本 (指数应为 1 / 11 = 0.09):
    配套设施  配套设施齐全指数  配套_床  配套_天然气  配套_空调
117  空调、  0.090

In [131]:
'配套设施' in train_rent_processed.columns

True

In [133]:
#%%
# --------------------------------------------------
# 保存处理后的数据
# --------------------------------------------------
import os
import pandas as pd

# 1. 定义新文件夹的名称
output_folder = 'wzy_processed_data'

# 2. 创建新文件夹 (如果它不存在)
try:
    os.makedirs(output_folder, exist_ok=True)
    print(f"文件夹 '{output_folder}' 已创建或已存在。")
except OSError as e:
    print(f"创建文件夹 '{output_folder}' 时出错: {e}")
    # 如果无法创建文件夹，后续保存会失败，所以最好停止或处理错误
    # 这里我们简单地打印错误信息

# 3. 定义要保存的文件名和对应的 DataFrame
files_to_save = {
    'train_price_processed.csv': 'train_price_processed',
    'test_price_processed.csv': 'test_price_processed',
    'train_rent_processed.csv': 'train_rent_processed',
    'test_rent_processed.csv': 'test_rent_processed'
}

# 4. 循环保存每个文件
for filename, df_name in files_to_save.items():
    try:
        # 检查 DataFrame 是否存在于当前环境中
        if df_name in locals():
            df_to_save = locals()[df_name]
            # 构建完整的文件路径
            filepath = os.path.join(output_folder, filename)
            # 保存为 CSV, 不保存索引列
            df_to_save.to_csv(filepath, index=False, encoding='utf-8-sig') # 使用 utf-8-sig 确保中文在 Excel 中正确显示
            print(f"✓ 文件已保存: {filepath}")
        else:
            print(f"✗ 错误: DataFrame '{df_name}' 未定义，无法保存 '{filename}'。")
            print("  请确保您已按顺序运行了所有处理步骤。")
            
    except Exception as e:
        print(f"✗ 保存文件 '{filename}' 时出错: {e}")

print("\n数据保存过程完成。")

文件夹 'wzy_processed_data' 已创建或已存在。
✓ 文件已保存: wzy_processed_data\train_price_processed.csv
✓ 文件已保存: wzy_processed_data\test_price_processed.csv
✓ 文件已保存: wzy_processed_data\train_rent_processed.csv
✓ 文件已保存: wzy_processed_data\test_rent_processed.csv

数据保存过程完成。
